## Get Started with Spark NLP for Healthcare

## Getting the keys and installation

1. In order to get trial keys for Spark NLP for Healthcare
, fill the form at https://www.johnsnowlabs.com/spark-nlp-try-free/ and you will get your keys to your email in a few minutes.

2. On a new cluster or existing one

  - add the following to the `Advanced Options -> Spark` tab, in `Spark.Config` box:

    ```bash
    spark.local.dir /var
    spark.kryoserializer.buffer.max 1000M
    spark.serializer org.apache.spark.serializer.KryoSerializer
    ```
  - add the following to the `Advanced Options -> Spark` tab, in `Environment Variables` box:

    ```bash
    AWS_ACCESS_KEY_ID=xxx
    AWS_SECRET_ACCESS_KEY=yyy
    SPARK_NLP_LICENSE=zzz
    ```

3. Download the followings with AWS CLI to your local computer

    `$ aws s3 cp --region us-east-2 s3://pypi.johnsnowlabs.com/$jsl_secret/spark-nlp-jsl-$jsl_version.jar spark-nlp-jsl-$jsl_version.jar`

    `$ aws s3 cp --region us-east-2 s3://pypi.johnsnowlabs.com/$jsl_secret/spark-nlp-jsl/spark_nlp_jsl-$jsl_version-py3-none-any.whl spark_nlp_jsl-$jsl_version-py3-none-any.whl` 

4. In `Libraries` tab inside your cluster:

 - Install New -> PyPI -> `spark-nlp==$public_version` -> Install
 - Install New -> Maven -> Coordinates -> `com.johnsnowlabs.nlp:spark-nlp_2.12:$public_version` -> Install

 - add following jars for the Healthcare library that you downloaded above:
        - Install New -> Python Whl -> upload `spark_nlp_jsl-$jsl_version-py3-none-any.whl`

        - Install New -> Jar -> upload `spark-nlp-jsl-$jsl_version.jar`

5. Now you can attach your notebook to the cluster and use Spark NLP!

For more information, see 

  https://nlp.johnsnowlabs.com/docs/en/install#databricks-support

  https://nlp.johnsnowlabs.com/docs/en/licensed_install#install-spark-nlp-for-healthcare-on-databricks
  
The follwing notebook is prepared and tested on **r2.2xlarge at 8.0 (includes Apache Spark 3.1.1, Scala 2.12)** on Databricks

In order to get more detailed examples, please check this repository : https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/tutorials/Certification_Trainings/Healthcare/databricks_notebooks

Let's import the libraries which we will use in the following cells.

In [0]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
#nlp.install()

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

import os
import json
import string
import numpy as np
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

spark

Out[5]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

# 1.Clinical Named Entity Recognition (NER)

The deep neural network architecture for NER model in
Spark NLP is BiLSTM-CNN-Char framework. a slightly modified version of the architecture proposed by Jason PC Chiu and Eric Nichols ([Named Entity Recognition with Bidirectional LSTM-CNNs
](https://arxiv.org/abs/1511.08308)). It is a neural network architecture that
automatically detects word and character-level features using a
hybrid bidirectional LSTM and CNN architecture, eliminating
the need for most feature engineering steps.

  In the original framework, the CNN extracts a fixed length
feature vector from character-level features. For each word,
these vectors are concatenated and fed to the BLSTM network
and then to the output layers. They employed a stacked
bi-directional recurrent neural network with long short-term
memory units to transform word features into named entity
tag scores. The extracted features of each word are fed into a
forward LSTM network and a backward LSTM network. The
output of each network at each time step is decoded by a linear
layer and a log-softmax layer into log-probabilities for each tag
category. These two vectors are then simply added together to
produce the final output. In the architecture of the proposed framework in the original paper, 50-dimensional pretrained word
embeddings is used for word features, 25-dimension character
embeddings is used for char features, and capitalization features
(allCaps, upperInitial, lowercase, mixedCaps, noinfo) are used
for case features.

## Blogposts and videos:

- [How to Setup Spark NLP for HEALTHCARE on UBUNTU - Video](https://www.youtube.com/watch?v=yKnF-_oz0GE)

- [Named Entity Recognition (NER) with BERT in Spark NLP](https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77)

- [State of the art Clinical Named Entity Recognition in Spark NLP - Youtube](https://www.youtube.com/watch?v=YM-e4eOiQ34)

- [Named Entity Recognition for Healthcare with SparkNLP NerDL and NerCRF](https://medium.com/spark-nlp/named-entity-recognition-for-healthcare-with-sparknlp-nerdl-and-nercrf-a7751b6ad571)

- [Named Entity Recognition for Clinical Text](https://medium.com/atlas-research/ner-for-clinical-text-7c73caddd180)

For more details about NER with SparkNLP, please visit [SparkNLP Workshop Repo](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.Clinical_Named_Entity_Recognition_Model.ipynb)

## Clinical NER Pipeline (with pretrained models)

SparkNLP has more than a hundred pretrained clinical NER pipelines which are trained with various domain-specific datasets and have various architectures. <br/>

**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?task=Named+Entity+Recognition&edition=Spark+NLP+for+Healthcare)**






## Clinical NER Models
|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [jsl_ner_wip_clinical](https://nlp.johnsnowlabs.com/2021/03/31/jsl_ner_wip_clinical_en.html)  | 2| [jsl_ner_wip_greedy_clinical](https://nlp.johnsnowlabs.com/2021/03/31/jsl_ner_wip_greedy_clinical_en.html)  | 3| [jsl_ner_wip_modifier_clinical](https://nlp.johnsnowlabs.com/2021/04/01/jsl_ner_wip_modifier_clinical_en.html)  | 4| [jsl_rd_ner_wip_greedy_clinical](https://nlp.johnsnowlabs.com/2021/04/01/jsl_rd_ner_wip_greedy_clinical_en.html)  |
| 5| [ner_abbreviation_clinical](https://nlp.johnsnowlabs.com/2021/12/30/ner_abbreviation_clinical_en.html)  | 6| [ner_ade_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinical_en.html)  | 7| [ner_ade_clinicalbert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinicalbert_en.html)  | 8| [ner_ade_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_healthcare_en.html)  |
| 9| [ner_anatomy](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_en.html)  | 10| [ner_anatomy_coarse](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_coarse_en.html)  | 11| [ner_anatomy_coarse_en](https://nlp.johnsnowlabs.com/2020/11/04/ner_anatomy_coarse_en.html)  | 12| [ner_anatomy_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_anatomy_en.html)  |
| 13| [ner_aspect_based_sentiment](https://nlp.johnsnowlabs.com/2021/03/31/ner_aspect_based_sentiment_en.html)  | 14| [ner_bacterial_species](https://nlp.johnsnowlabs.com/2021/04/01/ner_bacterial_species_en.html)  | 15| [ner_biomarker](https://nlp.johnsnowlabs.com/2021/11/26/ner_biomarker_en.html)  | 16| [ner_biomedical_bc2gm](https://nlp.johnsnowlabs.com/2022/05/11/ner_biomedical_bc2gm_en_2_4.html)  |
| 17| [ner_bionlp](https://nlp.johnsnowlabs.com/2021/03/31/ner_bionlp_en.html)  | 18| [ner_bionlp_en](https://nlp.johnsnowlabs.com/2020/01/30/ner_bionlp_en.html)  | 19| [ner_cancer_genetics](https://nlp.johnsnowlabs.com/2021/03/31/ner_cancer_genetics_en.html)  | 20| [ner_cellular](https://nlp.johnsnowlabs.com/2021/03/31/ner_cellular_en.html)  |
| 21| [ner_cellular_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_cellular_en.html)  | 22| [ner_chemd_clinical](https://nlp.johnsnowlabs.com/2021/11/04/ner_chemd_clinical_en.html)  | 23| [ner_chemicals](https://nlp.johnsnowlabs.com/2021/04/01/ner_chemicals_en.html)  | 24| [ner_chemprot_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_chemprot_clinical_en.html)  |
| 25| [ner_chexpert](https://nlp.johnsnowlabs.com/2021/09/30/ner_chexpert_en.html)  | 26| [ner_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_clinical_en.html)  | 27| [ner_clinical_en](https://nlp.johnsnowlabs.com/2020/01/30/ner_clinical_en.html)  | 28| [ner_clinical_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_clinical_large_en.html)  |
| 29| [ner_clinical_large_en](https://nlp.johnsnowlabs.com/2020/05/23/ner_clinical_large_en.html)  | 30| [ner_clinical_trials_abstracts](https://nlp.johnsnowlabs.com/2022/06/22/ner_clinical_trials_abstracts_en_3_0.html)  | 31| [ner_covid_trials](https://nlp.johnsnowlabs.com/2022/10/19/ner_covid_trials_en.html)  | 32| [ner_crf](https://nlp.johnsnowlabs.com/2020/01/28/ner_crf_en.html)  |
| 33| [ner_deid_augmented](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_augmented_en.html)  | 34| [ner_deid_enriched](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_enriched_en.html)  | 35| [ner_deid_generic_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_generic_glove_en.html)  | 36| [ner_deid_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_large_en.html)  |
| 37| [ner_deid_sd](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_en.html)  | 38| [ner_deid_sd_large](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_large_en.html)  | 39| [ner_deid_subentity_augmented](https://nlp.johnsnowlabs.com/2021/09/03/ner_deid_subentity_augmented_en.html)  | 40| [ner_deid_subentity_augmented_i2b2](https://nlp.johnsnowlabs.com/2021/11/29/ner_deid_subentity_augmented_i2b2_en.html)  |
| 41| [ner_deid_subentity_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_subentity_glove_en.html)  | 42| [ner_deid_synthetic](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_synthetic_en.html)  | 43| [ner_deidentify_dl](https://nlp.johnsnowlabs.com/2021/03/31/ner_deidentify_dl_en.html)  | 44| [ner_diseases](https://nlp.johnsnowlabs.com/2021/03/31/ner_diseases_en.html)  |
| 45| [ner_diseases_en](https://nlp.johnsnowlabs.com/2020/03/25/ner_diseases_en.html)  | 46| [ner_diseases_large](https://nlp.johnsnowlabs.com/2021/04/01/ner_diseases_large_en.html)  | 47| [ner_drugprot_clinical](https://nlp.johnsnowlabs.com/2021/12/20/ner_drugprot_clinical_en.html)  | 48| [ner_drugs](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_en.html)  |
| 49| [ner_drugs_en](https://nlp.johnsnowlabs.com/2020/03/25/ner_drugs_en.html)  | 50| [ner_drugs_greedy](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_greedy_en.html)  | 51| [ner_drugs_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_large_en.html)  | 52| [ner_drugs_large_en](https://nlp.johnsnowlabs.com/2021/01/29/ner_drugs_large_en.html)  |
| 53| [ner_events_admission_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_events_admission_clinical_en.html)  | 54| [ner_events_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_events_clinical_en.html)  | 55| [ner_events_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_events_healthcare_en.html)  | 56| [ner_financial_contract](https://nlp.johnsnowlabs.com/2021/04/01/ner_financial_contract_en.html)  |
| 57| [ner_genetic_variants](https://nlp.johnsnowlabs.com/2021/06/25/ner_genetic_variants_en.html)  | 58| [ner_healthcare](https://nlp.johnsnowlabs.com/2021/04/21/ner_healthcare_en.html)  | 59| [ner_healthcare_en](https://nlp.johnsnowlabs.com/2020/03/26/ner_healthcare_en.html)  | 60| [ner_human_phenotype_gene_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_human_phenotype_gene_clinical_en.html)  |
| 61| [ner_human_phenotype_go_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_human_phenotype_go_clinical_en.html)  | 62| [ner_jsl](https://nlp.johnsnowlabs.com/2022/10/19/ner_jsl_en.html)  | 63| [ner_jsl_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_jsl_en.html)  | 64| [ner_jsl_enriched](https://nlp.johnsnowlabs.com/2021/10/22/ner_jsl_enriched_en.html)  |
| 65| [ner_jsl_enriched_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_jsl_enriched_en.html)  | 66| [ner_jsl_greedy](https://nlp.johnsnowlabs.com/2021/06/24/ner_jsl_greedy_en.html)  | 67| [ner_jsl_slim](https://nlp.johnsnowlabs.com/2021/08/13/ner_jsl_slim_en.html)  | 68| [ner_living_species](https://nlp.johnsnowlabs.com/2022/06/22/ner_living_species_en_3_0.html)  |
| 69| [ner_measurements_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_measurements_clinical_en.html)  | 70| [ner_medmentions_coarse](https://nlp.johnsnowlabs.com/2021/04/01/ner_medmentions_coarse_en.html)  | 71| [ner_nature_nero_clinical](https://nlp.johnsnowlabs.com/2022/02/08/ner_nature_nero_clinical_en.html)  | 72| [ner_nihss](https://nlp.johnsnowlabs.com/2021/11/15/ner_nihss_en.html)  |
| 73| [ner_oncology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_en.html)  | 74| [ner_oncology_anatomy_general](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_anatomy_general_en.html)  | 75| [ner_oncology_anatomy_granular](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_anatomy_granular_en.html)  | 76| [ner_oncology_biomarker](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_biomarker_en.html)  |
| 77| [ner_oncology_demographics](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_demographics_en.html)  | 78| [ner_oncology_diagnosis](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_diagnosis_en.html)  | 79| [ner_oncology_posology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_posology_en.html)  | 80| [ner_oncology_response_to_treatment](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_response_to_treatment_en.html)  |
| 81| [ner_oncology_test](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_test_en.html)  | 82| [ner_oncology_therapy](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_therapy_en.html)  | 83| [ner_oncology_tnm](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_tnm_en.html)  | 84| [ner_oncology_unspecific_posology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_unspecific_posology_en.html)  |
| 85| [ner_pathogen](https://nlp.johnsnowlabs.com/2022/06/28/ner_pathogen_en_3_0.html)  | 86| [ner_posology](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_en.html)  | 87| [ner_posology_en](https://nlp.johnsnowlabs.com/2020/04/15/ner_posology_en.html)  | 88| [ner_posology_experimental](https://nlp.johnsnowlabs.com/2021/09/01/ner_posology_experimental_en.html)  |
| 89| [ner_posology_greedy](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_greedy_en.html)  | 90| [ner_posology_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_healthcare_en.html)  | 91| [ner_posology_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_large_en.html)  | 92| [ner_posology_small](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_small_en.html)  |
| 93| [ner_radiology](https://nlp.johnsnowlabs.com/2021/03/31/ner_radiology_en.html)  | 94| [ner_radiology_wip_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_radiology_wip_clinical_en.html)  | 95| [ner_risk_factors](https://nlp.johnsnowlabs.com/2021/03/31/ner_risk_factors_en.html)  | 96| [ner_risk_factors_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_risk_factors_en.html)  |
| 97| [ner_sdoh_mentions](https://nlp.johnsnowlabs.com/2022/12/18/ner_sdoh_mentions_en.html)  | 98| [ner_supplement_clinical](https://nlp.johnsnowlabs.com/2022/02/01/ner_supplement_clinical_en.html)  | 99| [nerdl_tumour_demo](https://nlp.johnsnowlabs.com/2021/04/01/nerdl_tumour_demo_en.html)  | 100| [ner_vop_slim_wip](https://nlp.johnsnowlabs.com/2023/02/25/ner_vop_slim_wip_en.html)  |
| 101| [ner_sdoh_substance_usage_wip](https://nlp.johnsnowlabs.com/2023/02/23/ner_sdoh_substance_usage_wip_en.html)  | 102| [ner_sdoh_access_to_healthcare_wip](https://nlp.johnsnowlabs.com/2023/02/24/ner_sdoh_access_to_healthcare_wip_en.html)  | 103| [ner_sdoh_community_condition_wip](https://nlp.johnsnowlabs.com/2023/02/24/ner_sdoh_community_condition_wip_en.html)  | 104| [ner_sdoh_health_behaviours_problems_wip](https://nlp.johnsnowlabs.com/2023/02/24/ner_sdoh_health_behaviours_problems_wip_en.html)  |
| 105| [ner_sdoh_demographics_wip](https://nlp.johnsnowlabs.com/2023/02/10/ner_sdoh_demographics_wip_en.html)  |  106| [ner_sdoh_income_social_status_wip](https://nlp.johnsnowlabs.com/2023/02/10/ner_sdoh_income_social_status_wip_en.html)  |  107| [ner_sdoh_social_environment_wip](https://nlp.johnsnowlabs.com/2023/02/10/ner_sdoh_social_environment_wip_en.html)  |  108| [ner_sdoh_wip](https://nlp.johnsnowlabs.com/2023/02/11/ner_sdoh_wip_en.html)  |
| 109| [ner_eu_clinical_case](https://nlp.johnsnowlabs.com/2023/02/02/ner_eu_clinical_case_eu.html)  | 110| [ner_eu_clinical_condition](https://nlp.johnsnowlabs.com/2023/02/06/ner_eu_clinical_condition_en.html)  | 111| [ner_oncology_anatomy_general_healthcare](https://nlp.johnsnowlabs.com/2023/01/11/ner_oncology_anatomy_general_healthcare_en.html)  | 112| [ner_oncology_biomarker_healthcare](https://nlp.johnsnowlabs.com/2023/01/11/ner_oncology_biomarker_healthcare_en.html)  |
| 113| [ner_oncology_unspecific_posology_healthcare](https://nlp.johnsnowlabs.com/2023/01/11/ner_oncology_unspecific_posology_healthcare_en.html)  |



- **BioBert NER Models**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [jsl_ner_wip_greedy_biobert](https://nlp.johnsnowlabs.com/2021/07/26/jsl_ner_wip_greedy_biobert_en.html)  | 2| [jsl_rd_ner_wip_greedy_biobert](https://nlp.johnsnowlabs.com/2021/07/26/jsl_rd_ner_wip_greedy_biobert_en.html)  | 3| [ner_ade_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_biobert_en.html)  | 4| [ner_anatomy_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_anatomy_biobert_en.html)  |
| 5| [ner_anatomy_coarse_biobert](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_coarse_biobert_en.html)  | 6| [ner_bionlp_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_bionlp_biobert_en.html)  | 7| [ner_cellular_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_cellular_biobert_en.html)  | 8| [ner_chemprot_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_chemprot_biobert_en.html)  |
| 9| [ner_clinical_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_clinical_biobert_en.html)  | 10| [ner_deid_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_biobert_en.html)  | 11| [ner_deid_enriched_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_enriched_biobert_en.html)  | 12| [ner_diseases_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_diseases_biobert_en.html)  |
| 13| [ner_events_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_events_biobert_en.html)  | 14| [ner_human_phenotype_gene_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_human_phenotype_gene_biobert_en.html)  | 15| [ner_human_phenotype_go_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_human_phenotype_go_biobert_en.html)  | 16| [ner_jsl_biobert](https://nlp.johnsnowlabs.com/2021/09/05/ner_jsl_biobert_en.html)  |
| 17| [ner_jsl_enriched_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_jsl_enriched_biobert_en.html)  | 18| [ner_jsl_greedy_biobert](https://nlp.johnsnowlabs.com/2021/08/13/ner_jsl_greedy_biobert_en.html)  | 19| [ner_living_species_biobert](https://nlp.johnsnowlabs.com/2022/06/22/ner_living_species_biobert_en_3_0.html)  | 20| [ner_posology_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_biobert_en.html)  |
| 21| [ner_posology_large_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_large_biobert_en.html)  | 22| [ner_risk_factors_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_risk_factors_biobert_en.html)  | 23| []()| 24| []()|



- **BertForTokenClassification Clinical NER models**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [bert_token_classifier_ade_tweet_binary](https://nlp.johnsnowlabs.com/2022/07/29/bert_token_classifier_ade_tweet_binary_en_3_0.html)  | 2| [bert_token_classifier_drug_development_trials](https://nlp.johnsnowlabs.com/2022/03/22/bert_token_classifier_drug_development_trials_en_3_0.html)  | 3| [bert_token_classifier_ner_ade](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_ade_en.html)  | 4| [bert_token_classifier_ner_ade_binary](https://nlp.johnsnowlabs.com/2022/07/27/bert_token_classifier_ner_ade_binary_en_3_0.html)  |
| 5| [bert_token_classifier_ner_anatem](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_anatem_en_3_0.html)  | 6| [bert_token_classifier_ner_anatomy](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_anatomy_en.html)  | 7| [bert_token_classifier_ner_bacteria](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_bacteria_en.html)  | 8| [bert_token_classifier_ner_bc2gm_gene](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc2gm_gene_en_3_0.html)  |
| 9| [bert_token_classifier_ner_bc4chemd_chemicals](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc4chemd_chemicals_en_3_0.html)  | 10| [bert_token_classifier_ner_bc5cdr_chemicals](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc5cdr_chemicals_en_3_0.html)  | 11| [bert_token_classifier_ner_bc5cdr_disease](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc5cdr_disease_en_3_0.html)  | 12| [bert_token_classifier_ner_bionlp](https://nlp.johnsnowlabs.com/2021/11/03/bert_token_classifier_ner_bionlp_en.html)  |
| 13| [bert_token_classifier_ner_cellular](https://nlp.johnsnowlabs.com/2021/11/03/bert_token_classifier_ner_cellular_en.html)  | 14| [bert_token_classifier_ner_chemicals](https://nlp.johnsnowlabs.com/2021/10/19/bert_token_classifier_ner_chemicals_en.html)  | 15| [bert_token_classifier_ner_chemprot](https://nlp.johnsnowlabs.com/2021/10/19/bert_token_classifier_ner_chemprot_en.html)  | 16| [bert_token_classifier_ner_clinical](https://nlp.johnsnowlabs.com/2022/01/06/bert_token_classifier_ner_clinical_en.html)  |
| 17| [bert_token_classifier_ner_clinical_trials_abstracts](https://nlp.johnsnowlabs.com/2022/06/29/bert_token_classifier_ner_clinical_trials_abstracts_en_3_0.html)  | 18| [bert_token_classifier_ner_deid](https://nlp.johnsnowlabs.com/2021/09/13/bert_token_classifier_ner_deid_en.html)  | 19| [bert_token_classifier_ner_drugs](https://nlp.johnsnowlabs.com/2021/09/20/bert_token_classifier_ner_drugs_en.html)  | 20| [bert_token_classifier_ner_jnlpba_cellular](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_jnlpba_cellular_en_3_0.html)  |
| 21| [bert_token_classifier_ner_jsl](https://nlp.johnsnowlabs.com/2021/09/16/bert_token_classifier_ner_jsl_en.html)  | 22| [bert_token_classifier_ner_jsl_slim](https://nlp.johnsnowlabs.com/2021/09/24/bert_token_classifier_ner_jsl_slim_en.html)  | 23| [bert_token_classifier_ner_linnaeus_species](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_linnaeus_species_en_3_0.html)  | 24| [bert_token_classifier_ner_living_species](https://nlp.johnsnowlabs.com/2022/06/26/bert_token_classifier_ner_living_species_en_3_0.html)  |
| 25| [bert_token_classifier_ner_ncbi_disease](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_ncbi_disease_en_3_0.html)  | 26| [bert_token_classifier_ner_pathogen](https://nlp.johnsnowlabs.com/2022/07/28/bert_token_classifier_ner_pathogen_en_3_0.html)  | 27| [bert_token_classifier_ner_species](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_species_en_3_0.html)  | 28| [bert_token_classifier_ner_supplement](https://nlp.johnsnowlabs.com/2022/02/09/bert_token_classifier_ner_supplement_en.html)  |






**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?task=Named+Entity+Recognition&edition=Spark+NLP+for+Healthcare)**

Let's start creating an example NER Pipeline. <br/>
In this pipeline, we will use `ner_jsl` pretrained model. By using this pretrained model, we can extract about 80 clinical entity labels by changing just only the ner model name in our pipeline.

**Entities**

| | | | | |
|-|-|-|-|-|
|Injury_or_Poisoning|Direction|Test|Admission_Discharge|Death_Entity|
|Relationship_Status|Duration|Respiration|Hyperlipidemia|Birth_Entity|
|Age|Labour_Delivery|Family_History_Header|BMI|Temperature|
|Alcohol|Kidney_Disease|Oncological|Medical_History_Header|Cerebrovascular_Disease|
|Oxygen_Therapy|O2_Saturation|Psychological_Condition|Heart_Disease|Employment|
|Obesity|Disease_Syndrome_Disorder|Pregnancy|ImagingFindings|Procedure|
|Medical_Device|Race_Ethnicity|Section_Header|Symptom|Treatment|
|Substance|Route|Drug_Ingredient|Blood_Pressure|Diet|
|External_body_part_or_region|LDL|VS_Finding|Allergen|EKG_Findings|
|Imaging_Technique|Triglycerides|RelativeTime|Gender|Pulse|
|Social_History_Header|Substance_Quantity|Diabetes|Modifier|Internal_organ_or_component|
|Clinical_Dept|Form|Drug_BrandName|Strength|Fetus_NewBorn|
|RelativeDate|Height|Test_Result|Sexually_Active_or_Sexual_Orientation|Frequency|
|Time|Weight|Vaccine|Vital_Signs_Header|Communicable_Disease|
|Dosage|Overweight|Hypertension|HDL|Total_Cholesterol|
|Smoking|Date||||

**Read Dataset**

We will download a sample file and create a spark dataframe.

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/oncology_notes/mt_oncology_0.txt -O ./mt_oncology_0.txt

In [0]:
df = spark.read.format("text").load("dbfs:/mt_oncology_0.txt")
df= df.withColumnRenamed("value", "text")
df.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
text |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Sample Type / Medical Specialty: |
Hematology - Oncology |
Sample Name: |
Discharge Summary - Mesothelioma - 1 |
Description: |
Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis. |
(Medical Transcription Sample Report) |
PRINCIPAL DIAGNOSIS: |
Mesothelioma. |
SECONDARY DIAGNOSES: |
Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis. |
PROCEDURES |
1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy. |
2. On August 20, 2007, thoracentesis. |
3. On August 31, 2007, Port-A-Cath placement. |
HISTORY AND PHYSICAL: |
The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.|
PAST MEDICAL HISTORY |
1. Pericardectomy. |
2. Pericarditis. |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
jsl_ner = medical.NerModel.pretrained("ner_jsl","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("jsl_ner")\
    # .setLabelCasing("upper") #decide if we want to return the tags in upper or lower case

jsl_ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","jsl_ner"])\
    .setOutputCol("jsl_ner_chunk")

nlpPipeline = nlp.Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        jsl_ner,
        jsl_ner_converter])


model = nlpPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][ — ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
ner_jsl download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
model.stages

Out[6]: [DocumentAssembler_26a624dca702,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_2752e6e1c234,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_c89cbceb1028,
 NerConverter_96de683bce57]

In [0]:
jsl_ner.getClasses()

Out[7]: ['O',
 'B-Injury_or_Poisoning',
 'B-Direction',
 'B-Test',
 'I-Route',
 'B-Admission_Discharge',
 'B-Death_Entity',
 'I-Oxygen_Therapy',
 'B-Relationship_Status',
 'I-Drug_BrandName',
 'B-Duration',
 'I-Alcohol',
 'I-Triglycerides',
 'I-Date',
 'B-Hyperlipidemia',
 'B-Respiration',
 'I-Test',
 'B-Birth_Entity',
 'I-VS_Finding',
 'B-Age',
 'I-Vaccine_Name',
 'I-Social_History_Header',
 'B-Labour_Delivery',
 'I-Medical_Device',
 'B-Family_History_Header',
 'B-BMI',
 'I-Fetus_NewBorn',
 'I-BMI',
 'B-Temperature',
 'I-Section_Header',
 'I-Communicable_Disease',
 'I-ImagingFindings',
 'I-Psychological_Condition',
 'I-Obesity',
 'I-Sexually_Active_or_Sexual_Orientation',
 'I-Modifier',
 'B-Alcohol',
 'I-Temperature',
 'I-Vaccine',
 'I-Symptom',
 'I-Pulse',
 'B-Kidney_Disease',
 'B-Oncological',
 'I-EKG_Findings',
 'B-Medical_History_Header',
 'I-Relationship_Status',
 'B-Cerebrovascular_Disease',
 'I-Blood_Pressure',
 'I-Diabetes',
 'B-Oxygen_Therapy',
 'B-O2_Saturation',
 'B-Psychological_Condition',
 'B-Heart_Disease',
 'I-Frequency',
 'B-Employment',
 'B-Obesity',
 'B-Disease_Syndrome_Disorder',
 'I-Oncological',
 'B-Pregnancy',
 'I-RelativeDate',
 'I-Procedure',
 'B-ImagingFindings',
 'B-Procedure',
 'I-Labour_Delivery',
 'B-Medical_Device',
 'I-Family_History_Header',
 'B-Race_Ethnicity',
 'I-Hypertension',
 'I-External_body_part_or_region',
 'I-Imaging_Technique',
 'I-Kidney_Disease',
 'B-Section_Header',
 'I-Medical_History_Header',
 'I-Test_Result',
 'I-Direction',
 'I-Substance',
 'B-Symptom',
 'I-Treatment',
 'B-Treatment',
 'B-Substance',
 'I-Clinical_Dept',
 'I-Death_Entity',
 'B-Route',
 'B-Drug_Ingredient',
 'I-LDL',
 'I-Heart_Disease',
 'I-Duration',
 'B-Blood_Pressure',
 'I-Respiration',
 'B-Diet',
 'I-Age',
 'B-External_body_part_or_region',
 'B-LDL',
 'B-Vaccine_Name',
 'B-VS_Finding',
 'I-O2_Saturation',
 'I-Race_Ethnicity',
 'I-Substance_Quantity',
 'B-Allergen',
 'B-EKG_Findings',
 'B-Imaging_Technique',
 'I-Diet',
 'I-Gender',
 'I-Allergen',
 'B-Triglycerides',
 'B-RelativeTime',
 'I-Disease_Syndrome_Disorder',
 'B-Gender',
 'B-Pulse',
 'I-Injury_or_Poisoning',
 'I-Total_Cholesterol',
 'B-Social_History_Header',
 'I-Cerebrovascular_Disease',
 'B-Substance_Quantity',
 'B-Diabetes',
 'I-Admission_Discharge',
 'B-Modifier',
 'B-Internal_organ_or_component',
 'B-Clinical_Dept',
 'I-Internal_organ_or_component',
 'I-Vital_Signs_Header',
 'I-Height',
 'I-Hyperlipidemia',
 'I-RelativeTime',
 'I-Smoking',
 'I-Drug_Ingredient',
 'B-Form',
 'I-Employment',
 'B-Drug_BrandName',
 'B-Strength',
 'I-Weight',
 'B-RelativeDate',
 'B-Fetus_NewBorn',
 'B-Height',
 'B-Test_Result',
 'B-Time',
 'B-Frequency',
 'B-Sexually_Active_or_Sexual_Orientation',
 'I-Strength',
 'B-Weight',
 'B-Vaccine',
 'I-Pregnancy',
 'I-Form',
 'B-Vital_Signs_Header',
 'I-Dosage',
 'I-Time',
 'B-Communicable_Disease',
 'B-Dosage',
 'B-Overweight',
 'B-Hypertension',
 'B-HDL',
 'B-Total_Cholesterol',
 'I-HDL',
 'B-Smoking',
 'B-Date']

In [0]:
#transform the pipeline with sample data
result = model.transform(df)

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| jsl_ner| jsl_ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Sample Type / Med...|[{document, 0, 31...|[{document, 0, 31...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...| []|
Hematology - Onco...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 9, He...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 9, He...|
 Sample Name:|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...| []|
Discharge Summary...|[{document, 0, 35...|[{document, 0, 35...|[{token, 0, 8, Di...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 8, Di...|
 Description:|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 10, D...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 11, D...|
Mesothelioma, ple...|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 11, M...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 11, M...|
(Medical Transcri...|[{document, 0, 36...|[{document, 0, 36...|[{token, 0, 0, (,...|[{word_embeddings...|[{named_entity, 0...| []|
PRINCIPAL DIAGNOSIS:|[{document, 0, 19...|[{document, 0, 19...|[{token, 0, 8, PR...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 19, P...|
 Mesothelioma.|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 11, M...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 11, M...|
SECONDARY DIAGNOSES:|[{document, 0, 19...|[{document, 0, 19...|[{token, 0, 8, SE...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 19, S...|
Pleural effusion,...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 6, Pl...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 15, P...|
 PROCEDURES|[{document, 0, 9,...|[{document, 0, 9,...|[{token, 0, 9, PR...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 9, PR...|
1. On August 24, ...|[{document, 0, 97...|[{document, 0, 97...|[{token, 0, 0, 1,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 6, 11, A...|
2. On August 20, ...|[{document, 0, 36...|[{document, 0, 36...|[{token, 0, 0, 2,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 6, 11, A...|
3. On August 31, ...|[{document, 0, 44...|[{document, 0, 44...|[{token, 0, 0, 3,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 6, 11, A...|
HISTORY AND PHYSI...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 6, HI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 20, H...|
The patient is a ...|[{document, 0, 38...|[{document, 0, 96...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 17, 27, ...|
PAST MEDICAL HISTORY|[{document, 0, 19...|[{document, 0, 19...|[{token, 0, 3, PA...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 19, P...|
 1. Pericardectomy.|[{document, 0, 17...|[{document, 0, 17...|[{token, 0, 0, 1,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 16, P...|
 2. Pericarditis.|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 0, 2,...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 3, 14, P...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select("sentence.result").show(truncate=100)

+----------------------------------------------------------------------------------------------------+
 result|
+----------------------------------------------------------------------------------------------------+
 [Sample Type / Medical Specialty:]|
 [Hematology - Oncology]|
 [Sample Name:]|
 [Discharge Summary - Mesothelioma - 1]|
 [Description:]|
[Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and his...|
 [(Medical Transcription Sample Report)]|
 [PRINCIPAL DIAGNOSIS:]|
 [Mesothelioma.]|
 [SECONDARY DIAGNOSES:]|
[Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep v...|
 [PROCEDURES]|
[1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.]|
 [2. On August 20, 2007, thoracentesis.]|
 [3. On August 31, 2007, Port-A-Cath placement.]|
 [HISTORY AND PHYSICAL:]|
[The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week...|
 [PAST MEDICAL HISTORY]|
 [1. Pericardectomy.]|
 [2. Pericarditis.]|
+----------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
result_df = result.select(F.explode(F.arrays_zip(result.token.result,result.jsl_ner.result)).alias("cols"))\
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)

+-------------+---------------------------+
 token| ner_label|
+-------------+---------------------------+
 Sample| O|
 Type| O|
 /| O|
 Medical| O|
 Specialty| O|
 :| O|
 Hematology| B-Clinical_Dept|
 -| O|
 Oncology| O|
 Sample| O|
 Name| O|
 :| O|
 Discharge| B-Admission_Discharge|
 Summary| O|
 -| O|
 Mesothelioma| B-Oncological|
 -| O|
 1| O|
 Description| B-Section_Header|
 :| I-Section_Header|
 Mesothelioma| B-Oncological|
 ,| O|
 pleural|B-Disease_Syndrome_Disorder|
 effusion|I-Disease_Syndrome_Disorder|
 ,| O|
 atrial| B-Heart_Disease|
 fibrillation| I-Heart_Disease|
 ,| O|
 anemia|B-Disease_Syndrome_Disorder|
 ,| O|
 ascites|B-Disease_Syndrome_Disorder|
 ,| O|
 esophageal|B-Disease_Syndrome_Disorder|
 reflux|I-Disease_Syndrome_Disorder|
 ,| O|
 and| O|
 history| O|
 of| O|
 deep|B-Disease_Syndrome_Disorder|
 venous|I-Disease_Syndrome_Disorder|
 thrombosis|I-Disease_Syndrome_Disorder|
 .| O|
 (| O|
 Medical| O|
Transcription| O|
 Sample| O|
 Report| O|
 )| O|
 PRINCIPAL| B-Section_Header|
 DIAGNOSIS| I-Section_Header|
+-------------+---------------------------+
only showing top 50 rows

Lets count the ner_labels.

In [0]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+-----------------------------+-----+
ner_label |count|
+-----------------------------+-----+
O |579 |
B-Gender |30 |
B-Symptom |28 |
B-Disease_Syndrome_Disorder |21 |
B-Procedure |20 |
I-Section_Header |19 |
I-Disease_Syndrome_Disorder |16 |
I-Procedure |16 |
I-Symptom |14 |
B-Test |12 |
B-Direction |11 |
B-Section_Header |11 |
I-Date |10 |
B-Date |10 |
B-Drug_BrandName |9 |
I-Test |8 |
B-Internal_organ_or_component|7 |
B-Oncological |7 |
I-Strength |7 |
B-Strength |7 |
+-----------------------------+-----+
only showing top 20 rows

As you see above, we have labels start with **"O", "B"** and **"I"**. <br/>
O means that the corresponding token is not labeled as any named entity. "B" means that the corresponding token refers to the beginning of the ner chunk while "I" means the continuation of the ner chunk.  <br/>

So, in our example, we have `deep venous thrombosis` `Disease_Syndrome_Disorder` chunk. In this case, `deep` labeled as `B-Disease_Syndrome_Disorder` since it is the beginnig of the chunk, `venous` and `thrombosis` labeled as `I-Disease_Syndrome_Disorder` as they are the continuation of the `Disease_Syndrome_Disorder` chunk.


Now, let's check out the results as a **chunk-based** instead of token-based.

In [0]:
result.select(F.explode(F.arrays_zip(result.jsl_ner_chunk.result, result.jsl_ner_chunk.begin, result.jsl_ner_chunk.end, result.jsl_ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['3']['sentence']").alias("sentence_id"),
              F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label")).show(truncate=False)

+-----------+----------------------+-----+---+-------------------------+
sentence_id|chunk |begin|end|ner_label |
+-----------+----------------------+-----+---+-------------------------+
0 |Hematology |0 |9 |Clinical_Dept |
0 |Discharge |0 |8 |Admission_Discharge |
0 |Mesothelioma |20 |31 |Oncological |
0 |Description: |0 |11 |Section_Header |
0 |Mesothelioma |0 |11 |Oncological |
0 |pleural effusion |14 |29 |Disease_Syndrome_Disorder|
0 |atrial fibrillation |32 |50 |Heart_Disease |
0 |anemia |53 |58 |Disease_Syndrome_Disorder|
0 |ascites |61 |67 |Disease_Syndrome_Disorder|
0 |esophageal reflux |70 |86 |Disease_Syndrome_Disorder|
0 |deep venous thrombosis|104 |125|Disease_Syndrome_Disorder|
0 |PRINCIPAL DIAGNOSIS: |0 |19 |Section_Header |
0 |Mesothelioma |0 |11 |Oncological |
0 |SECONDARY DIAGNOSES: |0 |19 |Section_Header |
0 |Pleural effusion |0 |15 |Disease_Syndrome_Disorder|
0 |atrial fibrillation |18 |36 |Heart_Disease |
0 |anemia |39 |44 |Disease_Syndrome_Disorder|
0 |ascites |47 |53 |Disease_Syndrome_Disorder|
0 |esophageal reflux |56 |72 |Disease_Syndrome_Disorder|
0 |deep venous thrombosis|90 |111|Disease_Syndrome_Disorder|
+-----------+----------------------+-----+---+-------------------------+
only showing top 20 rows

As you see, our ner model successfully extracted useful named entities. <br/>

### with LightPipeline

We created a pipeline with `ner_jsl`. This time, let's use **LightPipeline** instead of `transform()`.

**LightPipeline** is a Spark NLP specific Pipeline class equivalent to Spark ML Pipeline. The difference is that its execution does not hold to Spark principles, instead, it computes everything locally (but in parallel) in order to achieve faster inference when dealing with small amounts of data. This means, we don't have to Spark Dataframe, but a string or an array of strings instead, to be annotated. To create Light Pipelines, you need to input an already trained (fit) Spark ML Pipeline.

 It’s `transform()` stage is converted into `annotate()` or `fullAnnotate()` instead. <br/>

Let's convert our existing ner pipeline to LightPipeline and see the results with an example text.

In [0]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM )'''

print (text)

light_model = nlp.LightPipeline(model)

light_result = light_model.annotate(text)

light_result

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM )
Out[14]: {'document': ['\nA 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM )'],
 'jsl_ner_chunk': ['28-year-old',
 'female',
 'gestational diabetes mellitus',
 'eight years prior',
 'type two diabetes mellitus',
 'T2DM'],
 'jsl_ner': ['O',
 'B-Age',
 'B-Gender',
 'O',
 'O',
 'O',
 'O',
 'B-Diabetes',
 'I-Diabetes',
 'I-Diabetes',
 'O',
 'B-RelativeDate',
 'I-RelativeDate',
 'I-RelativeDate',
 'O',
 'O',
 'O',
 'O',
 'B-Diabetes',
 'I-Diabetes',
 'I-Diabetes',
 'I-Diabetes',
 'O',
 'B-Diabetes',
 'O'],
 'token': ['A',
 '28-year-old',
 'female',
 'with',
 'a',
 'history',
 'of',
 'gestational',
 'diabetes',
 'mellitus',
 'diagnosed',
 'eight',
 'years',
 'prior',
 'to',
 'presentation',
 'and',
 'subsequent',
 'type',
 'two',
 'diabetes',
 'mellitus',
 '(',
 'T2DM',
 ')'],
 'embeddings': ['A',
 '28-year-old',
 'female',
 'with',
 'a',
 'history',
 'of',
 'gestational',
 'diabetes',
 'mellitus',
 'diagnosed',
 'eight',
 'years',
 'prior',
 'to',
 'presentation',
 'and',
 'subsequent',
 'type',
 'two',
 'diabetes',
 'mellitus',
 '(',
 'T2DM',
 ')'],
 'sentence': ['A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM )']}

In [0]:
light_result.keys()

Out[15]: dict_keys(['document', 'jsl_ner_chunk', 'jsl_ner', 'token', 'embeddings', 'sentence'])

In [0]:
list(zip(light_result['token'], light_result['jsl_ner']))

Out[16]: [('A', 'O'),
 ('28-year-old', 'B-Age'),
 ('female', 'B-Gender'),
 ('with', 'O'),
 ('a', 'O'),
 ('history', 'O'),
 ('of', 'O'),
 ('gestational', 'B-Diabetes'),
 ('diabetes', 'I-Diabetes'),
 ('mellitus', 'I-Diabetes'),
 ('diagnosed', 'O'),
 ('eight', 'B-RelativeDate'),
 ('years', 'I-RelativeDate'),
 ('prior', 'I-RelativeDate'),
 ('to', 'O'),
 ('presentation', 'O'),
 ('and', 'O'),
 ('subsequent', 'O'),
 ('type', 'B-Diabetes'),
 ('two', 'I-Diabetes'),
 ('diabetes', 'I-Diabetes'),
 ('mellitus', 'I-Diabetes'),
 ('(', 'O'),
 ('T2DM', 'B-Diabetes'),
 (')', 'O')]

In [0]:
text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , and associated with an acute hepatitis , presented with a one-week history of polyuria , poor appetite , and vomiting .
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation .
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl ,  creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , and venous pH 7.27 .
'''

light_result_full = light_model.fullAnnotate(text)

In [0]:
light_result_full

Out[18]: [{'document': [Annotation(document, 0, 898, 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , and associated with an acute hepatitis , presented with a one-week history of polyuria , poor appetite , and vomiting .
 She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation .
 Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , and venous pH 7.27 .
 , {}, [])],
 'jsl_ner_chunk': [Annotation(chunk, 3, 13, 28-year-old, {'entity': 'Age', 'sentence': '0', 'chunk': '0', 'confidence': '0.9991'}, []),
 Annotation(chunk, 15, 20, female, {'entity': 'Gender', 'sentence': '0', 'chunk': '1', 'confidence': '0.9999'}, []),
 Annotation(chunk, 40, 68, gestational diabetes mellitus, {'entity': 'Diabetes', 'sentence': '0', 'chunk': '2', 'confidence': '0.9663334'}, []),
 Annotation(chunk, 80, 96, eight years prior, {'entity': 'RelativeDate', 'sentence': '0', 'chunk': '3', 'confidence': '0.6821'}, []),
 Annotation(chunk, 129, 154, type two diabetes mellitus, {'entity': 'Diabetes', 'sentence': '0', 'chunk': '4', 'confidence': '0.94152504'}, []),
 Annotation(chunk, 158, 161, T2DM, {'entity': 'Diabetes', 'sentence': '0', 'chunk': '5', 'confidence': '0.9233'}, []),
 Annotation(chunk, 187, 210, HTG-induced pancreatitis, {'entity': 'Disease_Syndrome_Disorder', 'sentence': '0', 'chunk': '6', 'confidence': '0.51625'}, []),
 Annotation(chunk, 212, 228, three years prior, {'entity': 'RelativeDate', 'sentence': '0', 'chunk': '7', 'confidence': '0.8452'}, []),
 Annotation(chunk, 271, 275, acute, {'entity': 'Modifier', 'sentence': '0', 'chunk': '8', 'confidence': '0.9484'}, []),
 Annotation(chunk, 277, 285, hepatitis, {'entity': 'Disease_Syndrome_Disorder', 'sentence': '0', 'chunk': '9', 'confidence': '0.5994'}, []),
 Annotation(chunk, 306, 313, one-week, {'entity': 'Duration', 'sentence': '0', 'chunk': '10', 'confidence': '0.9544'}, []),
 Annotation(chunk, 326, 333, polyuria, {'entity': 'Symptom', 'sentence': '0', 'chunk': '11', 'confidence': '0.986'}, []),
 Annotation(chunk, 337, 349, poor appetite, {'entity': 'Symptom', 'sentence': '0', 'chunk': '12', 'confidence': '0.9307'}, []),
 Annotation(chunk, 357, 364, vomiting, {'entity': 'Symptom', 'sentence': '0', 'chunk': '13', 'confidence': '0.9918'}, []),
 Annotation(chunk, 368, 370, She, {'entity': 'Gender', 'sentence': '1', 'chunk': '14', 'confidence': '1.0'}, []),
 Annotation(chunk, 379, 387, metformin, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '15', 'confidence': '0.9898'}, []),
 Annotation(chunk, 391, 399, glipizide, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '16', 'confidence': '0.9993'}, []),
 Annotation(chunk, 407, 419, dapagliflozin, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '17', 'confidence': '0.9971'}, []),
 Annotation(chunk, 425, 428, T2DM, {'entity': 'Diabetes', 'sentence': '1', 'chunk': '18', 'confidence': '0.8716'}, []),
 Annotation(chunk, 434, 445, atorvastatin, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '19', 'confidence': '0.9968'}, []),
 Annotation(chunk, 451, 461, gemfibrozil, {'entity': 'Drug_Ingredient', 'sentence': '1', 'chunk': '20', 'confidence': '0.9933'}, []),
 Annotation(chunk, 467, 469, HTG, {'entity': 'Hyperlipidemia', 'sentence': '1', 'chunk': '21', 'confidence': '0.938'}, []),
 Annotation(chunk, 473, 475, She, {'entity': 'Gender', 'sentence': '2', 'chunk': '22', 'confidence': '1.0'}, []),
 Annotation(chunk, 489, 501, dapagliflozin, {

We used `fullAnnotate()` to see more detailed result

In [0]:
chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result_full[0]['jsl_ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    sentence.append(n.metadata['sentence'])


import pandas as pd

df_clinical = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end,
                   'sentence_id':sentence, 'entities':entities})

df_clinical.head(20)

Out[19]:

,chunks,begin,end,sentence_id,entities
0,28-year-old,3,13,0,Age
1,female,15,20,0,Gender
2,gestational diabetes mellitus,40,68,0,Diabetes
3,eight years prior,80,96,0,RelativeDate
4,type two diabetes mellitus,129,154,0,Diabetes
5,T2DM,158,161,0,Diabetes
6,HTG-induced pancreatitis,187,210,0,Disease_Syndrome_Disorder
7,three years prior,212,228,0,RelativeDate
8,acute,271,275,0,Modifier
9,hepatitis,277,285,0,Disease_Syndrome_Disorder


### NER Visualizer

The **NerVisualizer** highlights the named entities that are identified by Spark NLP and also displays their labels as decorations on top of the analyzed text. The colors assigned to the predicted labels can be configured to fit the particular needs of the application.

In [0]:
visualiser = nlp.viz.NerVisualizer()

vis = visualiser.display(light_result_full[0], label_col='jsl_ner_chunk', document_col='document', return_html=True)

# Change color of an entity label
#visualiser.set_label_colors({'PROBLEM':'#008080', 'TEST':'#800080', 'TREATMENT':'#808080'})
#visualiser.display(light_result[0], label_col='ner_chunk')

# Set label filter
# vis = visualiser.display(light_result, label_col='ner_chunk', document_col='document',
                   #labels=['PROBLEM','TEST','TREATMENT])
  
displayHTML(vis)

A 28-year-old Age female Gender with a history of gestational diabetes mellitus Diabetes diagnosed eight years prior RelativeDate to presentation and subsequent type two diabetes mellitus Diabetes ( T2DM Diabetes ), one prior episode of HTG-induced pancreatitis Disease_Syndrome_Disorder three years prior RelativeDate to presentation , and associated with an acute Modifier hepatitis Disease_Syndrome_Disorder , presented with a one-week Duration history of polyuria Symptom , poor appetite Symptom , and vomiting Symptom . She Gender was on metformin Drug_Ingredient , glipizide Drug_Ingredient , and dapagliflozin Drug_Ingredient for T2DM Diabetes and atorvastatin Drug_Ingredient and gemfibrozil Drug_Ingredient for HTG Hyperlipidemia . She Gender had been on dapagliflozin Drug_Ingredient for six months Duration at the time of presentation . Physical examination on presentation was significant for dry oral mucosa Symptom ; significantly , her Gender abdominal External_body_part_or_region examination was benign with no tenderness Symptom , guarding Symptom , or rigidity Symptom . Pertinent laboratory findings on admission Admission_Discharge were : serum glucose Test 111 mg/dl Test_Result , creatinine Test 0.4 mg/dL Test_Result , triglycerides Triglycerides 508 mg/dL Test_Result , total cholesterol 122 mg/dL Total_Cholesterol , and venous pH Test 7.27 Test_Result .

## Clinical NER Chunk Merger

Sparknlp has the ability to merge the chunks that come from different annotators(NER, ContextualParser or any other annotator producing chunks) through the `ChunkMergeApproach()` annotator.  The decision on which chunk to select is made according to the chunk indices in the source document. (chunks with longer lengths and highest information will be kept from each source)


Let's create a sample pipeline consisting `ner_jsl`, `ner_posology` and `ner_deid_generic` models, and merge the outputs of these models.

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model to detect PHI entities
deid_ner = medical.NerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("deid_ner")

deid_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "deid_ner"]) \
    .setOutputCol("deid_ner_chunk")\
    .setWhiteList(["AGE", "DATE", "LOCATION"]) # only AGE, DATE and LOCATION entities

# NER model to detect clinical entities (PROBLEM, TEST, TREATMENT)
clinical_ner = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "clinical_ner"]) \
    .setOutputCol("clinical_ner_chunk")\
    .setBlackList(["TREATMENT"]) # blocks TREATMENT entities

# NER model to detect medication entities (Durg, Dosage, etc.))
posology_ner = medical.NerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("posology_ner")

posology_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "posology_ner"]) \
    .setOutputCol("posology_ner_chunk")


# merge ner_chunks by prioritizing the overlapping indices (chunks with longer lengths and highest information will be kept from each ner model)
chunk_merger = medical.ChunkMergeApproach()\
    .setInputCols('deid_ner_chunk', "clinical_ner_chunk", "posology_ner_chunk")\
    .setOutputCol('merged_ner_chunk')


nlpPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    deid_ner,
    deid_ner_converter,
    clinical_ner,
    clinical_ner_converter,
    posology_ner,
    posology_ner_converter,
    chunk_merger])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_generic_augmented download started this may take some time.
[ | ][ / ][OK!]
ner_clinical download started this may take some time.
[ | ][ / ][OK!]
ner_posology download started this may take some time.
[ | ][ / ][OK!]

In [0]:
text = """HISTORY AND PHYSICAL: The patient is from Vietnam, 41-year-old female with a nonproductive cough that started on Sep 2006.
          She has had right-sided chest pain radiating to her back with fever starting on November 5, 2006.
          She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.
          MEDICATIONS: Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and INR was 2.3. Amiodarone 100 mg p.o. daily.
       """

spark_df = spark.createDataFrame([[text]]).toDF("text")

merged_data = model.transform(spark_df)

%time merged_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| deid_ner| deid_ner_chunk| clinical_ner| clinical_ner_chunk| posology_ner| posology_ner_chunk| merged_ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
HISTORY AND PHYSI...|[{document, 0, 58...|[{document, 0, 12...|[{token, 0, 6, HI...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 42, 48, ...|[{named_entity, 0...|[{chunk, 75, 95, ...|[{named_entity, 0...|[{chunk, 464, 471...|[{chunk, 42, 48, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

CPU times: user 35.7 ms, sys: 4.86 ms, total: 40.6 ms
Wall time: 7.89 s

In [0]:
lp_model = nlp.LightPipeline(model)

In [0]:
result_df = merged_data.select(F.explode('merged_ner_chunk').alias("cols")) \
                       .select(F.expr("cols.begin").alias("begin"),
                               F.expr("cols.end").alias("end"),
                               F.expr("cols.result").alias("chunk"),
                               F.expr("cols.metadata.entity").alias("entity"))

result_df.show(50, truncate=100)

+-----+---+----------------------------+---------+
begin|end| chunk| entity|
+-----+---+----------------------------+---------+
 42| 48| Vietnam| LOCATION|
 51| 61| 41-year-old| AGE|
 75| 95| a nonproductive cough| PROBLEM|
 113|120| Sep 2006| DATE|
 145|166| right-sided chest pain| PROBLEM|
 195|199| fever| PROBLEM|
 213|228| November 5, 2006| DATE|
 262|273| pericarditis| PROBLEM|
 297|304| May 2006| DATE|
 320|324| cough| PROBLEM|
 331|352| right-sided chest pain| PROBLEM|
 390|400| Chest x-ray| TEST|
 411|438|right-sided pleural effusion| PROBLEM|
 464|471| Coumadin| DRUG|
 473|476| 1 mg| STRENGTH|
 478|482| daily|FREQUENCY|
 490|492| INR| TEST|
 501|507| Tuesday| DATE|
 510|524| August 14, 2007| DATE|
 531|533| INR| TEST|
 544|553| Amiodarone| DRUG|
 555|560| 100 mg| STRENGTH|
 562|564| p.o| ROUTE|
 567|571| daily|FREQUENCY|
+-----+---+----------------------------+---------+

## Clinical NER Training

In this section, we will train our own NER model. <br/>
Firstly, let's download NCBI Disease dataset for training.

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltrain.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltest.txt

`CoNLL(includeDocId=True).readDataset(spark, "conll_file.txt")`  this methos can allow if you have doc_id information in the conll file, you can add this information to the dataframe as a column.

```
conll="""-DOCSTART- -X- -1- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O

-DOCSTART- -X- 2 O

Rare NNP B-NP O
Hendrix NNP I-NP B-PER

-DOCSTART- -X- -3-1- O

China NNP B-NP B-LOC
says VBZ B-VP O

-DOCSTART-

China NNP B-NP B-LOC
says VBZ B-VP O
"""
```
```
with open('conll_file.txt', 'w') as f:
    f.write(conll)

data = CoNLL(includeDocId=True).readDataset(spark, "conll_file.txt")

data.show()
```



```
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|doc_id|                text|            document|            sentence|               token|                 pos|               label|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     1|EU rejects German...|[{document, 0, 28...|[{document, 0, 47...|[{token, 0, 1, EU...|[{pos, 0, 1, NNP,...|[{named_entity, 0...|
|     2|Rare Hendrix song...|[{document, 0, 97...|[{document, 0, 50...|[{token, 0, 3, Ra...|[{pos, 0, 3, NNP,...|[{named_entity, 0...|
|   3-1|China says Taiwan...|[{document, 0, 13...|[{document, 0, 46...|[{token, 0, 4, Ch...|[{pos, 0, 4, NNP,...|[{named_entity, 0...|
|     X|China says Taiwan...|[{document, 0, 13...|[{document, 0, 46...|[{token, 0, 4, Ch...|[{pos, 0, 4, NNP,...|[{named_entity, 0...|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
```

In [0]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, 'dbfs:/NER_NCBIconlltrain.txt')
conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
conll_data.count()

Out[26]: 3266

Checking the number of labels in the dataset

In [0]:

from pyspark.sql import functions as F

conll_data.select(F.explode(F.arrays_zip(conll_data.token.result, conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)

+------------+-----+
ground_truth|count|
+------------+-----+
O |75093|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
'''
As you can see, there are too many `O` labels in the dataset.
To make it more balanced, we can drop the sentences have only O labels.
(`c>1` means we drop all the sentences that have no valuable labels other than `O`)
'''

'''
conll_data = conll_data.withColumn('unique', F.array_distinct("label.result"))\
                       .withColumn('c', F.size('unique'))\
                       .filter(F.col('c')>1)

conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)
'''

Out[28]: '\nconll_data = conll_data.withColumn(\'unique\', F.array_distinct("label.result")) .withColumn(\'c\', F.size(\'unique\')) .filter(F.col(\'c\')>1)\n\nconll_data.select(F.explode(F.arrays_zip(\'token.result\',\'label.result\')).alias("cols")) .select(F.expr("cols[\'0\']").alias("token"),\n F.expr("cols[\'1\']").alias("ground_truth")) .groupBy(\'ground_truth\') .count() .orderBy(\'count\', ascending=False) .show(100,truncate=False)\n'

Now, let's download the embeddings. We will transform the embeddings with our test data in order to use the `setTestDataset()` parameter of `MedicalNerApproach()` during the training. This parameter is used to calculate statistics on test dataset during training.

In [0]:
# Clinical word embeddings trained on PubMED dataset
clinical_embeddings = nlp.WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

Let's read the test dataset and get its embeddings, then save it as a parquet format to use it in the training.

In [0]:

test_data = CoNLL().readDataset(spark, 'dbfs:/NER_NCBIconlltest.txt')

test_data = clinical_embeddings.transform(test_data)
test_data.write.parquet('dbfs:/NER_NCBIconlltest.parquet')

### NERDL Graph

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the `MedicalNerApproach` annotator.

In [0]:
!pip install -q tensorflow==2.11.0
!pip install -q tensorflow-addons

WARNING: You are using pip version 21.0.1; however, version 23.2.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
 WARNING: You are using pip version 21.0.1; however, version 23.2.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder_path = "medical_ner_graphs"

ner_graph_builder = medical.TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(20)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

Also, we could use custom graph like following;

In [0]:
# TensorFlow graph file (`.pb` extension) can be produced for NER training externally.
# If this method is used, graph folder should be added to MedicalNerApproach training
# model as `.setGraphFolder(graph_folder_path)` .

'''
from sparknlp_jsl.training import tf_graph

tf_graph.print_model_params("ner_dl")

graph_folder_path = "medical_ner_graphs"
tf_graph.build("ner_dl",
               build_params={"embeddings_dim": 200,
                             "nchars": 85,
                             "ntags": 3,
                             "is_medical": 1},
               model_location=graph_folder_path,
               model_filename="auto")
'''

Out[33]: '\nfrom sparknlp_jsl.training import tf_graph\n\ntf_graph.print_model_params("ner_dl")\n\ngraph_folder_path = "medical_ner_graphs"\ntf_graph.build("ner_dl",\n build_params={"embeddings_dim": 200,\n "nchars": 85,\n "ntags": 3,\n "is_medical": 1},\n model_location=graph_folder_path,\n model_filename="auto")\n'

In [0]:
# # for open source users
# '''
# !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py
# !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py
# !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py
# !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py
# !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py

# !pip -q install tensorflow==1.15.0

# import create_graph

# ntags = 3 # number of labels
# embeddings_dim = 200
# nchars =83

# create_graph.create_graph(ntags, embeddings_dim, nchars)
# '''

Now we can train our model through `MedicalNerApproach()`

We will use `setEarlyStoppingCriterion` and `setEarlyStoppingPatience` parameters to stop training if it doesn't improve any more. Also, these parameters will provide us to save time and resources.

- `setEarlyStoppingCriterion(float)`: This parameter is used to set the minimal improvement of the test metric to terminate training. The metric monitored is the same which is used to save the best best model (macro F1 if using test or validation set, loss otherwise). If it is F1, then training is stopped if `F1_current - F1_previous < earlyStoppingCriterion`. If monitoring loss, then training is stopped if `loss_previous - loss_current < earlyStoppingCriterion`. If earlyStoppingCriterion is `0`, then no early stopping is applied. Default is `0`.

- `setEarlyStoppingPatience(int)`: This is the number of epoch without improvement which will be tolerated. Default is `0`, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch (taking earlyStoppingCriterion into account).

In [0]:
nerTagger = medical.NerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(30)\
    .setBatchSize(64)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('dbfs:/ner/ner_logs')\
    .setGraphFolder('dbfs:/ner/medical_ner_graphs')\
    .setTestDataset("/NER_NCBIconlltest.parquet")\
    .setUseBestModel(True)\
    .setEarlyStoppingCriterion(0.04)\
    .setEarlyStoppingPatience(3)\
    .setGraphFolder(graph_folder_path)
   # .setEnableMemoryOptimizer(True) #>> if you have a limited memory and a large conll file, you can set this True to train batch by batch

ner_pipeline = nlp.Pipeline(stages=[
          clinical_embeddings,
          ner_graph_builder,
          nerTagger
 ])

You can visit [1.4.Resume_MedicalNer_Model_Training.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb) notebook for fine-tuning pretrained NER models and more details of `MedicalNerApproach()` parameters.

In [0]:
%%time
ner_model = ner_pipeline.fit(conll_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 20}
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:107: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:93: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:93: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:93: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
ner_dl graph exported to medical_ner_graphs/blstm_3_200_20_85.pb
CPU times: user 11.6 s, sys: 1.03 s, total: 12.7 s
Wall time: 3min 48s

If you get an error for incompatible TF graph, use [4.1 NerDL-Graph](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/4.1_NerDL_Graph.ipynb) notebook in public folder to create a graph

Licensed users can also use [17.Graph_builder_for_DL_models.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/17.Graph_builder_for_DL_models.ipynb) to create TF graphs easily.

`getTrainingClassDistribution()` parameter returns the distribution of labels used when training the NER model.

In [0]:
ner_model.stages[2].getTrainingClassDistribution()

Out[34]: {'O': 75093, 'B-Disease': 3093, 'I-Disease': 3547}

In [0]:
import os 
log_file= os.listdir("/dbfs/ner/ner_logs")[0]

with open (f"/dbfs/ner/ner_logs/{log_file}") as f:
  print(f.read())

Name of the selected graph: /local_disk0/tmp/sparknlp_tmp_6841870312883533780/blstm_5_200_20_86.pb
Training started - total epochs: 35 - lr: 0.001 - batch size: 8 - labels: 5 - chars: 85 - training examples: 1609


Epoch 1/35 started, lr: 0.001, dataset size: 1609


Epoch 1/35 - 16.93s - loss: 732.8118 - avg training loss: 4.551626 - batches: 161
Quality on validation dataset (20.0%), validation examples = 321
time to finish evaluation: 1.31s
Total validation loss: 75.4140	Avg validation loss: 1.6759
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 29 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 20 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 8 1 60 0.8888889 0.11764706 0.2077922
tp: 8 fp: 1 fn: 109 labels: 3
Macro-average prec: 0.2962963, rec: 0.039215688, f1: 0.06926408
Micro-average prec: 0.8888889, rec: 0.06837607, f1: 0.12698413


Epoch 2/35 started, lr: 9.950249E-4, dataset size: 1609


Epoch 2/35 - 15.15s - loss: 282.72308 - avg training loss: 1.756044 - batches: 161
Quality on validation dataset (20.0%), validation examples = 321
time to finish evaluation: 0.73s
Total validation loss: 42.3602	Avg validation loss: 0.9413
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 29 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 20 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 47 7 21 0.8703704 0.6911765 0.77049184
tp: 47 fp: 7 fn: 70 labels: 3
Macro-average prec: 0.29012346, rec: 0.23039216, f1: 0.2568306
Micro-average prec: 0.8703704, rec: 0.4017094, f1: 0.5497076


Epoch 3/35 started, lr: 9.90099E-4, dataset size: 1609


Epoch 3/35 - 14.96s - loss: 219.9309 - avg training loss: 1.3660303 - batches: 161
Quality on validation dataset (20.0%), validation examples = 321
time to finish evaluation: 0.66s
Total validation loss: 34.0379	Avg validation loss: 0.7564
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 2 0 27 1.0 0.06896552 0.12903225
B-SUICIDE_PSYCHACHE 14 1 6 0.93333334 0.7 0.79999995
B-SUICIDE_BEHAVIOR 51 7 17 0.87931037 0.75 0.8095238
tp: 67 fp: 8 fn: 50 labels: 3
Macro-average prec: 0.9375479, rec: 0.50632185, f1: 0.65753984
Micro-average prec: 0.8933333, rec: 0.5726496, f1: 0.6979166


Epoch 4/35 started, lr: 9.852217E-4, dataset size: 1609


Epoch 4/35 - 15.06s - loss: 178.52916 - avg training loss: 1.1088768 - batches: 161
Quality on validation dataset (20.0%), validation examples = 321
time to finish evaluation: 0.69s
Total validation loss: 35.9763	Avg validation loss: 0.7995
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 1 0 28 1.0 0.03448276 0.06666667
B-SUICIDE_PSYCHACHE 19 2 1 0.9047619 0.95 0.92682934
B-SUICIDE_BEHAVIOR 47 4 21 0.92156863 0.6911765 0.78991604
tp: 67 fp: 6 fn: 50 labels: 3
Macro-average prec: 0.94211024, rec: 0.55855304, f1: 0.7013146
Micro-average prec: 0.91780823, rec: 0.5726496, f1: 0.7052632


Epoch 5/35 started, lr: 9.803922E-4, dataset size: 1609


Epoch 5/35 - 14.72s - loss: 157.93848 - avg training loss: 0.98098433 - batches: 161
Quality on validation dataset (20.0%), validation examples = 321
time to finish evaluation: 0.68s
Total validation loss: 23.3555	Avg validation loss: 0.5190
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 6 0 23 1.0 0.20689656 0.34285715
B-SUICIDE_PSYCHACHE 19 1 1 0.95 0.95 0.95
B-SUICIDE_BEHAVIOR 53 4 15 0.9298246 0.7794118 0.84800005
tp: 78 fp: 5 fn: 39 labels: 3
Macro-average prec: 0.95994157, rec: 0.6454361, f1: 0.7718818
Micro-average prec: 0.939759, rec: 0.6666667, f1: 0.78


Epoch 6/35 started, lr: 9.756098E-4, dataset size: 1609


Epoch 6/35 - 14.58s - loss: 131.64915 - avg training loss: 0.81769663 - batches: 161
Quality on validation dataset (20.0%), validation examples = 321
time to finish evaluation: 0.69s
Total validation loss: 31.8526	Avg validation loss: 0.7078
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 14 0 15 1.0 0.4827586 0.65116274
B-SUICIDE_PSYCHACHE 19 2 1 0.9047619 0.95 0.92682934
B-SUICIDE_BEHAVIOR 56 7 12 0.8888889 0.8235294 0.8549618
tp: 89 fp: 9 fn: 28 labels: 3
Macro-average prec: 0.93121696, rec: 0.752096, f1: 0.8321264
Micro-average prec: 0.90816325, rec: 0.7606838, f1: 0.82790697




As you see above, our **earlyStopping** feature worked, trainining was terminated before 30th epoch.

### Evaluating the Model

Firstly, we will get the predictions from test data

In [0]:
pred_df = ner_model.stages[2].transform(test_data)

In [0]:
pred_df.columns

Out[37]: ['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

Now, let's see the metrics by using **eval** package of sparknlp_jsl

In [0]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+------+
 entity| tp| fp| fn|total|precision|recall| f1|
+-------+-----+-----+-----+-----+---------+------+------+
Disease|490.0|174.0|214.0|704.0| 0.738| 0.696|0.7164|
+-------+-----+-----+-----+-----+---------+------+------+

+-----------------+
 macro|
+-----------------+
0.716374269005848|
+-----------------+

None
+-----------------+
 micro|
+-----------------+
0.716374269005848|
+-----------------+

None

In [0]:
# token-based evaluation (partial_chunk_per_token mode)

evaler = medical.NerDLMetrics(mode="partial_chunk_per_token")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+------+-----+-----+------+---------+------+------+
 entity| tp| fp| fn| total|precision|recall| f1|
+-------+------+-----+-----+------+---------+------+------+
Disease|1161.0|127.0|336.0|1497.0| 0.9014|0.7756|0.8338|
+-------+------+-----+-----+------+---------+------+------+

+------------------+
 macro|
+------------------+
0.8337522441651706|
+------------------+

None
+------------------+
 micro|
+------------------+
0.8337522441651706|
+------------------+

None

`ner_utils`: This new module is used after NER training to calculate chunkbase mertics and plot training logs.

`evaluate`: if verbose, returns overall performance, as well as performance per chunk type; otherwise, simply returns overall precision, recall, f1 scores

`loss_plot`: Plots the figure of loss vs epochs

`get_charts` : Plots the figures of metrics ( precision, recall, f1) vs epochs

```
from sparknlp_jsl.utils.ner_utils import get_charts, loss_plot, evaluate

metrics = evaluate( preds_df['prediction'].values, preds_df['ground_truth'].values)

loss_plot(f"./ner_logs/{log_file}")

get_charts('./ner_logs/'+log_files[0])
```

### Saving the model and using it in different pipeline

We can save our custom ner model and use it in other pieplines with the `load()` method of `MedicalNerModel`

In [0]:
#saving the model
ner_model.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/custom_NER')

In [0]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = nlp.SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = nlp.Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

#custom ner model
loaded_ner_model = medical.NerModel.load("dbfs:/databricks/driver/models/custom_NER")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

converter = medical.NerConverterInternal()\
    .setInputCols(["document", "token", "ner"])\
    .setOutputCol("ner_span")

ner_prediction_pipeline = nlp.Pipeline(
    stages = [
        document,
        sentence,
        token,
        clinical_embeddings,
        loaded_ner_model,
        converter])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)


We just created a pipeline with our ner model, let's apply it over a sample data and see the results with LightPipeline quickly

In [0]:
from sparknlp.base import LightPipeline

light_model = nlp.LightPipeline(prediction_model)

In [0]:
text = "The patient just received a lung cancer diagnosis"

result = light_model.fullAnnotate(text)[0]

[(i.result, i.metadata['entity']) for i in result['ner_span']]

Out[43]: [('lung cancer', 'Disease')]

As you see, we succesfully trained an NER model with our own data, saved it and got predictions!

**FINE-TUNING NER MODELS:**

You can visit [1.4.Resume_MedicalNer_Model_Training.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb) notebook for fine-tuning pretrained NER models and more details of `MedicalNerApproach()` parameters.

## BertForTokenClassification NER models

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

tokenClassifier = medical.BertForTokenClassification.pretrained("bert_token_classifier_ner_clinical", "en", "clinical/models")\
    .setInputCols("token", "sentence")\
    .setOutputCol("ner")\
    .setCaseSensitive(True)

ner_converter = medical.NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

pipeline =  nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    tokenClassifier,
    ner_converter
    ])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

text = """A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .
Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection .
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation .
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity .
Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 .
Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia .
The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission .
However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L .
The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again .
The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours .
Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use .
The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .
It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge ."""

res = model.transform(spark.createDataFrame([[text]]).toDF("text"))

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
bert_token_classifier_ner_clinical download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
from pyspark.sql import functions as F

res.select(F.explode(F.arrays_zip(res.ner_chunk.result,
                                  res.ner_chunk.begin,
                                  res.ner_chunk.end,
                                  res.ner_chunk.metadata)).alias("cols")) \
   .select(F.expr("cols['3']['sentence']").alias("sentence_id"),
           F.expr("cols['0']").alias("chunk"),
           F.expr("cols['2']").alias("end"),
           F.expr("cols['3']['entity']").alias("ner_label"))\
   .filter("ner_label!='O'")\
   .show(truncate=False)

+-----------+-----------------------------+---+---------+
sentence_id|chunk |end|ner_label|
+-----------+-----------------------------+---+---------+
0 |gestational diabetes mellitus|67 |PROBLEM |
0 |type two diabetes mellitus |153|PROBLEM |
0 |T2DM |160|PROBLEM |
0 |HTG-induced pancreatitis |209|PROBLEM |
0 |an acute hepatitis |280|PROBLEM |
0 |obesity |294|PROBLEM |
0 |a body mass index |317|TEST |
0 |BMI |323|TEST |
0 |polyuria |387|PROBLEM |
0 |polydipsia |400|PROBLEM |
0 |poor appetite |416|PROBLEM |
0 |vomiting |431|PROBLEM |
1 |amoxicillin |521|TREATMENT|
1 |a respiratory tract infection|555|PROBLEM |
2 |metformin |578|TREATMENT|
2 |glipizide |590|TREATMENT|
2 |dapagliflozin |610|TREATMENT|
2 |T2DM |619|PROBLEM |
2 |atorvastatin |636|TREATMENT|
2 |gemfibrozil |652|TREATMENT|
+-----------+-----------------------------+---+---------+
only showing top 20 rows

In [0]:
light_model = nlp.LightPipeline(model)

light_result = light_model.fullAnnotate(text)

from sparknlp_display import NerVisualizer

visualiser = nlp.viz.NerVisualizer()

vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

displayHTML(vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM ( T2DM PROBLEM ), one prior episode of HTG-induced pancreatitis PROBLEM three years prior to presentation , associated with an acute hepatitis PROBLEM , and obesity PROBLEM with a body mass index TEST ( BMI TEST ) of 33.5 kg/m2 , presented with a one-week history of polyuria PROBLEM , polydipsia PROBLEM , poor appetite PROBLEM , and vomiting PROBLEM . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin TREATMENT for a respiratory tract infection PROBLEM . She was on metformin TREATMENT , glipizide TREATMENT , and dapagliflozin TREATMENT for T2DM PROBLEM and atorvastatin TREATMENT and gemfibrozil TREATMENT for HTG PROBLEM . She had been on dapagliflozin TREATMENT for six months at the time of presentation . Physical examination TEST on presentation was significant for dry oral mucosa PROBLEM ; significantly , her abdominal examination TEST was benign with no tenderness PROBLEM , guarding PROBLEM , or rigidity PROBLEM . Pertinent laboratory findings on admission were : serum glucose TEST 111 mg/dl , bicarbonate TEST 18 mmol/l , anion gap TEST 20 , creatinine TEST 0.4 mg/dL , triglycerides TEST 508 mg/dL , total cholesterol TEST 122 mg/dL , glycated hemoglobin ( TEST HbA1c TEST ) TEST 10% , and venous pH TEST 7.27 . Serum lipase TEST was normal at 43 U/L . Serum acetone levels TEST could not be assessed as blood samples TEST kept hemolyzing TEST due to significant lipemia PROBLEM . The patient was initially admitted for starvation ketosis PROBLEM , as she reported poor oral intake PROBLEM for three days prior to admission . However , serum chemistry TEST obtained six hours after presentation revealed her glucose TEST was 186 mg/dL , the anion gap TEST was still elevated PROBLEM at 21 , serum bicarbonate TEST was 16 mmol/L , triglyceride level TEST peaked at 2050 mg/dL , and lipase TEST was 52 U/L . The β-hydroxybutyrate level TEST was obtained and found to be elevated PROBLEM at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer TREATMENT removed prior to analysis TEST due to interference from turbidity PROBLEM caused by lipemia PROBLEM again . The patient was treated with an insulin drip TREATMENT for euDKA PROBLEM and HTG PROBLEM with a reduction in the anion gap TEST to 13 and triglycerides TEST to 1400 mg/dL , within 24 hours . Her euDKA PROBLEM was thought to be precipitated by her respiratory tract infection PROBLEM in the setting of SGLT2 inhibitor use TREATMENT . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine TREATMENT at night , 12 units of insulin lispro TREATMENT with meals , and metformin TREATMENT 1000 mg two times a day . It was determined that all TREATMENT SGLT2 inhibitors TREATMENT should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .

**Training Clinical BertForTokenClassification Model**

For training your own BertForTokenClassification NER model, you can check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.5.BertForTokenClassification_NER_SparkNLP_with_Transformers.ipynb)

## Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once. In Spark NLP we have two different NER profiling pipelines;

- `ner_profiling_clinical` : Returns results for clinical NER models trained with `embeddings_clinical`.
- `ner_profiling_biobert` : Returns results for clinical NER models trained with `biobert_pubmed_base_cased`.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).





<center><b>NER Profiling Clinical Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| jsl_ner_wip_clinical | jsl_ner_wip_greedy_clinical | jsl_ner_wip_modifier_clinical | jsl_rd_ner_wip_greedy_clinical |
| ner_abbreviation_clinical | ner_ade_binary | ner_ade_clinical | ner_anatomy |
| ner_anatomy_coarse | ner_bacterial_species | ner_biomarker | ner_biomedical_bc2gm |
| ner_bionlp | ner_cancer_genetics | ner_cellular | ner_chemd_clinical |
| ner_chemicals | ner_chemprot_clinical | ner_chexpert | ner_clinical |
| ner_clinical_large | ner_clinical_trials_abstracts | ner_covid_trials | ner_deid_augmented |
| ner_deid_enriched | ner_deid_generic_augmented | ner_deid_large | ner_deid_sd |
| ner_deid_sd_large | ner_deid_subentity_augmented | ner_deid_subentity_augmented_i2b2 | ner_deid_synthetic |
| ner_deidentify_dl | ner_diseases | ner_diseases_large | ner_drugprot_clinical |
| ner_drugs | ner_drugs_greedy | ner_drugs_large | ner_events_admission_clinical |
| ner_events_clinical | ner_genetic_variants | ner_human_phenotype_gene_clinical | ner_human_phenotype_go_clinical |
| ner_jsl | ner_jsl_enriched | ner_jsl_greedy | ner_jsl_slim |
| ner_living_species | ner_measurements_clinical | ner_medmentions_coarse | ner_nature_nero_clinical |
| ner_nihss | ner_pathogen | ner_posology | ner_posology_experimental |
| ner_posology_greedy | ner_posology_large | ner_posology_small | ner_radiology |
| ner_radiology_wip_clinical | ner_risk_factors | ner_supplement_clinical | nerdl_tumour_demo |

<b>NER Profiling BioBert Model List</b>

| | |
|-|-|
| ner_cellular_biobert           | ner_clinical_biobert             |
| ner_diseases_biobert           | ner_anatomy_coarse_biobert       |
| ner_events_biobert             | ner_human_phenotype_gene_biobert |
| ner_bionlp_biobert             | ner_posology_large_biobert       |
| ner_jsl_greedy_biobert         | jsl_rd_ner_wip_greedy_biobert    |
| ner_jsl_biobert                | ner_posology_biobert             |
| ner_anatomy_biobert            | jsl_ner_wip_greedy_biobert       |
| ner_jsl_enriched_biobert       | ner_chemprot_biobert             |
| ner_human_phenotype_go_biobert | ner_ade_biobert                  |
| ner_deid_biobert               | ner_risk_factors_biobert         |
| ner_deid_enriched_biobert      | ner_living_species_biobert                                |


</center>

We can easily apply `ner_profiling_clinical` as follows

In [0]:
from sparknlp.pretrained import PretrainedPipeline

clinical_profiling_pipeline = nlp.PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")

ner_profiling_clinical download started this may take some time.
Approx size to download 2.9 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
#sample text for ner profiling
text = '''A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .'''

We will `fullAnnotate()` with our sample text and see the results

In [0]:
clinical_result = clinical_profiling_pipeline.fullAnnotate(text)[0]
clinical_result.keys()

Out[49]: dict_keys(['ner_vop_problem_wip_chunks', 'ner_sdoh_demographics_wip_chunks', 'ner_ade_clinical_chunks', 'ner_deid_augmented', 'ner_deid_subentity_augmented_i2b2', 'ner_posology_greedy_chunks', 'ner_vop_demographic_wip_chunks', 'ner_sdoh_community_condition_wip_chunks', 'ner_radiology_wip_clinical', 'ner_vop_slim_wip', 'ner_oncology_diagnosis_chunks', 'ner_vop_test_wip_chunks', 'ner_jsl_slim', 'ner_vop_wip', 'ner_risk_factors_chunks', 'jsl_ner_wip_clinical_chunks', 'ner_oncology_unspecific_posology_chunks', 'ner_vop_problem_reduced_wip', 'ner_vop_anatomy_wip', 'ner_vop_problem_reduced_wip_chunks', 'ner_eu_clinical_case_chunks', 'ner_oncology_test_chunks', 'ner_deid_synthetic', 'ner_oncology_posology_chunks', 'ner_oncology_tnm_chunks', 'ner_oncology_anatomy_general_chunks', 'ner_drugs_greedy', 'ner_abbreviation_clinical_chunks', 'ner_covid_trials_chunks', 'ner_human_phenotype_gene_clinical_chunks', 'ner_events_admission_clinical', 'jsl_ner_wip_greedy_clinical_chunks', 'ner_posology_greedy', 'ner_cellular_chunks', 'ner_vop_clinical_dept_wip', 'ner_cancer_genetics_chunks', 'ner_biomedical_bc2gm_chunks', 'ner_oncology', 'ner_jsl_greedy', 'jsl_ner_wip_modifier_clinical_chunks', 'ner_drugs_greedy_chunks', 'ner_sdoh_access_to_healthcare_wip', 'ner_deid_sd_large_chunks', 'ner_diseases_chunks', 'ner_diseases_large', 'ner_chemprot_clinical', 'ner_posology_large', 'nerdl_tumour_demo_chunks', 'ner_oncology_chunks', 'ner_deid_subentity_augmented_chunks', 'ner_jsl_enriched_chunks', 'ner_genetic_variants_chunks', 'ner_sdoh_income_social_status_wip_chunks', 'ner_chexpert', 'ner_bionlp_chunks', 'ner_eu_clinical_condition_chunks', 'ner_measurements_clinical_chunks', 'ner_diseases_large_chunks', 'ner_drugs_large', 'ner_living_species_chunks', 'ner_oncology_therapy_chunks', 'ner_clinical_large', 'ner_chemicals', 'ner_sdoh_slim_wip', 'ner_radiology_chunks', 'ner_bacterial_species', 'ner_deid_augmented_chunks', 'ner_bionlp', 'ner_drugprot_clinical', 'ner_anatomy_chunks', 'ner_deid_subentity_augmented_i2b2_chunks', 'ner_deid_large', 'jsl_ner_wip_modifier_clinical', 'ner_vop_slim_wip_chunks', 'ner_chemprot_clinical_chunks', 'ner_eu_clinical_condition', 'nerdl_tumour_demo', 'ner_posology_experimental_chunks', 'ner_cancer_genetics', 'ner_biomarker', 'ner_drugprot_clinical_chunks', 'ner_vop_problem_wip', 'ner_sdoh_mentions', 'ner_posology_small', 'ner_vop_wip_chunks', 'ner_oncology_biomarker_chunks', 'ner_drugs_chunks', 'ner_deid_sd_chunks', 'ner_chemd_clinical', 'ner_human_phenotype_gene_clinical', 'ner_ade_clinical', 'ner_human_phenotype_go_clinical', 'ner_sdoh_wip_chunks', 'ner_sdoh_social_environment_wip', 'ner_sdoh_mentions_chunks', 'ner_risk_factors', 'ner_clinical', 'ner_posology_large_chunks', 'ner_cellular', 'ner_nature_nero_clinical_chunks', 'ner_deid_large_chunks', 'ner_clinical_trials_abstracts_chunks', 'ner_deid_generic_augmented_chunks', 'ner_posology_chunks', 'ner_oncology_anatomy_general', 'ner_sdoh_social_environment_wip_chunks', 'ner_supplement_clinical_chunks', 'ner_oncology_response_to_treatment_chunks', 'ner_deid_enriched_chunks', 'ner_chemd_clinical_chunks', 'ner_deid_sd_large', 'ner_oncology_biomarker', 'jsl_ner_wip_greedy_clinical', 'ner_sdoh_slim_wip_chunks', 'ner_bacterial_species_chunks', 'ner_sdoh_health_behaviours_problems_wip_chunks', 'ner_sdoh_demographics_wip', 'ner_sdoh_community_condition_wip', 'ner_vop_anatomy_wip_chunks', 'ner_living_species', 'ner_abbreviation_clinical', 'ner_oncology_test', 'ner_vop_temporal_wip_chunks', 'ner_diseases', 'jsl_rd_ner_wip_greedy_clinical', 'ner_oncology_response_to_treatment', 'ner_drugs', 'ner_drugs_large_chunks', 'ner_vop_treatment_wip', 'ner_biomarker_chunks', 'ner_clinical_trials_abstracts', 'ner_clinical_large_chunks', 'ner_anatomy_coarse', 'ner_nature_nero_clinical', 'token', 'ner_oncology_anatomy_granular', 'ner_medmentions_coarse', 'ner_oncology_therapy', 'ner_deid_sd', 'jsl_rd_ner_wip_greedy_clinical_chunks', 'ner_oncology_demographics', 'ner_deid_generic_augmented', '

Let's write a function to get the results properly and more clear

In [0]:
def get_token_results(light_result):

  tokens = [j.result for j in light_result["token"]]
  sentences = [j.metadata["sentence"] for j in light_result["token"]]
  begins = [j.begin for j in light_result["token"]]
  ends = [j.end for j in light_result["token"]]
  model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

  df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

  for model_name in model_list:

    temp_df = pd.DataFrame(light_result[model_name])
    temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
    temp_df = temp_df[["jsl_label"]]

    # temp_df = get_ner_result(model_name)
    temp_df.columns = [model_name]
    df = pd.concat([df, temp_df], axis=1)

  return df

In [0]:
get_token_results(clinical_result)

Out[51]:

,sentence,begin,end,token,ner_deid_augmented,ner_deid_subentity_augmented_i2b2,ner_radiology_wip_clinical,ner_vop_slim_wip,ner_jsl_slim,ner_vop_wip,ner_vop_problem_reduced_wip,ner_vop_anatomy_wip,ner_deid_synthetic,ner_drugs_greedy,ner_events_admission_clinical,ner_posology_greedy,ner_vop_clinical_dept_wip,ner_oncology,ner_jsl_greedy,ner_sdoh_access_to_healthcare_wip,ner_diseases_large,ner_chemprot_clinical,ner_posology_large,ner_chexpert,ner_drugs_large,ner_clinical_large,ner_chemicals,ner_sdoh_slim_wip,ner_bacterial_species,ner_bionlp,ner_drugprot_clinical,ner_deid_large,jsl_ner_wip_modifier_clinical,ner_eu_clinical_condition,nerdl_tumour_demo,ner_cancer_genetics,ner_biomarker,ner_vop_problem_wip,ner_sdoh_mentions,ner_posology_small,ner_chemd_clinical,ner_human_phenotype_gene_clinical,ner_ade_clinical,ner_human_phenotype_go_clinical,ner_sdoh_social_environment_wip,ner_risk_factors,ner_clinical,ner_cellular,ner_oncology_anatomy_general,ner_deid_sd_large,...,ner_sdoh_demographics_wip,ner_sdoh_community_condition_wip,ner_living_species,ner_abbreviation_clinical,ner_oncology_test,ner_diseases,jsl_rd_ner_wip_greedy_clinical,ner_oncology_response_to_treatment,ner_drugs,ner_vop_treatment_wip,ner_clinical_trials_abstracts,ner_anatomy_coarse,ner_nature_nero_clinical,ner_oncology_anatomy_granular,ner_medmentions_coarse,ner_oncology_therapy,ner_deid_sd,ner_oncology_demographics,ner_deid_generic_augmented,ner_anatomy,ner_deid_enriched,ner_deid_subentity_augmented,ner_oncology_diagnosis,ner_sdoh_health_behaviours_problems_wip,ner_measurements_clinical,ner_ade_binary,ner_nihss,ner_pathogen,ner_vop_test_wip,ner_jsl_enriched,ner_oncology_tnm,ner_posology_experimental,jsl_ner_wip_clinical,ner_biomedical_bc2gm,ner_vop_temporal_wip,ner_oncology_wip,ner_vop_demographic_wip,ner_sdoh_income_social_status_wip,ner_jsl,ner_sdoh_substance_usage_wip,ner_events_clinical,ner_supplement_clinical,ner_oncology_unspecific_posology,ner_genetic_variants,ner_sdoh_wip,ner_radiology,ner_eu_clinical_case,ner_posology,ner_oncology_posology,ner_covid_trials
0,0,0,0,A,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-patient,O,O,O
1,0,2,12,28-year-old,O,O,O,O,B-Age,B-Age,O,O,O,O,O,O,O,B-Gender,B-Age,O,O,O,O,O,O,O,O,B-Marital_Status,O,O,O,O,B-Age,O,O,O,B-Age,O,O,O,O,O,O,O,O,O,O,O,O,O,...,B-Gender,O,O,O,O,O,B-Age,O,O,O,O,O,B-Person,O,O,O,O,B-Gender,B-AGE,O,O,B-AGE,O,O,O,O,O,O,O,B-Age,O,O,B-Age,O,O,B-Gender,B-Age,O,B-Age,O,O,O,O,O,B-Gender,O,I-patient,O,O,B-Gender
2,0,14,19,female,O,O,O,B-Gender,B-Demographics,B-Gender,O,O,O,O,O,O,O,B-Gender,B-Gender,O,O,O,O,O,O,O,O,B-Gender,O,B-Organism,O,O,B-Gender,O,O,O,B-Gender,O,O,O,O,O,O,O,O,O,O,O,O,O,...,B-Gender,O,B-HUMAN,O,O,O,B-Gender,O,O,O,O,O,I-Person,O,B-Organism_Attribute,O,O,B-Gender,O,O,O,O,O,O,O,O,O,O,O,B-Gender,O,O,B-Gender,O,O,B-Gender,B-Gender,O,B-Gender,O,O,O,O,O,B-Gender,O,I-patient,O,O,B-Gender
3,0,21,24,with,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
4,0,26,26,a,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0,409,416,appetite,O,O,I-Symptom,I-Symptom,I-Symptom,I-Symptom,I-Problem,O,O,O,I-PROBLEM,O,O,O,I-Symptom,O,O,O,O,O,O,I-PROBLEM,O,O,O,O,O,O,I-Symptom,I-clinical_condition,

**Let's plot the labelled entity counts by NER models.****Let's plot the labelled entity counts by NER models.**

In [0]:
import plotly.graph_objects as go

result_df = get_token_results(clinical_result)

label_result_df = result_df[result_df.columns[4:]].copy()
label_count_list = [label_result_df[label_result_df[i] != "O"].shape[0] for i in label_result_df.columns]

fig = go.Figure(go.Bar(x=label_count_list, y=label_result_df.columns, orientation='h'))

fig.update_layout(
    autosize = False,
    width = 1500,
    height = 1500,
    margin = dict(l=50, r=50, b=100, t=100, pad=4),
    paper_bgcolor = "LightSteelBlue",
    title={'text': "Counts of Labelled Entities", 'y':0.98, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    titlefont=dict(size=30),
    yaxis=dict(title_text="Clinical NER Models", titlefont=dict(size=30)),
)

fig.show()

## NER Model Finder Pretrained Pipeline
`ner_model_finder`  pretrained pipeline trained with bert embeddings that can be used to find the most appropriate NER model given the entity name.

In [0]:
from sparknlp.pretrained import PretrainedPipeline
finder_pipeline = nlp.PretrainedPipeline("ner_model_finder", "en", "clinical/models")

ner_model_finder download started this may take some time.
Approx size to download 148.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
result = finder_pipeline.fullAnnotate("oncology")[0]
result.keys()

Out[54]: dict_keys(['model_names'])

From the metadata in the `model_names` column, we'll get to the top models to the given 'oncology' entity and oncology related categories.

In [0]:
df= pd.DataFrame(zip(result["model_names"][0].metadata["all_k_resolutions"].split(":::"),
                     result["model_names"][0].metadata["all_k_results"].split(":::")),
                 columns=["category", "top_models"])

df.head(10)

Out[55]:

,category,top_models
0,oncology therapy,"['ner_jsl', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical', 'ner_jsl_enriched..."
1,clinical department,"['ner_jsl', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical', 'ner_events_clini..."
2,biomedical unit,['ner_clinical_trials_abstracts']
3,cancer genetics,['ner_cancer_genetics']
4,anatomy,"['ner_bionlp', 'ner_medmentions_coarse', 'ner_chexpert', 'ner_anatomy_coarse', 'ner_anatomy', 'n..."
5,cancer surgery,['ner_biomarker']
6,molecular biology research technique,['ner_medmentions_coarse']
7,dysarthria,['ner_nihss']
8,oncological,"['ner_jsl', 'ner_biomarker', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical', ..."
9,hospital,"['ner_deid_subentity_augmented', 'ner_deid_subentity_augmented_i2b2', 'ner_deid_subentity_glove'..."


**NER Model Playground:**

You can check [Spark NLP Clinical NER Model Playground Notebook](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/Spark_NLP_Clinical_NER_Playground_Streamlit_app.ipynb#scrollTo=yFLcEPoAPB48) for a streamlit demo with all NER models in Spark NLP.

# 2.ContextualParser (Rule Based NER)

Spark NLP's `ContextualParser` is a licensed annotator that allows users to extract entities from a document based on pattern matching. It provides more functionality than its open-source counterpart `EntityRuler` by allowing users to customize specific characteristics for pattern matching. You're able to find entities using regex rules for full and partial matches, a dictionary with normalizing options and context parameters to take into account things such as token distances.

There are 3 components necessary to understand when using the `ContextualParser` annotator:

- **`ContextualParser` annotator's parameters**
Here are all the parameters available to use with the ContextualParserApproach
```
contextualParser = ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("entity") \
    .setCaseSensitive(True) \
    .setJsonPath("context_config.json") \
    .setPrefixAndSuffixMatch(True) \
    .setDictionary("dictionary.tsv", options=.    
                    {"orientation":"vertical"})
```

- **JSON configuration file** ContextualParser extracts entity from a document based on user defined rules. Rule matching is based on a RegexMatcher defined in a JSON file. It is set through the parameter `setJsonPath()`. In this JSON file, regex is defined that you want to match along with the information that will output on metadata field.

- **Dictionary** You can specify a path to a dictionary in tsv or csv format using the `setDictionary()` parameter. Using a dictionary is a useful when you have a list of exact words that you want the parser to pick up when processing some text.

For more details about the components(parameters, JSON and dictionary), please visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.2.Contextual_Parser_Rule_Based_NER.ipynb)

Now, we will create a dictionary and json file to build a sample `ContextualParser` pipeline to detect "gender" and "age" from given text. And merge their results with NER model which we will get only `DATE` and `LOCATION` labels.

In [0]:
# Here's some sample text
sample_text = """The patient is a 28 years old female with a history of gestational diabetes mellitus was diagnosed in April 2002 in County Baptist Hospital .
                 3 years ago, he reported an episode of HTG-induced pancreatitis .
                 5 months old boy with repeated concussions."""

In [0]:
# Create a dictionary to detect gender
gender = '''male,man,male,boy,gentleman,he,him
female,woman,female,girl,lady,old-lady,she,her
neutral,they,neutral,it'''

with open('/dbfs/gender.csv', 'w') as f:
    f.write(gender)

# Check what dictionary looks like
!cat /dbfs/gender.csv

male,man,male,boy,gentleman,he,him
female,woman,female,girl,lady,old-lady,she,her
neutral,they,neutral,it

In [0]:
# Create JSON file for gender
gender = {
  "entity": "Gender",
  "ruleScope": "sentence", 
  "completeMatchRegex": "true",
  "matchScope":"token"
}

import json
with open('/dbfs/gender.json', 'w') as f:
    json.dump(gender, f)

In [0]:
# Create JSON file for age
age = {
  "entity": "Age",
  "ruleScope": "sentence",
  "matchScope":"token",
  "regex":"\d{1,3}",
  "prefix":["age of", "age"],
  "suffix": ["-years-old", "years-old", "-year-old",
             "-months-old", "-month-old", "-months-old",
             "-day-old", "-days-old", "month old",
             "days old", "year old", "years old", 
             "years", "year", "months", "old"],
  "contextLength": 25,
  "contextException": ["ago"],
  "exceptionDistance": 12
}

with open('/dbfs/age.json', 'w') as f:
    json.dump(age, f)

In [0]:
# Build pipeline
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = nlp.SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

gender_contextual_parser = medical.ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("gender_chunk") \
    .setJsonPath("/dbfs/gender.json") \
    .setCaseSensitive(False) \
    .setDictionary('file:/dbfs/gender.csv', options={"delimiter":","}) \
    .setPrefixAndSuffixMatch(False)      

age_contextual_parser = medical.ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("chunk_age") \
    .setJsonPath("/dbfs/age.json") \
    .setCaseSensitive(False) \
    .setPrefixAndSuffixMatch(False)\
    .setShortestContextMatch(True)\
    .setOptionalContextRules(False)

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

deid_ner = medical.NerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("deid_ner")

deid_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "deid_ner"]) \
    .setOutputCol("deid_ner_chunk")\
    .setWhiteList(["DATE", "LOCATION"])

# merge CP and NER model chunks
chunk_merger = medical.ChunkMergeApproach() \
    .setInputCols(["gender_chunk", "age_chunk", "deid_ner_chunk"]) \
    .setOutputCol("ner_chunk")

parserPipeline = nlp.Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        gender_contextual_parser,
        age_contextual_parser,
        word_embeddings,
        deid_ner,
        deid_ner_converter,
        chunk_merger
        ])

# Create a lightpipeline model
parserModel = parserPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = nlp.LightPipeline(parserModel)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_generic_augmented download started this may take some time.
[ | ][OK!]

**In the metadata, `ner_source` is for showing the source of the entities.**

In [0]:
# Annotate the sample text
annotations = light_model.fullAnnotate(sample_text)[0]

# Check outputs
annotations.get('ner_chunk')

Out[61]: [Annotation(chunk, 30, 35, female, {'tokenIndex': '7', 'entity': 'Gender', 'field': 'Gender', 'chunk': '0', 'normalized': 'female', 'sentence': '0', 'confidenceValue': '0.50'}, []),
 Annotation(chunk, 102, 111, April 2002, {'entity': 'DATE', 'confidence': '0.99950004', 'ner_source': 'deid_ner_chunk', 'chunk': '1', 'sentence': '0'}, []),
 Annotation(chunk, 116, 138, County Baptist Hospital, {'entity': 'LOCATION', 'confidence': '0.97850007', 'ner_source': 'deid_ner_chunk', 'chunk': '2', 'sentence': '0'}, []),
 Annotation(chunk, 172, 173, he, {'tokenIndex': '4', 'entity': 'Gender', 'field': 'Gender', 'chunk': '3', 'normalized': 'male', 'sentence': '1', 'confidenceValue': '0.50'}, []),
 Annotation(chunk, 255, 257, boy, {'tokenIndex': '3', 'entity': 'Gender', 'field': 'Gender', 'chunk': '4', 'normalized': 'male', 'sentence': '2', 'confidenceValue': '0.50'}, [])]

In [0]:

visualiser = NerVisualizer()

ner_vis = visualiser.display(annotations, label_col='ner_chunk', document_col='document', return_html=True)
  
displayHTML(ner_vis)

The patient is a 28 years old female Gender with a history of gestational diabetes mellitus was diagnosed in April 2002 DATE in County Baptist Hospital LOCATION . 3 years ago, he Gender reported an episode of HTG-induced pancreatitis . 5 months old boy Gender with repeated concussions.

# 3.Clinical Assertion Status

The deep neural network architecture for assertion status detection in Spark NLP is based on a Bi-LSTM framework, and is a modified version of the architecture proposed by Federico Fancellu, Adam Lopez and Bonnie Webber ([Neural Networks For Negation Scope Detection](https://aclanthology.org/P16-1047.pdf)). Its goal is to classify the assertions made on given medical concepts as being present, absent, or possible in the patient, conditionally present in the patient under certain circumstances,
hypothetically present in the patient at some future point, and
mentioned in the patient report but associated with someoneelse.
In the proposed implementation, input units depend on the
target tokens (a named entity) and the neighboring words that
are explicitly encoded as a sequence using word embeddings.
Similar to paper mentioned above,  it is observed that that 95% of the scope tokens (neighboring words) fall in a window of 9 tokens to the left and 15
to the right of the target tokens in the same dataset. Therefore, the same window size was implemented and it following parameters were used: learning
rate 0.0012, dropout 0.05, batch size 64 and a maximum sentence length 250. The model has been implemented within
Spark NLP as an annotator called AssertionDLModel. After
training 20 epoch and measuring accuracy on the official test
set, this implementation exceeds the latest state-of-the-art
accuracy benchmarks as summarized as following table:

|Assertion Label|Spark NLP|Latest Best|
|-|-|-|
|Absent       |0.944 |0.937|
|Someone-else |0.904|0.869|
|Conditional  |0.441|0.422|
|Hypothetical |0.862|0.890|
|Possible     |0.680|0.630|
|Present      |0.953|0.957|
|micro F1     |0.939|0.934|


### Pretrained Assertion Status Models

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
|  1 | [assertion_dl](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_en.html)            |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  2 | [assertion_dl_biobert](https://nlp.johnsnowlabs.com/2021/01/26/assertion_dl_biobert_en.html)    |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  3 | [assertion_dl_healthcare](https://nlp.johnsnowlabs.com/2020/09/23/assertion_dl_healthcare_en.html) |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  4 | [assertion_dl_large](https://nlp.johnsnowlabs.com/2020/05/21/assertion_dl_large_en.html)      |Present, Absent, Possible, conditional, hypothetical, associated_with_someone_else|
|  5 | [assertion_dl_radiology](https://nlp.johnsnowlabs.com/2021/03/18/assertion_dl_radiology_en.html)   |Confirmed, Suspected, Negative|
|  6 | [assertion_jsl](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_en.html)           |Present, Absent, Possible, Planned, Someoneelse, Past, Family, Hypotetical|
|  7 | [assertion_jsl_large](https://nlp.johnsnowlabs.com/2021/07/24/assertion_jsl_large_en.html)     |present, absent, possible, planned, someoneelse, past, hypothetical|
|  8 |  [assertion_ml](https://nlp.johnsnowlabs.com/2020/01/30/assertion_ml_en.html) |Hypothetical, Present, Absent, Possible, Conditional, Associated_with_someone_else|
|  9 | [assertion_dl_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/17/assertion_dl_scope_L10R10_en_3_0.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 10 | [assertion_dl_biobert_scope_L10R10](https://nlp.johnsnowlabs.com/2022/03/24/assertion_dl_biobert_scope_L10R10_en_2_4.html)| hypothetical, associated_with_someone_else, conditional, possible, absent, present|
| 11 | [assertion_jsl_augmented](https://nlp.johnsnowlabs.com/2022/09/15/assertion_jsl_augmented_en.html)| Present, Absent, Possible, Planned, Past, Family, Hypotetical, SomeoneElse|






### Oncology Assertion Models

|    | model_name              |Predicted Entities|
|---:|:------------------------|-|
| 1 | [assertion_oncology_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_wip_en.html) | Medical_History, Family_History, Possible, Hypothetical_Or_Absent|
| 2 | [assertion_oncology_problem_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_problem_wip_en.html) |Present, Possible, Hypothetical, Absent, Family|
| 3 | [assertion_oncology_treatment_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_treatment_binary_wip_en.html) |Present, Planned, Past, Hypothetical, Absent|
| 3 | [assertion_oncology_treatment_wip]() |Present, Planned, Past, Hypothetical, Absent|
| 4 | [assertion_oncology_response_to_treatment_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_response_to_treatment_wip_en.html) |Present_Or_Past, Hypothetical_Or_Absent|
| 5 | [assertion_oncology_test_binary_wip](https://nlp.johnsnowlabs.com/2022/10/01/assertion_oncology_test_binary_wip_en.html) |Present_Or_Past, Hypothetical_Or_Absent|
| 6 | [assertion_oncology_smoking_status_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_smoking_status_wip_en.html) |Absent, Past, Present|
| 7 | [assertion_oncology_family_history_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_family_history_wip_en.html) |Family_History, Other|
| 8 | [assertion_oncology_demographic_binary_wip](https://nlp.johnsnowlabs.com/2022/10/11/assertion_oncology_demographic_binary_wip_en.html) |Patient, Someone_Else|

Let's start to create a Pipeline in order to detect assertion status with `assertion_jsl_augmented` pretrained model. <br/>
This model detects following assertion status;
`Absent`, `Present`, `Past`, `Family`, `Planned`, `SomeoneElse`, `Hypotetical`, `Possible`.

And we will use **`AssertionFilterer`** for filtering out the NERs by the list of acceptable assertion statuses. This annotator would be quite handy if you want to set a white list for the acceptable assertion statuses like `Present` or `Planned`; and do not want `Absent` conditions get out of your pipeline.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

clinical_ner = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "clinical_ner"]) \
    .setOutputCol("clinical_ner_chunk")


# Assertion model trained on i2b2 (sampled from MIMIC) dataset
clinical_assertion = medical.AssertionDLModel.pretrained("assertion_jsl_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "clinical_ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    # .setScopeWindow([9,15]) # default [9,15]

# filter out the named entities by the list of acceptable assertion statuses. Present here.
assertion_filterer = medical.AssertionFilterer()\
    .setInputCols("sentence","clinical_ner_chunk","assertion")\
    .setOutputCol("assertion_filtered")\
    .setCaseSensitive(False)\
    .setWhiteList(["Present"])

assertionPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    clinical_ner_converter,
    clinical_assertion,
    assertion_filterer
    ])


assertion_model = assertionPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
[ | ][OK!]
assertion_jsl_augmented download started this may take some time.
[ | ][ / ][OK!]

Creating example dataframe with sample text and check the results

In [0]:
#text = """
#Patient with severe fever and sore throat also suffers from pain.
#He shows no stomach pain and he maintained on an epidural and PCA for pain control.
#He also become short of breath with climbing a flight of stairs.
#After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
#"""

text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. Alopecia noted. She denies pain.'


data = spark.createDataFrame([[text]]).toDF("text")

# fit/transform and show
results = assertion_model.transform(data)
results.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| clinical_ner| clinical_ner_chunk| assertion| assertion_filtered|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Patient has a hea...|[{document, 0, 14...|[{document, 0, 10...|[{token, 0, 6, Pa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 12, 21, ...|[{assertion, 12, ...|[{chunk, 12, 21, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
results.select(F.explode(F.arrays_zip(results.clinical_ner_chunk.result, results.clinical_ner_chunk.metadata, results.assertion.result)).alias("cols"))\
       .select(F.expr("cols['0']").alias("ner_chunk"),
               F.expr("cols['1']['entity']").alias("entities"),
               F.expr("cols['2']").alias("assertion")).show(truncate=False)

+----------+--------+------------+
ner_chunk |entities|assertion |
+----------+--------+------------+
a headache|PROBLEM |Present |
a head CT |TEST |Hypothetical|
anxious |PROBLEM |Possible |
Alopecia |PROBLEM |Present |
pain |PROBLEM |Absent |
+----------+--------+------------+

#### Assertion Filterer Results

In [0]:
assertion_filterer.getWhiteList()

Out[66]: ['Present']

Here are our filtered (only present assertions) chunks;

In [0]:
results.select("assertion_filtered.result").show(truncate=False)

+----------------------+
result |
+----------------------+
[a headache, Alopecia]|
+----------------------+

As you see above, there is no **"a head C"**, **"anxious"** and **"pain"** in filtered results

### Assertion Visualizer

Also, we can visualize the assertion results by using `AssertionVisualizer` of Sparknlp-Display library.

Now, we will create a LightPipeline with our existing model and visualize the results.

In [0]:
from sparknlp_display import AssertionVisualizer

light_model = LightPipeline(assertion_model)
light_result = light_model.fullAnnotate(text)[0]

vis = AssertionVisualizer()
asser_vis= vis.display(light_result, 'clinical_ner_chunk', 'assertion', return_html=True)
displayHTML(asser_vis)

Patient has a headache PROBLEM Present for the last 2 weeks, needs to get a head CT TEST Hypothetical , and appears anxious PROBLEM Possible when she walks fast. Alopecia PROBLEM Present noted. She denies pain PROBLEM Absent .

As you see above, there is no **"a head C"**, **"anxious"** and **"pain"** in filtered results

### Assertion Visualizer

Also, we can visualize the assertion results by using `AssertionVisualizer` of Sparknlp-Display library.

Now, we will create a LightPipeline with our existing model and visualize the results.

### Train a Custom Assertion Model

In this section, we will train a custom assertion model. <br/>

Firstly, download a sample dataset for training.

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_assertion_sample_short.csv

In [0]:
import pandas as pd

In [0]:

assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("/i2b2_assertion_sample_short.csv")
assertion_df.show(5, truncate=100)

+-----------------------------------------------------------------------------------------------+--------------------+-------+-----+---+
 text| target| label|start|end|
+-----------------------------------------------------------------------------------------------+--------------------+-------+-----+---+
 She has no history of liver disease , hepatitis .| liver disease| absent| 5| 6|
 1. Undesired fertility .| undesired fertility|present| 1| 2|
 3) STATUS POST FALL .| fall|present| 3| 3|
 On physical exam , she is a healthy-appearing female with a normal voice and no exophthalmos .| exophthalmos| absent| 15| 15|
Abdomen distended , mild tenderness on the left , bowel sounds present , nohepatosplenomegaly .|nohepatosplenomegaly| absent| 13| 13|
+-----------------------------------------------------------------------------------------------+--------------------+-------+-----+---+
only showing top 5 rows

Splitting data into train and test set

In [0]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 721
Test Dataset Count: 170

In [0]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)

+-------+-----+
label |count|
+-------+-----+
present|546 |
absent |175 |
+-------+-----+

We will transform our test data with a pipeline consisting of same steps with the pipeline which contains `AssertionDLApproach`.
By doing this, we enable that the test data will have the same columns with the training data in `AssertionDLApproach`. <br/>
The goal of this implementation is enabling the usage of `setTestDataset()` parameter in `AssertionDLApproach`.

In [0]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = nlp.Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = nlp.Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings")

clinical_assertion_pipeline = nlp.Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings])

assertion_test_data = clinical_assertion_pipeline.fit(test_data).transform(test_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
assertion_test_data.columns

Out[75]: ['text',
 'target',
 'label',
 'start',
 'end',
 'document',
 'chunk',
 'token',
 'embeddings']

We will save the test data in parquet format to use in `AssertionDLApproach()`.

In [0]:
assertion_test_data.write.parquet('dbfs:/i2b2_assertion_sample_test_data.parquet')

#### Assertion Graph

Also, we could use custom graph like following;

In [0]:
!pip install -q tensorflow==2.11.0
!pip install -q tensorflow-addons

WARNING: You are using pip version 21.0.1; however, version 23.2.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
 WARNING: You are using pip version 21.0.1; however, version 23.2.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

We will use TFGraphBuilder annotator which can be used to create graphs in the model training pipeline.

TFGraphBuilder inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [0]:
assertion_graph_builder = TFGraphBuilder()\
    .setModelName("assertion_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder('file:/dbfs/tf_graphs')\
    .setGraphFile("assertion_graph.pb")\
    .setMaxSequenceLength(250)\
    .setHiddenUnitsNumber(25)

In [0]:
'''
# ready to use tf_graph

!mkdir training_logs
!mkdir assertion_tf_graph

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/tf_graphs/blstm_34_32_30_200_2.pb -P /content/assertion_tf_graph
'''

Out[82]: '\n# ready to use tf_graph\n\n!mkdir training_logs\n!mkdir assertion_tf_graph\n\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/tf_graphs/blstm_34_32_30_200_2.pb -P /content/assertion_tf_graph\n'

In [0]:
'''
# create custom graph

from sparknlp_jsl.training import tf_graph
tf_graph.print_model_params("assertion_dl")

feat_size = 200
n_classes = 6

tf_graph.build("assertion_dl",
              build_params={"n_classes": n_classes},
              model_location= "./tf_graphs",
              model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))
'''

Out[83]: '\n# create custom graph\n\nfrom sparknlp_jsl.training import tf_graph\ntf_graph.print_model_params("assertion_dl")\n\nfeat_size = 200\nn_classes = 6\n\ntf_graph.build("assertion_dl",\n build_params={"n_classes": n_classes},\n model_location= "./tf_graphs",\n model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))\n'

**Setting the Scope Window (Target Area) Dynamically in Assertion Status Detection Models**


This parameter allows you to train the Assertion Status Models to focus on specific context windows when resolving the status of a NER chunk. The window is in format `[X,Y]` being `X` the number of tokens to consider on the left of the chunk, and `Y` the max number of tokens to consider on the right. Let’s take a look at what different windows mean:


*   By default, the window is `[-1,-1]` which means that the Assertion Status will look at all of the tokens in the sentence/document (up to a maximum of tokens set in `setMaxSentLen()` ).
*   `[0,0]` means “don’t pay attention to any token except the ner_chunk”, what basically is not considering any context for the Assertion resolution.
*   `[9,15]` is what empirically seems to be the best baseline, meaning that we look up to 9 tokens on the left and 15 on the right of the ner chunk to understand the context and resolve the status.


Check this [Scope Window Tuning Assertion Status Detection notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/2.1.Scope_window_tuning_assertion_status_detection.ipynb)  that illustrates the effect of the different windows and how to properly fine-tune your AssertionDLModels to get the best of them.

In our case, the best Scope Window is around [10,10]

Now, let's define the `AssertionDLApproach()` to train a model

In [0]:
dbutils.fs.ls('/dbfs/ner')

Out[79]: [FileInfo(path='dbfs:/dbfs/ner/assertion_graphs', name='assertion_graphs', size=1332219, modificationTime=1650993066000),
 FileInfo(path='dbfs:/dbfs/ner/assertion_tf_graph', name='assertion_tf_graph', size=1332219, modificationTime=1672398228000),
 FileInfo(path='dbfs:/dbfs/ner/medical_ner_graphs', name='medical_ner_graphs', size=1754469, modificationTime=1672397701000),
 FileInfo(path='dbfs:/dbfs/ner/ner_logs/', name='ner_logs/', size=0, modificationTime=1689104351000),
 FileInfo(path='dbfs:/dbfs/ner/ner_logs_best/', name='ner_logs_best/', size=0, modificationTime=1665662120000),
 FileInfo(path='dbfs:/dbfs/ner/re_graphs', name='re_graphs', size=135162, modificationTime=1650999349000)]

In [0]:
scope_window = [10,10]

assertionStatus = medical.AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(20)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('dbfs:/assertion_logs')\
    .setGraphFolder('dbfs:/tf_graphs')\
    .setGraphFile("file:/dbfs/tf_graphs/assertion_graph.pb")\
    .setTestDataset(path="/i2b2_assertion_sample_test_data.parquet", read_as='SPARK', options={'format': 'parquet'})\
    .setScopeWindow(scope_window)


'''
If .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed
by a pipeline consisting of document, chunk, embeddings stages.
'''

Out[80]: '\nIf .setTestDataset parameter is employed, raw test data cannot be fitted. .setTestDataset only works for dataframes which are correctly transformed\nby a pipeline consisting of document, chunk, embeddings stages.\n'

In [0]:
'''
assertionStatus = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setMaxIter(100) # default: 26
'''

Out[86]: '\nassertionStatus = AssertionLogRegApproach() .setLabelCol("label") .setInputCols("document", "chunk", "embeddings") .setOutputCol("assertion") .setMaxIter(100) # default: 26\n'

In [0]:
%%time

#training pipeline
clinical_assertion_pipeline = nlp.Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertion_graph_builder,
    assertionStatus])

assertion_model = clinical_assertion_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: assertion_dl
Graph folder: file:/dbfs/tf_graphs
Graph file name: assertion_graph.pb
Build params: {'n_classes': 2, 'feat_size': 200, 'max_seq_len': 250, 'n_hidden': 25}
Device mapping: no known devices.
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/sparknlp_jsl/_tf_graph_builders/tf2contrib/rnn.py:229: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/rnn.py:437: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Device mapping: no known devices.
assertion_dl graph exported to file:/dbfs/tf_graphs/assertion_graph.pb
CPU times: user 5.91 s, sys: 1.38 s, total: 7.29 s
Wall time: 1min 59s

In [0]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

+-------+---------+
 label| result|
+-------+---------+
present|[present]|
 absent|[present]|
present|[present]|
present|[present]|
present|[present]|
present| [absent]|
present|[present]|
present|[present]|
present| [absent]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
present| [absent]|
present|[present]|
present|[present]|
present|[present]|
present|[present]|
+-------+---------+
only showing top 20 rows

In [0]:
preds_df = preds.toPandas()
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])
preds_df

Out[83]:

,label,result
0,present,present
1,absent,present
2,present,present
3,present,present
4,present,present
...,...,...
165,present,present
166,absent,absent
167,absent,absent
168,absent,absent


#### Evaluating the Model

We are going to use sklearn to evalute the results on test dataset

In [0]:
from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))

precision recall f1-score support

 absent 0.71 0.45 0.55 53
 present 0.79 0.91 0.85 117

 accuracy 0.77 170
 macro avg 0.75 0.68 0.70 170
weighted avg 0.76 0.77 0.75 170

And we can save the trained model

In [0]:
assertion_model.stages[-1].write().overwrite().save('/databricks/driver/models/assertion_model')

And we can save the trained model

# 4.Clinical Deidentification

Protected Health Information:
- individual’s past, present, or future physical or mental health or condition
- provision of health care to the individual
- past, present, or future payment for the health care

Protected health information includes many common identifiers (e.g., name, address, birth date, Social Security Number) when they can be associated with the health information.

<center><b>Deidentification NER Models for English</b></center>

|index|model|lang|index|model|lang|index|model|lang|
|-----:|:-----|----|-----:|:-----|----|-----:|:-----|----|
| 1| [deidentify_dl](https://nlp.johnsnowlabs.com/2021/01/28/deidentify_dl_en.html)  |en| 7| [ner_deid_enriched_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_enriched_biobert_en.html)  |en| 13| [ner_deid_subentity_augmented](https://nlp.johnsnowlabs.com/2021/09/03/ner_deid_subentity_augmented_en.html)  |en|
| 2| [deidentify_large](https://nlp.johnsnowlabs.com/2020/08/04/deidentify_large_en.html)  |en| 8| [ner_deid_generic_augmented](https://nlp.johnsnowlabs.com/2021/06/30/ner_deid_generic_augmented_en.html)  |en| 14| [ner_deid_subentity_augmented_i2b2](https://nlp.johnsnowlabs.com/2021/11/29/ner_deid_subentity_augmented_i2b2_en.html)  |en|
| 3| [deidentify_rb](https://nlp.johnsnowlabs.com/2019/06/04/deidentify_rb_en.html)  |en| 9| [ner_deid_generic_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_generic_glove_en.html)  |en| 15| [ner_deid_subentity_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_subentity_glove_en.html)  |en|
| 4| [ner_deid_augmented](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_augmented_en.html)  |en| 10| [ner_deid_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_large_en.html)  |en| 16| [ner_deid_synthetic](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_synthetic_en.html)  |en|
| 5| [ner_deid_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_biobert_en.html)  |en| 11| [ner_deid_sd](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_en.html)  |en| 17| [ner_deidentify_dl](https://nlp.johnsnowlabs.com/2021/03/31/ner_deidentify_dl_en.html)  |en|
| 6| [ner_deid_enriched](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_enriched_en.html)  |en| 12| [ner_deid_sd_large](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_large_en.html)  |en| 18| []()|


<center><b>Deidentification NER Models for Other Languages</b></center>

|index|model|lang|index|model|lang|
|-----:|:-----|----|-----:|:-----|----|
| 1| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/01/06/ner_deid_generic_de.html)  |de| 11| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/02/11/ner_deid_generic_fr.html)  |fr|
| 2| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/01/06/ner_deid_subentity_de.html)  |de| 12| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/02/14/ner_deid_subentity_fr.html)  |fr|
| 3| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/01/18/ner_deid_generic_es.html)  |es| 13| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/03/25/ner_deid_generic_it_3_0.html)  |it|
| 4| [ner_deid_generic_augmented](https://nlp.johnsnowlabs.com/2022/02/16/ner_deid_generic_augmented_es.html)  |es| 14| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/03/25/ner_deid_subentity_it_2_4.html)  |it|
| 5| [ner_deid_generic_roberta](https://nlp.johnsnowlabs.com/2022/01/17/ner_deid_generic_roberta_es.html)  |es| 15| [ner_deid_generic](https://nlp.johnsnowlabs.com/2022/04/13/ner_deid_generic_pt_3_0.html)  |pt|
| 6| [ner_deid_generic_roberta_augmented](https://nlp.johnsnowlabs.com/2022/02/16/ner_deid_generic_roberta_augmented_es.html)  |es| 16| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/04/13/ner_deid_subentity_pt_3_0.html)  |pt|
| 7| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/01/18/ner_deid_subentity_es.html)  |es| 17| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2022/06/27/ner_deid_subentity_ro_3_0.html)  |ro|
| 8| [ner_deid_subentity_augmented](https://nlp.johnsnowlabs.com/2022/02/16/ner_deid_subentity_augmented_es.html)  |es| 18| [ner_deid_subentity_bert](https://nlp.johnsnowlabs.com/2022/06/27/ner_deid_subentity_bert_ro_3_0.html)  |ro|
| 9| [ner_deid_subentity_roberta](https://nlp.johnsnowlabs.com/2022/01/17/ner_deid_subentity_roberta_es.html)  |es| 19| [ner_deid_generic](https://nlp.johnsnowlabs.com/models)  |ro|
| 10| [ner_deid_subentity_roberta_augmented](https://nlp.johnsnowlabs.com/2022/02/16/ner_deid_subentity_roberta_augmented_es.html)  |es| 20| [ner_deid_generic_bert](https://nlp.johnsnowlabs.com/models)  |ro|


You can find German, Spanish, French, Portuguese, Romanian and Italian deidentification models and pretrained pipeline examples in [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.1.Clinical_Multi_Language_Deidentification.ipynb):



**ONE-LINER DEIDENTIFICATION MODULE** <br/>
Additionally, Spark NLP for Healthcare provides functionality to apply Deidentification using **one-liner module** called `Deid`. <br/>

The `Deid` module is a tool for deidentifying Protected Health Information (PHI) from data in a file path. It can be used with or without ant Spark NLP NER pipelines. It can apply deidentification and obfuscation on different columns at the same time.
It returns the deidentification & obfuscation results as a spark dataframe as well as a `csv` or `json file` saved locally.
The module also includes functionality for applying Structured Deidentification task to data from a file path. <br/>

Please visit [Clinical Deidentification Utility Module Notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.5.Clinical_Deidentification_Utility_Module.ipynb) for datailed usage and examples.

Firstly, we will load NER pipeline to identify protected entities. Pretrained NER models extracts:

- Name
- Profession
- Age
- Date
- Contact(Telephone numbers, FAX numbers, Email addresses)
- Location (Address, City, Postal code, Hospital Name, Employment information)
- Id (Social Security numbers, Medical record numbers, Internet protocol addresses)

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on n2c2 (de-identification and Heart Disease Risk Factors Challenge) datasets)
deid_ner = medical.NerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("deid_ner")

deid_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "deid_ner"]) \
    .setOutputCol("deid_ner_chunk")


deidPipeline = nlp.Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter])

deid_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
ner_deid_generic_augmented download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 . PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .
'''

In [0]:
result = deid_model.transform(spark.createDataFrame([[text]]).toDF("text"))

Let's check extracted sensitive entities by NER piepline

In [0]:
result.select(F.explode(F.arrays_zip(result.deid_ner_chunk.result, result.deid_ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------+---------+
chunk |ner_label|
+-----------------------------+---------+
2093-01-13 |DATE |
David Hale |NAME |
Hendrickson Ora |NAME |
#7194334 |ID |
01/13/93 |DATE |
Oliveira |NAME |
25 |AGE |
2079-11-09 |DATE |
Cocke County Baptist Hospital|LOCATION |
0295 Keats Street |LOCATION |
55-555-5555 |CONTACT |
+-----------------------------+---------+

Sometimes we need to leave some entities in the text, for example, if we want to analyze the frequency of the disease by the hospital. In this case, we may use the parameter **`setBlackList()`** to modify `ner_chunk` output. This parameter having using a list of entity types to deidentify as an input. So, if we want to exclude the location, we need to specify this tag in the list:

In [0]:
filtered_ner_converter = medical.NerConverterInternal()\
    .setInputCols(["sentence", "token", "deid_ner"])\
    .setOutputCol("deid_ner_chunk") \
    .setBlackList(['LOCATION'])

deidPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    deid_ner,
    filtered_ner_converter])


model_with_black_list = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

result_with_black_list = model_with_black_list.transform(spark.createDataFrame([[text]]).toDF("text"))

In [0]:
print("All Labels :")
result.select(F.explode(F.arrays_zip(result.deid_ner_chunk.result, result.deid_ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

print("BlackListed Labels: ")
result_with_black_list.select(F.explode(F.arrays_zip(result_with_black_list.deid_ner_chunk.result, 
                                                     result_with_black_list.deid_ner_chunk.metadata)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("chunk"),
                              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)
     

All Labels :
+-----------------------------+---------+
chunk |ner_label|
+-----------------------------+---------+
2093-01-13 |DATE |
David Hale |NAME |
Hendrickson Ora |NAME |
#7194334 |ID |
01/13/93 |DATE |
Oliveira |NAME |
25 |AGE |
2079-11-09 |DATE |
Cocke County Baptist Hospital|LOCATION |
0295 Keats Street |LOCATION |
55-555-5555 |CONTACT |
+-----------------------------+---------+

BlackListed Labels: 
+---------------+---------+
chunk |ner_label|
+---------------+---------+
2093-01-13 |DATE |
David Hale |NAME |
Hendrickson Ora|NAME |
#7194334 |ID |
01/13/93 |DATE |
Oliveira |NAME |
25 |AGE |
2079-11-09 |DATE |
55-555-5555 |CONTACT |
+---------------+---------+

### Masking

Now, we will replace this enitites with tags, this process is called **"masking"**. We will specifiy `setMode()` parameter as `setMode("mask")` in `DeIdentification()` in order to apply masking

In [0]:
deid_ner_converter = medical.NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")

deidentification = medical.DeIdentification() \
      .setInputCols(["sentence", "token", "deid_ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setReturnEntityMappings(True) #  return a new column to save the mappings between the mask/obfuscated entities and original entities.
      #.setMappingsColumn("MappingCol") # change the name of the column, 'aux' is default

deidPipeline = nlp.Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      deidentification])

deid_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
result = deid_model.transform(spark.createDataFrame([[text]]).toDF("text"))
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| deid_ner| deid_ner_chunk| deidentified| aux|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
\nRecord date : 2...|[{document, 0, 23...|[{document, 1, 45...|[{token, 1, 6, Re...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 15, 24, ...|[{document, 0, 36...|[{chunk, 14, 19, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
pd.set_option('display.max_colwidth', None)

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"),
              F.expr("cols['1']").alias("deidentified")).toPandas()

Out[14]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D ."
1,", Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 .",", Name : <NAME> , MR <ID> Date : <DATE> ."
2,"PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","PCP : <NAME> , <AGE> years-old , Record date : <DATE> ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .","<LOCATION> , <LOCATION> , Phone <CONTACT> ."


We have three modes to mask the entities in the Deidentification annotator. You can select the modes using the `.setMaskingPolicy()` parameter. The methods are the followings:

**“entity_labels”**: Mask with the entity type of that chunk. (default) <br/>
**“same_length_chars”**: Mask the deid entities with same length of asterix ( * ) with brackets ( [ , ] ) on both end. <br/>
**“fixed_length_chars”**: Mask the deid entities with a fixed length of asterix ( * ). The length is setting up using the `setFixedMaskLength()` method. <br/>
Let's try each of these and compare the results.

In [0]:
#deid model with "entity_labels"
deid_entity_labels= medical.DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"])\
    .setOutputCol("deid_entity_label")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("entity_labels")

#deid model with "same_length_chars"
deid_same_length= medical.DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"])\
    .setOutputCol("deid_same_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("same_length_chars")

#deid model with "fixed_length_chars"
deid_fixed_length= medical.DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"])\
    .setOutputCol("deid_fixed_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)


deidPipeline = nlp.Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      deid_entity_labels,
      deid_same_length,
      deid_fixed_length])

deid_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
policy_result = deid_model.transform(spark.createDataFrame([[text]]).toDF("text"))
policy_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| deid_ner| deid_ner_chunk| deid_entity_label| aux| deid_same_length| deid_fixed_length|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
\nRecord date : 2...|[{document, 0, 23...|[{document, 1, 45...|[{token, 1, 6, Re...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 15, 24, ...|[{document, 0, 36...|[{chunk, 14, 17, ...|[{document, 0, 44...|[{document, 0, 32...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:

policy_result.select(F.explode(F.arrays_zip(policy_result.sentence.result, 
                                            policy_result.deid_entity_label.result, 
                                            policy_result.deid_same_length.result, 
                                            policy_result.deid_fixed_length.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("sentence"),
                     F.expr("cols['1']").alias("deid_entity_label"),
                     F.expr("cols['2']").alias("deid_same_length"),
                     F.expr("cols['3']").alias("deid_fixed_length")).toPandas()

Out[17]:

,sentence,deid_entity_label,deid_same_length,deid_fixed_length
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D .","Record date : [********] , [********] , M.D .","Record date : **** , **** , M.D ."
1,", Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 .",", Name : <NAME> , MR <ID> Date : <DATE> .",", Name : [*************] , MR [******] Date : [******] .",", Name : **** , MR **** Date : **** ."
2,"PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","PCP : <NAME> , <AGE> years-old , Record date : <DATE> .","PCP : [******] , ** years-old , Record date : [********] .","PCP : **** , **** years-old , Record date : **** ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .","<LOCATION> , <LOCATION> , Phone <CONTACT> .","[***************************] , [***************] , Phone [*********] .","**** , **** , Phone **** ."


### Reidentification

We can use `ReIdentification` annotator to go back to the original sentence. Let's see how it works

In [0]:
reIdentification = medical.ReIdentification()\
    .setInputCols(["aux","deidentified"])\
    .setOutputCol("original")

In [0]:
reid_result = reIdentification.transform(result)
reid_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| deid_ner| deid_ner_chunk| deidentified| aux| original|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
\nRecord date : 2...|[{document, 0, 23...|[{document, 1, 45...|[{token, 1, 6, Re...|[{word_embeddings...|[{named_entity, 1...|[{chunk, 15, 24, ...|[{document, 0, 36...|[{chunk, 14, 19, ...|[{document, 1, 45...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
print(text)

reid_result.select('original.result').show(truncate=False)

Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 . PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[Record date : 2093-01-13 , David Hale , M.D ., , Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 ., PCP : Oliveira , 25 years-old , Record date : 2079-11-09 ., Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .]|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

### Enriching with Regex and Override NER




We can enrich our existing Deidentification model with both custom and embedded regex.

Firstly, let's build our deid pipeline again, but this time by filtering "`CONTACT`" entities from ner model.

Then, we will set `setReturnEntityMappings(True)`. In this way, we are going to see that we still be able to deidentify "`CONTACT`" as `PHONE` even though ner model doesn't detect "`CONTACT`" entity since the regex embedded in deid model will be taken account.

In [0]:
filtered_ner_converter = medical.NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")\
      .setBlackList(["CONTACT"])

deidentification = medical.DeIdentification() \
      .setInputCols(["sentence", "token", "deid_ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setRegexOverride(True)

deidPipeline = nlp.Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      filtered_ner_converter,
      deidentification])

model_default_rgx = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
result = model_default_rgx.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()
     

Out[22]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D ."
1,", Name : Hendrickson Ora , MR #7194334 Date : 01/13/93 .",", Name : <NAME> , MR <ID> Date : <DATE> ."
2,"PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","PCP : <NAME> , <AGE> years-old , Record date : <DATE> ."
3,"Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .","<LOCATION> , <LOCATION> , Phone <PHONE> ."


As you see above, we were able to deidentify `CONTACT` entity as `PHONE` which comes from the embedded regex even though ner model doesn't detect them.

Now, let's identify a custom regex and see how it works.

We will write a regex to deidentify "number" and save it as .txt file. Then, we will specify this file in `Deidentification()` through `setRegexPatternsDictionary()` parameter.

In [0]:
import os
import json
import string

In [0]:
rgx = '''NUMBER ([0-9]{2})
NUMBER (\d{7})''' 

with open("/dbfs/custom_regex.txt", "w") as f:
  f.write(rgx)

In [0]:
deid_ner_converter = medical.NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")

deidentification_rgx = medical.DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"])\
    .setOutputCol("deidentified")\
    .setMode("mask") \
    .setRegexPatternsDictionary("file:/dbfs/custom_regex.txt")\
    .setRegexOverride(True)

deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      deidentification_rgx])

model_rgx = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
text ='''Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street.'''
result = model_rgx.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()
     

Out[26]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : <DATE> , <NAME> , M.D ."
1,", Name : Hendrickson , Ora MR .",", Name : <NAME> MR ."
2,"# 7194334 Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 .","# <NUMBER> Date : <DATE> PCP : <NAME> , <NUMBER> years-old , Record date : <DATE> ."
3,Cocke County Baptist Hospital .,<LOCATION> .
4,0295 Keats Street.,<LOCATION>.


As you see above, we deidentified the "number" through our custom regex even though ner model doesn't detect the "number" entity.

### Obfuscation

In the obfuscation mode **DeIdentificationModel** will replace sensetive entities with random values of the same type.
We can specify `.setMode()` parameter as `.setMode("obfuscate")` to use this feature.

Let's create a piepline and see how it works

In [0]:
deid_ner_converter = medical.NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")

obfuscation = medical.DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\

deidPipeline = nlp.Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      obfuscation])

obfuscation_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:
text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR #7194334 Date : 01/13/93 . Patient : Oliveira, 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''

result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()
     

Out[28]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-02-05 , Violet Baldy , M.D ."
1,", Name : Hendrickson , Ora MR #7194334 Date : 01/13/93 .",", Name : Stan Head #3789610 Date : 02/05/93 ."
2,"Patient : Oliveira, 25 years-old , Record date : 2079-11-09 .","Patient : Raye Sorrow, 29 years-old , Record date : 2079-12-02 ."
3,Cocke County Baptist Hospital .,615 Clinic Drive .
4,0295 Keats Street,2115 Parkview Drive


As you see, we succesfully obfuscated the entities.


In addition, we can define a reference file for obfuscation. In this way, deid model uses the fake entities that we defined in this file.

If we want to use reference file, we should give the path of txt file that includes our custom entities to the `setObfuscateRefFile()` parameter. Then, we should set `setObfuscateRefSource("file")` parameter.

However, we can set `setObfuscateRefSource` parameter as **faker** or **both** as well.

The faker module allows  the user to use a set of fake entities that are in the memory of the spark-nlp-internal. You can set up this module for using the following property `setObfuscateRefSource('faker')`.

If we select the `setObfuscateRefSource('both')` then we choose randomly the entities using the faker and the fake entities from the obfuscateRefFile.





Now, let's define a reference file and use it in our deidentification model.%md

In [0]:
obs_lines = """Marvin MARSHALL#NAME
Hubert GROGAN#NAME
ALTHEA COLBURN#NAME
Kalil AMIN#NAME
Inci FOUNTAIN#NAME
Ekaterina Rosa#NAME
Rudiger Chao#NAME
COLLETTE KOHLER#NAME
Mufi HIGGS#NAME"""


with open ('/dbfs/obfuscation.txt', 'w') as f:
  f.write(obs_lines)

In [0]:
deid_ner_converter = medical.NerConverterInternal()\
      .setInputCols(["sentence", "token", "deid_ner"])\
      .setOutputCol("deid_ner_chunk")

obfuscation = medical.DeIdentification()\
    .setInputCols(["sentence", "token", "deid_ner_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('/dbfs/obfuscation.txt')\
    .setObfuscateRefSource("file") #both or faker

deidPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      deid_ner_converter,
      obfuscation])


obfuscation_model = deidPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [0]:

text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR # 7194334 Date : 01/13/93 . Patient : Oliveira, 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''

result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"),
              F.expr("cols['1']").alias("deidentified")).toPandas()

Out[31]:

,sentence,deidentified
0,"Record date : 2093-01-13 , David Hale , M.D .","Record date : 2093-01-26 , Marvin MARSHALL , M.D ."
1,", Name : Hendrickson , Ora MR # 7194334 Date : 01/13/93 .",", Name : Rudiger Chao MR # 6126748 Date : 01/26/93 ."
2,"Patient : Oliveira, 25 years-old , Record date : 2079-11-09 .","Patient : Rudiger Chao, <AGE> years-old , Record date : 2079-11-22 ."
3,Cocke County Baptist Hospital .,<LOCATION> .
4,0295 Keats Street,<LOCATION>


**`.setObfuscateDate(True)` parameter obfuscated the `DATE` entities and the `NAME`s were replaced with the custom list.**

## Shifting Days

In the example above, we replaced date entities with another date randomly. Instead of that, we can shift the date according to any column.

In [0]:
#sample data for day shifting
data = pd.DataFrame(
    {'patientID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was discharged on 10/02/2022',
               'Mark White was discharged on 02/28/2020',
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ]
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate = False)

+---------+----------------------------------------+
patientID|text |
+---------+----------------------------------------+
A001 |Chris Brown was discharged on 10/02/2022|
A001 |Mark White was discharged on 02/28/2020 |
A002 |John was discharged on 03/15/2022 |
A002 |John Moore was discharged on 12/31/2022 |
+---------+----------------------------------------+

### Shifting days according to the ID column

We use the `DocumentHashCoder()` annotator to determine shifting days. This annotator gets the hash of the specified column and creates a new document column containing day shift information. And then, the `DeIdentification` annotator deidentifies this new doc. We should set the seed parameter to hash consistently.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = medical.DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setPatientIdColumn("patientID")\
    .setRangeDays(100)\
    .setNewDateShift("shift_days")\
    .setSeed(100)

tokenizer = nlp.Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document2", "token"])\
    .setOutputCol("word_embeddings")

clinical_ner = medical.NerModel\
    .pretrained("ner_deid_subentity_augmented", "en", "clinical/models")\
    .setInputCols(["document2","token", "word_embeddings"])\
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal()\
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk")

de_identification = medical.DeIdentification() \
    .setInputCols(["ner_chunk", "token", "document2"]) \
    .setOutputCol("deid_text") \
    .setMode("obfuscate") \
    .setObfuscateDate(True) \
    .setDateTag("DATE") \
    .setLanguage("en") \
    .setObfuscateRefSource('faker') \
    .setUseShifDays(True)\
    .setRegion('us')


pipeline = nlp.Pipeline().setStages([
    documentAssembler,
    documentHasher,
    tokenizer,
    embeddings,
    clinical_ner,
    ner_converter,
    de_identification

])

empty_data = spark.createDataFrame([["", ""]]).toDF("text", "patientID")

pipeline_model = pipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_subentity_augmented download started this may take some time.
[ | ][ / ][OK!]

In [0]:
output = pipeline_model.transform(my_input_df)

output.select('patientID','text', "shift_days", 'deid_text.result').show(truncate = False)

+---------+----------------------------------------+----------+-----------------------------------------------+
patientID|text |shift_days|result |
+---------+----------------------------------------+----------+-----------------------------------------------+
A001 |Chris Brown was discharged on 10/02/2022|-5 |[Mason Jim was discharged on 09/27/2022] |
A001 |Mark White was discharged on 02/28/2020 |-5 |[Celestia Khat was discharged on 02/23/2020] |
A002 |John was discharged on 03/15/2022 |29 |[Willette Cluster was discharged on 04/13/2022]|
A002 |John Moore was discharged on 12/31/2022 |29 |[Sofie Hartigan was discharged on 01/29/2023] |
+---------+----------------------------------------+----------+-----------------------------------------------+

### Shifting days according to specified values

Instead of shifting days according to ID column, we can specify shifting values with another column.

```python
documentHasher = DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\
```

In [0]:
data = pd.DataFrame(
    {'patientID' : ['A001', 'A002', 'A003', 'A004'],
     'text' : ['Chris Brown was discharged on 10/02/2022',
               'Mark White was discharged on 03/01/2020',
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ],
     'dateshift' : ['10', '-2', '30', '-8']
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate=False)

+---------+----------------------------------------+---------+
patientID|text |dateshift|
+---------+----------------------------------------+---------+
A001 |Chris Brown was discharged on 10/02/2022|10 |
A002 |Mark White was discharged on 03/01/2020 |-2 |
A003 |John was discharged on 03/15/2022 |30 |
A004 |John Moore was discharged on 12/31/2022 |-8 |
+---------+----------------------------------------+---------+

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = medical.DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\

tokenizer = nlp.Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document2", "token"])\
    .setOutputCol("word_embeddings")

clinical_ner = medical.NerModel\
    .pretrained("ner_deid_subentity_augmented", "en", "clinical/models")\
    .setInputCols(["document2","token", "word_embeddings"])\
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal()\
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk")

de_identification = medical.DeIdentification() \
    .setInputCols(["ner_chunk", "token", "document2"]) \
    .setOutputCol("deid_text") \
    .setMode("obfuscate") \
    .setObfuscateDate(True) \
    .setDateTag("DATE") \
    .setLanguage("en") \
    .setObfuscateRefSource('faker') \
    .setUseShifDays(True)\
    .setRegion('us')

pipeline_col = nlp.Pipeline().setStages([
    documentAssembler,
    documentHasher,
    tokenizer,
    embeddings,
    clinical_ner,
    ner_converter,
    de_identification

])

empty_data = spark.createDataFrame([["", "", ""]]).toDF("patientID","text", "dateshift")

pipeline_col_model = pipeline_col.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_subentity_augmented download started this may take some time.
[ | ][OK!]

In [0]:
output = pipeline_col_model.transform(my_input_df)

output.select('text', 'dateshift', 'deid_text.result').show(truncate = False)

+----------------------------------------+---------+-----------------------------------------------+
text |dateshift|result |
+----------------------------------------+---------+-----------------------------------------------+
Chris Brown was discharged on 10/02/2022|10 |[Joline Salt was discharged on 10/12/2022] |
Mark White was discharged on 03/01/2020 |-2 |[Deatra Robinson was discharged on 02/28/2020] |
John was discharged on 03/15/2022 |30 |[Henrietta Hoover was discharged on 04/14/2022]|
John Moore was discharged on 12/31/2022 |-8 |[Rolanda Lundborg was discharged on 12/23/2022]|
+----------------------------------------+---------+-----------------------------------------------+

Here, dates were shifted according to `dateshift` column

## Age Groups Obfuscation

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on n2c2 (de-identification and Heart Disease Risk Factors Challenge) datasets)
deid_ner = medical.NerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("deid_ner")

deid_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "deid_ner"]) \
    .setOutputCol("deid_ner_chunk")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_generic_augmented download started this may take some time.
[ | ][OK!]

In [0]:
ner_converter = medical.NerConverterInternal()\
    .setInputCols(["sentence", "token", "deid_ner"])\
    .setOutputCol("age_chunk")\
    .setWhiteList(["AGE"])

obfuscation = medical.DeIdentification()\
    .setInputCols(["sentence", "token", "age_chunk"])\
    .setOutputCol("obfuscation")\
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefSource("faker")\
    .setAgeRanges([1, 4, 12, 20, 40, 60, 80])

nlpPipeline = nlp.Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      ner_converter,
      obfuscation
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_agerange = nlpPipeline.fit(empty_data)

In [0]:
# Infant = 0-1 year.
# Toddler = 2-4 yrs.
# Child = 5-12 yrs.
# Teen = 13-19 yrs.
# Adult = 20-39 yrs.
# Middle Age Adult = 40-59 yrs.
# Senior Adult = 60+

dates = [
'1 year old baby',
'4 year old kids',
'A 15 year old female with',
'Record date: 2093-01-13, Age: 25',
'Patient is 45 years-old',
'He is 65 years-old male'
]
from pyspark.sql.types import StringType
df_dates = spark.createDataFrame(dates,StringType()).toDF('text')


result = model_agerange.transform(df_dates)

result_df = result.select("text",F.explode(F.arrays_zip(result.age_chunk.result,
                                                        result.obfuscation.result)).alias("cols")) \
                  .select("text",F.expr("cols['0']").alias("age_chunk"),
                                 F.expr("cols['1']").alias("obfuscation"))

result_df.show(truncate=False)

+--------------------------------+---------+--------------------------------+
text |age_chunk|obfuscation |
+--------------------------------+---------+--------------------------------+
1 year old baby |1 |3 year old baby |
4 year old kids |4 |8 year old kids |
A 15 year old female with |15 |A 12 year old female with |
Record date: 2093-01-13, Age: 25|25 |Record date: 2093-01-18, Age: 34|
Patient is 45 years-old |45 |Patient is 46 years-old |
He is 65 years-old male |65 |He is 77 years-old male |
+--------------------------------+---------+--------------------------------+

## Multi-Mode Deidentification


We have multi-mode functionality in the `DeIdentification()`.
By providing a json file to the `setSelectiveObfuscationModes("a JSON path")` parameter, we are able to use multi-mode in de-identification. <br/>

The entities which have not been given in dictionary will be deidentified according to the `setMode()` parameter.

In [0]:
#json to choose deid modes
sample_json= {
	"obfuscate": ["PHONE"] ,
	"mask_entity_labels": ["ID"],
	"skip": ["DATE"],
	"mask_same_length_chars":["NAME"],
	"mask_fixed_length_chars":["zip", "location"]
}

import json
with open('/dbfs/sample_multi_mode.json', 'w', encoding='utf-8') as f:
    json.dump(sample_json, f, ensure_ascii=False, indent=4)

In [0]:
clinical_ner = medical.NerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setBlackList(["CONTACT"])

deid_doc = medical.DeIdentification() \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setSelectiveObfuscationModesPath("dbfs:/sample_multi_mode.json")\
      .setSameLengthFormattedEntities(["PHONE"])\
      .setFixedMaskLength(2)

deidPipeline = nlp.Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      deid_doc])

empty_data = spark.createDataFrame([[""]]).toDF("text")

text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 . PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .
'''

model_deid = deidPipeline.fit(empty_data)
result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))


ner_deid_generic_augmented download started this may take some time.
[ | ][OK!]

In [0]:
result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"),
              F.expr("cols['1']").alias("deidentified")).show(truncate=False)

+-----------------------------------------------------------------------+-------------------------------------------------------------+
sentence |deidentified |
+-----------------------------------------------------------------------+-------------------------------------------------------------+
Record date : 2093-01-13 , David Hale , M.D . |Record date : 2093-01-13 , [********] , M.D . |
, Name : Hendrickson Ora , MR # 7194334 Date : 01/13/93 . |, Name : [*************] , MR # <ID> Date : 01/13/93 . |
PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . |PCP : [******] , <AGE> years-old , Record date : 2079-11-09 .|
Cocke County Baptist Hospital , 0295 Keats Street , Phone 55-555-5555 .|** , ** , Phone 41-287-4115 . |
+-----------------------------------------------------------------------+-------------------------------------------------------------+

## Structured Deidentification

We can use deidentification not only text but also structured data as well.

Let's see how it works

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/hipaa-table-001.txt

df = spark.read.format("csv") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("dbfs:/hipaa-table-001.txt")

df = df.withColumnRenamed("PATIENT","NAME")
df.show(truncate=False)

+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
NAME |DOB |AGE|ADDRESS |ZIPCODE|TEL |SBP|DBP|
+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
Cecilia Chapman|04/02/1935|83 |711-2880 Nulla St. Mankato Mississippi |69200 |(257) 563-7401|101|42 |
Iris Watson |03/10/2009|9 |P.O. Box 283 8562 Fusce Rd. Frederick Nebraska |20620 |(372) 587-2335|159|122|
Bryar Pitts |11/01/1921|98 |5543 Aliquet St. Fort Dodge GA |20783 |(717) 450-4729|149|52 |
Theodore Lowe |13/02/2002|16 |Ap #867-859 Sit Rd. Azusa New York |39531 |(793) 151-6230|134|115|
Calista Wise |20/08/1942|76 |7292 Dictum Av. San Antonio MI |47096 |(492) 709-6392|139|78 |
Kyla Olsen |12/05/1973|45 |Ap #651-8679 Sodales Av. Tamuning PA |10855 |(654) 393-5734|120|112|
Forrest Ray |11/01/1991|27 |191-103 Integer Rd. Corona New Mexico |8219 |(404) 960-3807|143|126|
Hiroko Potter |18/11/1937|81 |P.O. Box 887 2508 Dolor. Av. Muskegon KY |12482 |(314) 244-6306|147|75 |
Celeste Slater |12/05/1980|38 |606-3727 Ullamcorper. Street Roseville NH |11523 |(786) 713-8616|147|123|
Nyssa Vazquez |24/09/1956|62 |511-5762 At Rd. Chelsea MI |67708 |(947) 278-5929|129|50 |
Lawrence Moreno|26/12/1906|112|935-9940 Tortor. Street Santa Rosa MN |98804 |(684) 579-1879|133|102|
Ina Moran |26/10/1983|35 |P.O. Box 929 4189 Nunc Road Lebanon KY |69409 |(389) 737-2852|101|67 |
Aaron Hawkins |26/09/2009|9 |5587 Nunc. Avenue Erie Rhode Island |24975 |(660) 663-4518|87 |81 |
Hedy Greene |03/10/1920|98 |Ap #696-3279 Viverra. Avenue Latrobe DE |38100 |(608) 265-2215|128|123|
Melvin Porter |14/08/1911|107|P.O. Box 132 1599 Curabitur Rd. Bandera South Dakota|45149 |(959) 119-8364|83 |43 |
Keefe Sellers |16/05/1937|81 |347-7666 Iaculis St. Woodruff SC |49854 |(468) 353-2641|148|109|
Joan Romero |08/12/2004|14 |666-4366 Lacinia Avenue Idaho Falls Ohio |19253 |(248) 675-4007|75 |53 |
Davis Patrick |09/01/1956|63 |P.O. Box 147 2546 Sociosqu Rd. Bethlehem Utah |2913 |(939) 353-1107|142|62 |
Leilani Boyer |18/10/1934|84 |557-6308 Lacinia Road San Bernardino ND |9289 |(570) 873-7090|137|48 |
Colby Bernard |02/10/1905|113|Ap #285-7193 Ullamcorper Avenue Amesbury HI |93373 |(302) 259-2375|84 |41 |
+---------------+----------+---+----------------------------------------------------+-------+--------------+---+---+
only showing top 20 rows

In [0]:
from sparknlp_jsl.structured_deidentification import StructuredDeidentification

In [0]:
obfuscator = medical.StructuredDeidentification(spark,{"NAME":"PATIENT","AGE":"AGE"}, obfuscateRefSource = "faker")
obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.show(truncate=False)

+-------------------+----------+-----+----------------------------------------------------+-------+--------------+---+---+
NAME |DOB |AGE |ADDRESS |ZIPCODE|TEL |SBP|DBP|
+-------------------+----------+-----+----------------------------------------------------+-------+--------------+---+---+
[Mel Almond] |04/02/1935|[94] |711-2880 Nulla St. Mankato Mississippi |69200 |(257) 563-7401|101|42 |
[Sheilah Mins] |03/10/2009|[10] |P.O. Box 283 8562 Fusce Rd. Frederick Nebraska |20620 |(372) 587-2335|159|122|
[Rosilyn Mings] |11/01/1921|[87] |5543 Aliquet St. Fort Dodge GA |20783 |(717) 450-4729|149|52 |
[Barbaraann Boys] |13/02/2002|[13] |Ap #867-859 Sit Rd. Azusa New York |39531 |(793) 151-6230|134|115|
[Donell Beers] |20/08/1942|[60] |7292 Dictum Av. San Antonio MI |47096 |(492) 709-6392|139|78 |
[Deniece Portela] |12/05/1973|[49] |Ap #651-8679 Sodales Av. Tamuning PA |10855 |(654) 393-5734|120|112|
[Frutoso Schatz] |11/01/1991|[28] |191-103 Integer Rd. Corona New Mexico |8219 |(404) 960-3807|143|126|
[Malvin Johns] |18/11/1937|[80] |P.O. Box 887 2508 Dolor. Av. Muskegon KY |12482 |(314) 244-6306|147|75 |
[Odis Hollingshead]|12/05/1980|[30] |606-3727 Ullamcorper. Street Roseville NH |11523 |(786) 713-8616|147|123|
[Allyne Gee] |24/09/1956|[75] |511-5762 At Rd. Chelsea MI |67708 |(947) 278-5929|129|50 |
[Alonna Buckler] |26/12/1906|[118]|935-9940 Tortor. Street Santa Rosa MN |98804 |(684) 579-1879|133|102|
[Kristopher Glee] |26/10/1983|[23] |P.O. Box 929 4189 Nunc Road Lebanon KY |69409 |(389) 737-2852|101|67 |
[Carlena Sax] |26/09/2009|[10] |5587 Nunc. Avenue Erie Rhode Island |24975 |(660) 663-4518|87 |81 |
[Laurey Arrow] |03/10/1920|[87] |Ap #696-3279 Viverra. Avenue Latrobe DE |38100 |(608) 265-2215|128|123|
[Darliss Ridgel] |14/08/1911|[111]|P.O. Box 132 1599 Curabitur Rd. Bandera South Dakota|45149 |(959) 119-8364|83 |43 |
[Hulan Saas] |16/05/1937|[80] |347-7666 Iaculis St. Woodruff SC |49854 |(468) 353-2641|148|109|
[Jannette Fogo] |08/12/2004|[15] |666-4366 Lacinia Avenue Idaho Falls Ohio |19253 |(248) 675-4007|75 |53 |
[Phyllis Ginger] |09/01/1956|[76] |P.O. Box 147 2546 Sociosqu Rd. Bethlehem Utah |2913 |(939) 353-1107|142|62 |
[Gatha Mayer] |18/10/1934|[92] |557-6308 Lacinia Road San Bernardino ND |9289 |(570) 873-7090|137|48 |
[Louretta Parma] |02/10/1905|[118]|Ap #285-7193 Ullamcorper Avenue Amesbury HI |93373 |(302) 259-2375|84 |41 |
+-------------------+----------+-----+----------------------------------------------------+-------+--------------+---+---+
only showing top 20 rows

We can define reference file for obfuscation as well like we did before. All we need to do is defining `obfuscateRefFile` and `onfuscateRefSource` parameters.

In [0]:
obfuscator_unique_ref_test = '''Will Perry#PATIENT
John Smith#PATIENT
Marvin MARSHALL#PATIENT
Hubert GROGAN#PATIENT
ALTHEA COLBURN#PATIENT
Kalil AMIN#PATIENT
Inci FOUNTAIN#PATIENT
Jackson WILLE#PATIENT
Jack SANTOS#PATIENT
Mahmood ALBURN#PATIENT
Marnie MELINGTON#PATIENT
Aysha GHAZI#PATIENT
Maryland CODER#PATIENT
Darene GEORGIOUS#PATIENT
Shelly WELLBECK#PATIENT
Min Kun JAE#PATIENT
Thomson THOMAS#PATIENT
Christian SUDDINBURG#PATIENT
Aberdeen#CITY
Louisburg St#STREET
France#LOC
Nick Riviera#DOCTOR
5552312#PHONE
St James Hospital#HOSPITAL
Calle del Libertador#ADDRESS
111#ID
Will#DOCTOR
20#AGE
30#AGE
40#AGE
50#AGE
60#AGE
'''

with open('/dbfs/obfuscator_unique_ref_test.txt', 'w') as f:
  f.write(obfuscator_unique_ref_test)

In [0]:
# obfuscateRefSource = "file"

obfuscator = StructuredDeidentification(spark,{"NAME":"PATIENT","AGE":"AGE"}, 
                                        obfuscateRefFile = "/dbfs/obfuscator_unique_ref_test.txt",
                                        obfuscateRefSource = "file",
                                        columnsSeed={"NAME": 23, "AGE": 23})
obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.select("NAME","AGE").show(truncate=False)

+------------------+----+
NAME |AGE |
+------------------+----+
[Inci FOUNTAIN] |[60]|
[Jack SANTOS] |[30]|
[Darene GEORGIOUS]|[30]|
[Shelly WELLBECK] |[40]|
[Hubert GROGAN] |[40]|
[Kalil AMIN] |[40]|
[ALTHEA COLBURN] |[60]|
[Thomson THOMAS] |[60]|
[Jack SANTOS] |[60]|
[Will Perry] |[20]|
[Jackson WILLE] |[60]|
[Shelly WELLBECK] |[40]|
[Kalil AMIN] |[30]|
[Marnie MELINGTON]|[30]|
[Min Kun JAE] |[30]|
[Marvin MARSHALL] |[60]|
[Marvin MARSHALL] |[50]|
[Min Kun JAE] |[30]|
[Maryland CODER] |[20]|
[Marnie MELINGTON]|[20]|
+------------------+----+
only showing top 20 rows

We can **shift n days** in the structured deidentification through "days" parameter when the column is a Date.

In [0]:
df = spark.createDataFrame([
            ["Juan García", "13/02/1977", "711 Nulla St.", "140", "673 431234"],
            ["Will Smith", "23/02/1977", "1 Green Avenue.", "140", "+23 (673) 431234"],
            ["Pedro Ximénez", "11/04/1900", "Calle del Libertador, 7", "100", "912 345623"]
        ]).toDF("NAME", "DOB", "ADDRESS", "SBP", "TEL")
df.show(truncate=False)

+-------------+----------+-----------------------+---+----------------+
NAME |DOB |ADDRESS |SBP|TEL |
+-------------+----------+-----------------------+---+----------------+
Juan García |13/02/1977|711 Nulla St. |140|673 431234 |
Will Smith |23/02/1977|1 Green Avenue. |140|+23 (673) 431234|
Pedro Ximénez|11/04/1900|Calle del Libertador, 7|100|912 345623 |
+-------------+----------+-----------------------+---+----------------+

In [0]:
obfuscator = medical.StructuredDeidentification(spark=spark,
                                        columns={"NAME": "ID", "DOB": "DATE"},
                                        columnsSeed={"NAME": 23, "DOB": 23},
                                        obfuscateRefSource="faker",
                                        days=5
                                         )

In [0]:
result = obfuscator.obfuscateColumns(df)
result.show(truncate=False)

+---------------+------------+-----------------------+---+----------------+
NAME |DOB |ADDRESS |SBP|TEL |
+---------------+------------+-----------------------+---+----------------+
[GNBK RNXYIW] |[18/02/1977]|711 Nulla St. |140|673 431234 |
[TVXP KNZIB] |[28/02/1977]|1 Green Avenue. |140|+23 (673) 431234|
[ZGSWM RSUWRBH]|[16/04/1900]|Calle del Libertador, 7|100|912 345623 |
+---------------+------------+-----------------------+---+----------------+

## Pretrained Deidentification Pipeline

`clinical_deidentification` pipeline can be used to deidentify PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `AGE`, `CONTACT`, `DATE`, `ID`, `LOCATION`, `NAME`, `PROFESSION`, `CITY`, `COUNTRY`, `DOCTOR`, `HOSPITAL`, `IDNUM`, `MEDICALRECORD`, `ORGANIZATION`, `PATIENT`, `PHONE`, `PROFESSION`, `STREET`, `USERNAME`, `ZIP`, `ACCOUNT`, `LICENSE`, `VIN`, `SSN`, `DLN`, `PLATE`, `IPADDR` entities. There is also a slim version of deidentificaiton pipeline, `clinical_deidentification_slim`

Let's see how it works

In [0]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = nlp.PretrainedPipeline("clinical_deidentification", "en", "clinical/models")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
deid_pipeline.model.stages

Out[53]: [DocumentAssembler_8a57d822a7ce,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_abecab99e327,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_e8178a1262cc,
 NerConverter_4a729464bde5,
 MedicalNerModel_cb1ebd97c79c,
 NerConverter_fac59c55ab65,
 MERGE_d096b210dfcb,
 CONTEXTUAL-PARSER_3b72e11a3d0e,
 CONTEXTUAL-PARSER_da05c820c3e1,
 CONTEXTUAL-PARSER_21057aa997c9,
 CONTEXTUAL-PARSER_b30c682f3e88,
 CONTEXTUAL-PARSER_2e6558d597f5,
 CONTEXTUAL-PARSER_5ea2fa3cd839,
 ENTITY_EXTRACTOR_962b919bc1d5,
 CONTEXTUAL-PARSER_ceb994b5a1ec,
 REGEX_MATCHER_532487d749a2,
 CONTEXTUAL-PARSER_97ed8b42abbd,
 CONTEXTUAL-PARSER_ae9a6fa57b01,
 CONTEXTUAL-PARSER_7d04739023ad,
 CONTEXTUAL-PARSER_d9a318ea47aa,
 CONTEXTUAL-PARSER_b02af9559ee8,
 CONTEXTUAL-PARSER_d3511fb176c9,
 MERGE_1364755ae450,
 MERGE_b7ed7a44f0fc,
 DE-IDENTIFICATION_d30f0f188371,
 DE-IDENTIFICATION_aa8788f98145,
 DE-IDENTIFICATION_bcab2e588dd7,
 DE-IDENTIFICATION_9259fb14bcc5,
 Finisher_4bdc3ed6a7ca]

In [0]:
text= """Name : Hendrickson, Ora, Record date: 2093-01-13, Age: 25, # 719435. Dr. John Green, ID: 1231511863, IP 203.120.223.13. He is a 60-year-old male was admitted to the Day Hospital for cystectomy on 01/13/93. Patient's VIN : 1HGBH41JXMN109286, SSN #333-44-6666, Driver's license no:A334455B. Phone (302) 786-5227, 0295 Keats Street, San Francisco."""

In [0]:
deid_res= deid_pipeline.annotate(text)

In [0]:
deid_res.keys()

Out[56]: dict_keys(['masked', 'obfuscated', 'ner_chunk', 'masked_fixed_length_chars', 'sentence', 'masked_with_chars'])

In [0]:
pd.set_option("display.max_colwidth", 100)

df= pd.DataFrame(list(zip(deid_res["sentence"],
                          deid_res["masked"],
                          deid_res["masked_with_chars"],
                          deid_res["masked_fixed_length_chars"],
                          deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

Out[57]:

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"Name : Hendrickson, Ora, Record date: 2093-01-13, Age: 25, # 719435.","Name : <PATIENT>, Record date: <DATE>, Age: <AGE>, # <ZIP>.","Name : [**************], Record date: [********], Age: **, # [****].","Name : ****, Record date: ****, Age: ****, # ****.","Name : Carlyon Shadow, Record date: 2093-01-26, Age: 34, # 165304."
1,"Dr. John Green, ID: 1231511863, IP 203.120.223.13.","Dr. <DOCTOR>, ID<IDNUM>, IP <IPADDR>.","Dr. [********], ID[**********], IP [************].","Dr. ****, ID****, IP ****.","Dr. Annamarie Dawley, ID: 2862761644, IP 333.333.333.333."
2,He is a 60-year-old male was admitted to the Day Hospital for cystectomy on 01/13/93.,He is a <AGE>-year-old male was admitted to the <LOCATION> for cystectomy on <DATE>.,He is a **-year-old male was admitted to the [**********] for cystectomy on [******].,He is a ****-year-old male was admitted to the **** for cystectomy on ****.,He is a 64-year-old male was admitted to the 6200 Sw 73Rd St for cystectomy on 01/26/93.
3,"Patient's VIN : 1HGBH41JXMN109286, SSN #333-44-6666, Driver's license no:A334455B.","Patient's VIN : <VIN>, SSN <SSN>, Driver's license <DLN>.","Patient's VIN : [***************], SSN [**********], Driver's license [*********].","Patient's VIN : ****, SSN ****, Driver's license ****.","Patient's VIN : 8OVEZ91TRFO441690, SSN #883-76-2886, Driver's license NZ:R099060K."
4,"Phone (302) 786-5227, 0295 Keats Street, San Francisco.","Phone <PHONE>, <STREET>, <CITY>.","Phone [************], [***************], [***********].","Phone ****, ****, ****.","Phone (205) 342-3083, 1453 E Bert Kouns Industrial Loop, Paamiut."


# 5.Clinical Relation Extraction

### Pretrained Relation Extraction Models

Extracts and classifies instances of relations between named entities. For this, relation pairs need to be defined with `setRelationPairs`, to specify between which entities the extraction should be done.

We have also trained as end-to-end bert models using **BioBERT** and ported in to the Spark NLP ecosystem (**ReDL**). They offer **SOTA performance** on most benchmark tasks and outperform our existing Relation Extraction Models.


*  **Relation Extraction Models**

|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [generic_re](https://nlp.johnsnowlabs.com/2022/12/20/generic_re.html)  | 2| [posology_re](https://nlp.johnsnowlabs.com/2020/09/01/posology_re.html)  | 3| [re_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/16/re_ade_biobert_en.html)  |
| 4| [re_ade_clinical](https://nlp.johnsnowlabs.com/2021/07/12/re_ade_clinical_en.html)  | 5| [re_ade_conversational](https://nlp.johnsnowlabs.com/2022/07/27/re_ade_conversational_en_3_0.html)  | 6| [re_bodypart_directions](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_directions_en.html)  |
| 7| [re_bodypart_problem](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_problem_en.html)  | 8| [re_bodypart_proceduretest](https://nlp.johnsnowlabs.com/2021/01/18/re_bodypart_proceduretest_en.html)  | 9| [re_clinical](https://nlp.johnsnowlabs.com/2020/09/24/re_clinical_en.html)  |
| 10| [re_date_clinical](https://nlp.johnsnowlabs.com/2021/01/18/re_date_clinical_en.html)  | 11| [re_drug_drug_interaction_clinical](https://nlp.johnsnowlabs.com/2020/09/03/re_drug_drug_interaction_clinical_en.html)  | 12| [re_drugprot_clinical](https://nlp.johnsnowlabs.com/2022/01/05/re_drugprot_clinical_en.html)  |
| 13| [re_human_phenotype_gene_clinical](https://nlp.johnsnowlabs.com/2020/09/30/re_human_phenotype_gene_clinical_en.html)  | 14| [re_oncology_biomarker_result_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_biomarker_result_wip_en.html)  | 15| [re_oncology_granular_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_granular_wip_en.html)  |
| 16| [re_oncology_location_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_location_wip_en.html)  | 17| [re_oncology_size_wip](https://nlp.johnsnowlabs.com/2022/09/26/re_oncology_size_wip_en.html)  | 18| [re_oncology_temporal_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_temporal_wip_en.html)  |
| 19| [re_oncology_test_result_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_test_result_wip_en.html)  | 20| [re_oncology_wip](https://nlp.johnsnowlabs.com/2022/09/27/re_oncology_wip_en.html)  | 21| [re_temporal_events_clinical](https://nlp.johnsnowlabs.com/2020/09/28/re_temporal_events_clinical_en.html)  |
| 22| [re_temporal_events_enriched_clinical](https://nlp.johnsnowlabs.com/2020/09/28/re_temporal_events_enriched_clinical_en.html)  | 23| [re_test_problem_finding](https://nlp.johnsnowlabs.com/2021/04/19/re_test_problem_finding_en.html)  | 24| [re_test_result_date](https://nlp.johnsnowlabs.com/2021/02/24/re_test_result_date_en.html)  |
| 25| [re_zeroshot_biobert](https://nlp.johnsnowlabs.com/2022/04/05/re_zeroshot_biobert_en_3_0.html)  | 26| [redl_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/12/redl_ade_biobert_en.html)  | 27| [redl_bodypart_direction_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_bodypart_direction_biobert_en.html)  |
| 28| [redl_bodypart_problem_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_bodypart_problem_biobert_en.html)  | 29| [redl_bodypart_procedure_test_biobert](https://nlp.johnsnowlabs.com/2021/09/10/redl_bodypart_procedure_test_biobert_en.html)  | 30| [redl_chemprot_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_chemprot_biobert_en.html)  |
| 31| [redl_clinical_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_clinical_biobert_en.html)  | 32| [redl_date_clinical_biobert](https://nlp.johnsnowlabs.com/2021/06/01/redl_date_clinical_biobert_en.html)  | 33| [redl_drug_drug_interaction_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_drug_drug_interaction_biobert_en.html)  |
| 34| [redl_drugprot_biobert](https://nlp.johnsnowlabs.com/2023/01/14/redl_drugprot_biobert_en.html)  | 35| [redl_human_phenotype_gene_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_human_phenotype_gene_biobert_en.html)  | 36| [redl_nihss_biobert](https://nlp.johnsnowlabs.com/2021/11/16/redl_nihss_biobert_en.html)  |
| 37| [redl_oncology_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_biobert_wip_en.html)  | 38| [redl_oncology_biomarker_result_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_biomarker_result_biobert_wip_en.html)  | 39| [redl_oncology_granular_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_granular_biobert_wip_en.html)  |
| 40| [redl_oncology_location_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_location_biobert_wip_en.html)  | 41| [redl_oncology_size_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/28/redl_oncology_size_biobert_wip_en.html)  | 42| [redl_oncology_temporal_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_temporal_biobert_wip_en.html)  |
| 43| [redl_oncology_test_result_biobert_wip](https://nlp.johnsnowlabs.com/2022/09/29/redl_oncology_test_result_biobert_wip_en.html)  | 44| [redl_temporal_events_biobert](https://nlp.johnsnowlabs.com/2021/07/24/redl_temporal_events_biobert_en.html)  | 45| []()|


*  **Relation Extraction Pipelines**

|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [re_bodypart_directions_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_bodypart_directions_pipeline_en_3_0.html)  | 4| [re_human_phenotype_gene_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_human_phenotype_gene_clinical_pipeline_en_3_0.html)  | 7| [re_test_problem_finding_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_test_problem_finding_pipeline_en_3_0.html)  |
| 2| [re_bodypart_proceduretest_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_bodypart_proceduretest_pipeline_en_3_0.html)  | 5| [re_temporal_events_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_temporal_events_clinical_pipeline_en_3_0.html)  | 8| [re_test_result_date_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_test_result_date_pipeline_en_3_0.html)  |
| 3| [re_date_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_date_clinical_pipeline_en_3_0.html)  | 6| [re_temporal_events_enriched_clinical_pipeline](https://nlp.johnsnowlabs.com/2022/03/31/re_temporal_events_enriched_clinical_pipeline_en_3_0.html)  | |



[In this link](https://nlp.johnsnowlabs.com/docs/en/best_practices_pretrained_models#relation-extraction-models-and-relation-pairs-table), available Relation Extraction models, its labels, optimal NER model, and meaningful relation pairs are illustrated.

#### Posology Relation Extraction

This is a demonstration of using SparkNLP for extracting posology relations. The following relatios are supported:

DRUG-DOSAGE
DRUG-FREQUENCY
DRUG-ADE (Adversed Drug Events)
DRUG-FORM
DRUG-ROUTE
DRUG-DURATION
DRUG-REASON
DRUG=STRENGTH

The model has been validated against the posology dataset described in (Magge, Scotch, & Gonzalez-Hernandez, 2018).

| Relation | Recall | Precision | F1 | F1 (Magge, Scotch, & Gonzalez-Hernandez, 2018) |
| --- | --- | --- | --- | --- |
| DRUG-ADE | 0.66 | 1.00 | **0.80** | 0.76 |
| DRUG-DOSAGE | 0.89 | 1.00 | **0.94** | 0.91 |
| DRUG-DURATION | 0.75 | 1.00 | **0.85** | 0.92 |
| DRUG-FORM | 0.88 | 1.00 | **0.94** | 0.95* |
| DRUG-FREQUENCY | 0.79 | 1.00 | **0.88** | 0.90 |
| DRUG-REASON | 0.60 | 1.00 | **0.75** | 0.70 |
| DRUG-ROUTE | 0.79 | 1.00 | **0.88** | 0.95* |
| DRUG-STRENGTH | 0.95 | 1.00 | **0.98** | 0.97 |


*Magge, Scotch, Gonzalez-Hernandez (2018) collapsed DRUG-FORM and DRUG-ROUTE into a single relation.

**Build pipeline using SparNLP pretrained models and the relation extration model optimized for posology**.

 The precision of the RE model is controlled by "setMaxSyntacticDistance(4)", which sets the maximum syntactic distance between named entities to 4. A larger value will improve recall at the expense at lower precision. A value of 4 leads to literally perfect precision (i.e. the model doesn't produce any false positives) and reasonably good recall.

In [0]:
documenter = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

words_embedder = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

posology_ner = medical.NerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("posology_ner")

posology_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "posology_ner"]) \
    .setOutputCol("posology_ner_chunk")

dependency_parser = nlp.DependencyParserModel.pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

reModel = medical.RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "posology_ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
#   .setRelationPairs(["DRUG-FREQUENCY, FREQUENCY-DRUG", "DOSAGE-DRUG, DRUG-DOSAGE"])\
#   .setRelationPairsCaseSensitive(False)

relationPipeline = nlp.Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    posology_ner,
    posology_ner_converter,
    dependency_parser,
    reModel
])


relation_model = relationPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][ / ][OK!]
ner_posology download started this may take some time.
[ | ][ / ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

Now, let's create a light pipeline for annotating free text

In [0]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night ,
12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

lmodel = nlp.LightPipeline(relation_model)
results = lmodel.fullAnnotate(text)

In [0]:

from sparknlp_display import RelationExtractionVisualizer

'''
The `RelationExtractionVisualizer` can be used to visualize the relations predicted by Spark NLP. 
The two entities involved in a relation will be highlighted and their label will be displayed. 
Also a directed and labeled arc(line) will be used to connect the two entities.
'''
visualizer = RelationExtractionVisualizer()
vis= visualizer.display(results[0], 'relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

The patient was prescribed 1 unit DOSAGE of Advil DRUG for 5 days DURATION after meals. The patient was also given 1 unit DOSAGE of Metformin DRUG daily FREQUENCY . He was seen by the endocrinology service and she was discharged on 40 units DOSAGE of insulin glargine DRUG at night FREQUENCY , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG 1000 mg STRENGTH two times a day FREQUENCY . <polyline fill="none" points="333.0,155.0 332.11224489795916,153.7960798646822 331.2244897959183,152.59312871337622 330.3367346938776,151.39211553009378 329.44897959183675,150.19400929884657 328.5612244897959,148.99977900364644 327.67346938775506,147.81039362850512 326.78571428571433,146.62682215743445 325.8979591836735,145.45003357444605 325.01020408163265,144.28099686355176 324.1224489795918,143.12068100876337 323.234693877551,141.97005499409258 322.34693877551024,140.83008780355124 321.4591836734694,139.70174842115105 320.57142857142856,138.5860058309038 319.6836734693878,137.48382901682123 318.7959183673469,136.39618696291512 317.90816326530614,135.32404865319725 317.0204081632653,134.2683830716793 316.1326530612245,133.23015920237316 315.2448979591836,132.2103460292905 314.3571428571428,131.20991253644314 313.46938775510205,130.22982770784282 312.5816326530612,129.2710605275013 311.6938775510204,128.33457997943034 310.80612244897964,127.42135504764173 309.9183673469388,126.53235471614718 309.03061224489795,125.6685479689585 308.14285714285717,124.83090379008749 307.2551020408163,124.02039116354581 306.36734693877554,123.2379790733453 305.4795918367347,122.48463650349771 304.59183673469386,121.76133243801479 303.7040816326531,121.06903586090831 302.81632653061223,120.40871575619002 301.92857142857144,119.78134110787173 301.0408163265306,119.18788089996517 300.1530612244898,118.62930411648207 299.265306122449,118.10657974143427 298.3775510204082,117.62067675883348 297.48979591836735,117.17256415269148 296.60204081632656,116.76321090702004 295.7142857142857,116.39358600583091 294.8265306122449,116.06465843313586 293.9387755102041,115.77739717294665 293.05102040816325,115.53277120927505 292.16326530612247,115.33174952613282 291.2755102040816,115.17530110753174 290.38775510204084,115.06439493748353 289.5,115.0 289.5,115.0 288.61224489795916,114.9827622844223 287.7244897959184,115.01203580140927 286.83673469387753,115.08685156694916 285.94897959183675,115.20624059703015 285.0612244897959,115.36923390764053 284.17346938775506,115.57486251476851 283.28571428571433,115.82215743440234 282.39795918367355,116.11014968253025 281.51020408163276,116.43787027514047 280.6224489795918,116.80435022822121 279.73469387755097,117.20862055776078 278.8469387755101,117.64971227974738 277.95918367346934,118.12665641016923 277.07142857142856,118.63848396501459 276.1836734693878,119.18422596027166 275.2959183673469,119.7629134119287 274.40816326530614,120.37357733597398 273.5204081632653,121.01524874839565 272.63265306122446,121.68695866518203 271.7448979591836,122.3877381023213 270.85714285714283,123.11661807580174 269.96938775510205,123.87262960161156 269.0816326530612,124.654803695739 268.1938775510204,125.46217137417233 267.3061224489796,126.29376365289971 266.41836734693874,127.14861154790944 265.53061224489795,128.02574607518974 264.64285714285717,128.92419825072886 263.7551020408163,129.842999090515 262.86734693877554,130.7811796105364 261.97959183673476,131.73777082678137 261.09183673469386,132.71180375523804 260.2040816326531,133.7023094118947 259.31632653061223,134.7083188127396 258.42857142857144,135.72886297376095 257.54081632653066,136.76297291094698 256.6530612244898,137.80967964028594 255.76530612244898,138.86801417776604 254.87755102040816,139.9370075393756 253.98979591836735,141.01569074110276 253.10204081632654,142.1030947989358 252.21428571428572,143.19825072886297 251.32653061224488,144.30018954687245 250.4387755102041,145.40794226895255 249.55102040816325,146.52053991109145 248.66326530612247,147.63701348927742 247.7

In [0]:
results[0].keys()


Out[61]: dict_keys(['posology_ner', 'document', 'posology_ner_chunk', 'token', 'relations', 'embeddings', 'pos_tags', 'dependencies', 'sentence'])

In [0]:
results[0]['posology_ner_chunk']


Out[62]: [Annotation(chunk, 28, 33, 1 unit, {'chunk': '0', 'confidence': '0.71675', 'ner_source': 'posology_ner_chunk', 'entity': 'DOSAGE', 'sentence': '0'}, []),
 Annotation(chunk, 38, 42, Advil, {'chunk': '1', 'confidence': '0.9984', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '0'}, []),
 Annotation(chunk, 44, 53, for 5 days, {'chunk': '2', 'confidence': '0.7455', 'ner_source': 'posology_ner_chunk', 'entity': 'DURATION', 'sentence': '0'}, []),
 Annotation(chunk, 95, 100, 1 unit, {'chunk': '3', 'confidence': '0.72360003', 'ner_source': 'posology_ner_chunk', 'entity': 'DOSAGE', 'sentence': '1'}, []),
 Annotation(chunk, 105, 113, Metformin, {'chunk': '4', 'confidence': '0.9998', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '1'}, []),
 Annotation(chunk, 115, 119, daily, {'chunk': '5', 'confidence': '0.9997', 'ner_source': 'posology_ner_chunk', 'entity': 'FREQUENCY', 'sentence': '1'}, []),
 Annotation(chunk, 189, 196, 40 units, {'chunk': '6', 'confidence': '0.84085', 'ner_source': 'posology_ner_chunk', 'entity': 'DOSAGE', 'sentence': '2'}, []),
 Annotation(chunk, 201, 216, insulin glargine, {'chunk': '7', 'confidence': '0.87525', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '2'}, []),
 Annotation(chunk, 218, 225, at night, {'chunk': '8', 'confidence': '0.76839995', 'ner_source': 'posology_ner_chunk', 'entity': 'FREQUENCY', 'sentence': '2'}, []),
 Annotation(chunk, 229, 236, 12 units, {'chunk': '9', 'confidence': '0.88755', 'ner_source': 'posology_ner_chunk', 'entity': 'DOSAGE', 'sentence': '2'}, []),
 Annotation(chunk, 241, 254, insulin lispro, {'chunk': '10', 'confidence': '0.77805', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '2'}, []),
 Annotation(chunk, 256, 265, with meals, {'chunk': '11', 'confidence': '0.9213', 'ner_source': 'posology_ner_chunk', 'entity': 'FREQUENCY', 'sentence': '2'}, []),
 Annotation(chunk, 273, 281, metformin, {'chunk': '12', 'confidence': '0.9997', 'ner_source': 'posology_ner_chunk', 'entity': 'DRUG', 'sentence': '2'}, []),
 Annotation(chunk, 283, 289, 1000 mg, {'chunk': '13', 'confidence': '0.84535', 'ner_source': 'posology_ner_chunk', 'entity': 'STRENGTH', 'sentence': '2'}, []),
 Annotation(chunk, 291, 305, two times a day, {'chunk': '14', 'confidence': '0.806225', 'ner_source': 'posology_ner_chunk', 'entity': 'FREQUENCY', 'sentence': '2'}, [])]

In [0]:
results[0]['relations']

Out[63]: [Annotation(category, 28, 42, DOSAGE-DRUG, {'chunk2': 'Advil', 'confidence': '1.0', 'entity2_end': '42', 'chunk1': '1 unit', 'entity1': 'DOSAGE', 'entity2_begin': '38', 'chunk2_confidence': '0.9984', 'entity1_begin': '28', 'sentence': '0', 'direction': 'both', 'entity1_end': '33', 'entity2': 'DRUG', 'chunk1_confidence': '0.71675'}, []),
 Annotation(category, 38, 53, DRUG-DURATION, {'chunk2': 'for 5 days', 'confidence': '1.0', 'entity2_end': '53', 'chunk1': 'Advil', 'entity1': 'DRUG', 'entity2_begin': '44', 'chunk2_confidence': '0.7455', 'entity1_begin': '38', 'sentence': '0', 'direction': 'both', 'entity1_end': '42', 'entity2': 'DURATION', 'chunk1_confidence': '0.9984'}, []),
 Annotation(category, 95, 113, DOSAGE-DRUG, {'chunk2': 'Metformin', 'confidence': '1.0', 'entity2_end': '113', 'chunk1': '1 unit', 'entity1': 'DOSAGE', 'entity2_begin': '105', 'chunk2_confidence': '0.9998', 'entity1_begin': '95', 'sentence': '1', 'direction': 'both', 'entity1_end': '100', 'entity2': 'DRUG', 'chunk1_confidence': '0.72360003'}, []),
 Annotation(category, 105, 119, DRUG-FREQUENCY, {'chunk2': 'daily', 'confidence': '1.0', 'entity2_end': '119', 'chunk1': 'Metformin', 'entity1': 'DRUG', 'entity2_begin': '115', 'chunk2_confidence': '0.9997', 'entity1_begin': '105', 'sentence': '1', 'direction': 'both', 'entity1_end': '113', 'entity2': 'FREQUENCY', 'chunk1_confidence': '0.9998'}, []),
 Annotation(category, 189, 216, DOSAGE-DRUG, {'chunk2': 'insulin glargine', 'confidence': '1.0', 'entity2_end': '216', 'chunk1': '40 units', 'entity1': 'DOSAGE', 'entity2_begin': '201', 'chunk2_confidence': '0.87525', 'entity1_begin': '189', 'sentence': '2', 'direction': 'both', 'entity1_end': '196', 'entity2': 'DRUG', 'chunk1_confidence': '0.84085'}, []),
 Annotation(category, 201, 225, DRUG-FREQUENCY, {'chunk2': 'at night', 'confidence': '1.0', 'entity2_end': '225', 'chunk1': 'insulin glargine', 'entity1': 'DRUG', 'entity2_begin': '218', 'chunk2_confidence': '0.76839995', 'entity1_begin': '201', 'sentence': '2', 'direction': 'both', 'entity1_end': '216', 'entity2': 'FREQUENCY', 'chunk1_confidence': '0.87525'}, []),
 Annotation(category, 229, 254, DOSAGE-DRUG, {'chunk2': 'insulin lispro', 'confidence': '1.0', 'entity2_end': '254', 'chunk1': '12 units', 'entity1': 'DOSAGE', 'entity2_begin': '241', 'chunk2_confidence': '0.77805', 'entity1_begin': '229', 'sentence': '2', 'direction': 'both', 'entity1_end': '236', 'entity2': 'DRUG', 'chunk1_confidence': '0.88755'}, []),
 Annotation(category, 241, 265, DRUG-FREQUENCY, {'chunk2': 'with meals', 'confidence': '1.0', 'entity2_end': '265', 'chunk1': 'insulin lispro', 'entity1': 'DRUG', 'entity2_begin': '256', 'chunk2_confidence': '0.9213', 'entity1_begin': '241', 'sentence': '2', 'direction': 'both', 'entity1_end': '254', 'entity2': 'FREQUENCY', 'chunk1_confidence': '0.77805'}, []),
 Annotation(category, 273, 289, DRUG-STRENGTH, {'chunk2': '1000 mg', 'confidence': '1.0', 'entity2_end': '289', 'chunk1': 'metformin', 'entity1': 'DRUG', 'entity2_begin': '283', 'chunk2_confidence': '0.84535', 'entity1_begin': '273', 'sentence': '2', 'direction': 'both', 'entity1_end': '281', 'entity2': 'STRENGTH', 'chunk1_confidence': '0.9997'}, []),
 Annotation(category, 273, 305, DRUG-FREQUENCY, {'chunk2': 'two times a day', 'confidence': '1.0', 'entity2_end': '305', 'chunk1': 'metformin', 'entity1': 'DRUG', 'entity2_begin': '291', 'chunk2_confidence': '0.806225', 'entity1_begin': '273', 'sentence': '2', 'direction': 'both', 'entity1_end': '281', 'entity2': 'FREQUENCY', 'chunk1_confidence': '0.9997'}, [])]

Showing extracted relations

In [0]:
for rel in results[0]["relations"]:
    print("{}({}={} - {}={})".format(
        rel.result,
        rel.metadata['entity1'],
        rel.metadata['chunk1'],
        rel.metadata['entity2'],
        rel.metadata['chunk2']
    ))

DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Advil)
DRUG-DURATION(DRUG=Advil - DURATION=for 5 days)
DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Metformin)
DRUG-FREQUENCY(DRUG=Metformin - FREQUENCY=daily)
DOSAGE-DRUG(DOSAGE=40 units - DRUG=insulin glargine)
DRUG-FREQUENCY(DRUG=insulin glargine - FREQUENCY=at night)
DOSAGE-DRUG(DOSAGE=12 units - DRUG=insulin lispro)
DRUG-FREQUENCY(DRUG=insulin lispro - FREQUENCY=with meals)
DRUG-STRENGTH(DRUG=metformin - STRENGTH=1000 mg)
DRUG-FREQUENCY(DRUG=metformin - FREQUENCY=two times a day)

In [0]:
# get relations in a pandas dataframe

import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result,
          rel.metadata['entity1'],
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'],
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'],
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df


rel_df = get_relations_df (results)
rel_df

Out[65]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,DOSAGE-DRUG,DOSAGE,28,33,1 unit,DRUG,38,42,Advil,1.0
1,DRUG-DURATION,DRUG,38,42,Advil,DURATION,44,53,for 5 days,1.0
2,DOSAGE-DRUG,DOSAGE,95,100,1 unit,DRUG,105,113,Metformin,1.0
3,DRUG-FREQUENCY,DRUG,105,113,Metformin,FREQUENCY,115,119,daily,1.0
4,DOSAGE-DRUG,DOSAGE,189,196,40 units,DRUG,201,216,insulin glargine,1.0
5,DRUG-FREQUENCY,DRUG,201,216,insulin glargine,FREQUENCY,218,225,at night,1.0
6,DOSAGE-DRUG,DOSAGE,229,236,12 units,DRUG,241,254,insulin lispro,1.0
7,DRUG-FREQUENCY,DRUG,241,254,insulin lispro,FREQUENCY,256,265,with meals,1.0
8,DRUG-STRENGTH,DRUG,273,281,metformin,STRENGTH,283,289,1000 mg,1.0
9,DRUG-FREQUENCY,DRUG,273,281,metformin,FREQUENCY,291,305,two times a day,1.0


#### ReDL - ADE

Now, let's create a sample pipeline with `redl_ade_biobert` model and see how it works

In [0]:
documenter = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

words_embedder = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

ade_ner = medical.NerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token", "embeddings")\
    .setOutputCol("ade_ner")

ade_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ade_ner"]) \
    .setOutputCol("ade_ner_chunk")

ade_re_ner_chunk_filter = medical.RENerChunksFilter() \
    .setInputCols(["ade_ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])

ade_re_model = medical.RelationExtractionDLModel()\
    .pretrained('redl_ade_biobert', 'en', "clinical/models") \
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunks", "sentence"]) \
    .setOutputCol("relations")

ade_pipeline = nlp.Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    ade_ner,
    ade_ner_converter,
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

ade_model = ade_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
ner_ade_clinical download started this may take some time.
[ | ][ / ][OK!]
redl_ade_biobert download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
text ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

ade_lmodel = nlp.LightPipeline(ade_model)
ade_results = ade_lmodel.fullAnnotate(text)

rel_df = get_relations_df (ade_results)
rel_df

Out[67]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,DRUG,25,32,naproxen,ADE,137,148,tense bullae,0.9989047
1,1,DRUG,25,32,naproxen,ADE,154,210,cutaneous fragility on the face and the back of the hands,0.9989704
2,1,DRUG,87,95,oxaprozin,ADE,137,148,tense bullae,0.99895453
3,1,DRUG,87,95,oxaprozin,ADE,154,210,cutaneous fragility on the face and the back of the hands,0.99900633


### Merging Multiple RE Model Results

We can merge multiple RE model results by using `AnnotationMerger()`. <br/>
Now, we will build a pipeline consisting of `posology_re`, `re_ade_clinical` models and `AnnotationMerger()` to merge these RE models' results.

In [0]:
documenter = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

words_embedder = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

posology_ner = medical.NerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("posology_ner")

posology_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "posology_ner"]) \
    .setOutputCol("posology_ner_chunk")

dependency_parser = nlp.DependencyParserModel.pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

pos_reModel = medical.RelationExtractionModel.pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "posology_ner_chunk", "dependencies"])\
    .setOutputCol("pos_relations")\
    .setMaxSyntacticDistance(4)

ade_ner = medical.NerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token", "embeddings")\
    .setOutputCol("ade_ner")

ade_ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ade_ner"]) \
    .setOutputCol("ade_ner_chunk")

ade_reModel = medical.RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ade_ner_chunk", "dependencies"])\
    .setOutputCol("ade_relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setCustomLabels({"1": "is_ade", "0": "not_ade"})

annotation_merger = medical.AnnotationMerger()\
    .setInputCols("ade_relations", "pos_relations")\
    .setInputType("category")\
    .setOutputCol("all_relations")

merger_pipeline = nlp.Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    posology_ner,
    posology_ner_converter,
    dependency_parser,
    pos_reModel,
    ade_ner,
    ade_ner_converter,
    ade_reModel,
    annotation_merger
])


merger_model= merger_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
ner_posology download started this may take some time.
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
ner_ade_clinical download started this may take some time.
[ | ][OK!]
re_ade_clinical download started this may take some time.
[ | ][ / ][ — ][ \ ][OK!]

In [0]:
text = """
The patient was prescribed 1 unit of naproxen for 5 days after meals for chronic low back pain. The patient was also given 1 unit of oxaprozin daily for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands..
"""
data = spark.createDataFrame([[text]]).toDF("text")

result = merger_model.transform(data)
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| pos_tags| posology_ner| posology_ner_chunk| dependencies| pos_relations| ade_ner| ade_ner_chunk| ade_relations| all_relations|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
\nThe patient was...|[{document, 0, 26...|[{document, 1, 95...|[{token, 1, 3, Th...|[{word_embeddings...|[{pos, 1, 3, DD, ...|[{named_entity, 1...|[{chunk, 28, 33, ...|[{dependency, 1, ...|[{category, 28, 4...|[{named_entity, 1...|[{chunk, 38, 45, ...|[{category, 134, ...|[{category, 134, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:

from pyspark.sql import functions as F

result_df = result.select(F.explode(F.arrays_zip(result.pos_relations.result, result.pos_relations.metadata, result.ade_relations.result, result.ade_relations.metadata, result.all_relations.result, result.all_relations.metadata)).alias("cols")) \
                  .select(
                          F.expr("cols['0']").alias("pos_relations"),\
                          F.expr("cols['1']['entity1']").alias("pos_relations_entity1"),\
                          F.expr("cols['1']['chunk1']" ).alias("pos_relations_chunk1" ),\
                          F.expr("cols['1']['entity2']").alias("pos_relations_entity2"),\
                          F.expr("cols['1']['chunk2']" ).alias("pos_relations_chunk2" ),\
                          F.expr("cols['2']").alias("ade_relations"),\
                          F.expr("cols['3']['entity1']").alias("ade_relations_entity1"),\
                          F.expr("cols['3']['chunk1']" ).alias("ade_relations_chunk1" ),\
                          F.expr("cols['3']['entity2']").alias("ade_relations_entity2"),\
                          F.expr("cols['3']['chunk2']" ).alias("ade_relations_chunk2" ),\
                          F.expr("cols['4']").alias("all_relations"),\
                          F.expr("cols['5']['entity1']").alias("all_relations_entity1"),\
                          F.expr("cols['5']['chunk1']" ).alias("all_relations_chunk1" ),\
                          F.expr("cols['5']['entity2']").alias("all_relations_entity2"),\
                          F.expr("cols['5']['chunk2']" ).alias("all_relations_chunk2" )
                          )

result_df.show()

+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+
 pos_relations|pos_relations_entity1|pos_relations_chunk1|pos_relations_entity2|pos_relations_chunk2|ade_relations|ade_relations_entity1|ade_relations_chunk1|ade_relations_entity2|ade_relations_chunk2| all_relations|all_relations_entity1|all_relations_chunk1|all_relations_entity2|all_relations_chunk2|
+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+
 DOSAGE-DRUG| DOSAGE| 1 unit| DRUG| naproxen| is_ade| DRUG| oxaprozin| ADE| tense bullae| is_ade| DRUG| oxaprozin| ADE| tense bullae|
 DRUG-DURATION| DRUG| naproxen| DURATION| for 5 days| is_ade| DRUG| oxaprozin| ADE|cutaneous fragili...| is_ade| DRUG| oxaprozin| ADE|cutaneous fragili...|
 DOSAGE-DRUG| DOSAGE| 1 unit| DRUG| oxaprozin| null| null| null| null| null| DOSAGE-DRUG| DOSAGE| 1 unit| DRUG| naproxen|
DRUG-FREQUENCY| DRUG| oxaprozin| FREQUENCY| daily| null| null| null| null| null| DRUG-DURATION| DRUG| naproxen| DURATION| for 5 days|
 null| null| null| null| null| null| null| null| null| null| DOSAGE-DRUG| DOSAGE| 1 unit| DRUG| oxaprozin|
 null| null| null| null| null| null| null| null| null| null|DRUG-FREQUENCY| DRUG| oxaprozin| FREQUENCY| daily|
+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+

In [0]:

lmodel = LightPipeline(merger_model)
merged_results = lmodel.fullAnnotate(text)

visualizer = RelationExtractionVisualizer()
vis= visualizer.display(merged_results[0], 'all_relations', show_relations=True, return_html=True) # default show_relations: True
displayHTML(vis)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2903428274375476> in <module> 
 ----> 1 lmodel = LightPipeline ( merger_model ) 
 2 merged_results = lmodel . fullAnnotate ( text ) 
 3 
 4 visualizer = RelationExtractionVisualizer ( ) 
 5 vis = visualizer . display ( merged_results [ 0 ] , 'all_relations' , show_relations = True , return_html = True ) # default show_relations: True 

 NameError : name 'LightPipeline' is not defined

## Zero-shot Clinical Relation Extraction Model

Zero-shot Relation Extraction to extract relations between clinical entities with no training dataset. zero-shot relation extraction model leverages `BertForSequenceClassificaiton` to return, based on a predefined set of relation candidates (including no-relation / O), which one has the higher probability to be linking two entities.

The dataset will be a csv which contains the following columns: `sentence`, `chunk1`, `firstCharEnt1`, `lastCharEnt1`, `label1`, `chunk2`, `firstCharEnt2`, `lastCharEnt2`, `label2`, `rel`.

For more detailed explanation of the usage of this model, please visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/10.3.ZeroShot_Clinical_Relation_Extraction.ipynb)

Now, we will use the pretrained `re_zeroshot_biobert` model, this model available in Models Hub under the Relation Extraction category.

This is our Zero-shot RE pipeline;

In [0]:
# Clinical NER

documenter = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentencer = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("tokens")

words_embedder = nlp.WordEmbeddingsModel() \
    .pretrained("embeddings_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"]) \
    .setOutputCol("embeddings")

ner_clinical = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"]) \
    .setOutputCol("ner_clinical")

ner_clinical_converter = medical.NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_clinical"]) \
    .setOutputCol("ner_clinical_chunks")\
    .setWhiteList(["PROBLEM", "TEST"])      # PROBLEM-TEST-TREATMENT

ner_posology = medical.NerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens", "embeddings"]) \
    .setOutputCol("ner_posology")

ner_posology_converter = medical.NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_posology"]) \
    .setOutputCol("ner_posology_chunks")\
    .setWhiteList(["DRUG"])                # DRUG-FREQUENCY-DOSAGE-DURATION-FORM-ROUTE-STRENGTH

chunk_merger = medical.ChunkMergeApproach()\
    .setInputCols("ner_clinical_chunks", "ner_posology_chunks")\
    .setOutputCol('merged_ner_chunks')


## ZERO-SHOT RE Starting...

pos_tagger = nlp.PerceptronModel() \
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"]) \
    .setOutputCol("pos_tags")

dependency_parser = nlp.DependencyParserModel() \
    .pretrained("dependency_conllu", "en") \
    .setInputCols(["document", "pos_tags", "tokens"]) \
    .setOutputCol("dependencies")

re_ner_chunk_filter = medical.RENerChunksFilter() \
    .setRelationPairs(["problem-test","problem-drug"]) \
    .setMaxSyntacticDistance(4)\
    .setDocLevelRelations(False)\
    .setInputCols(["merged_ner_chunks", "dependencies"]) \
    .setOutputCol("re_ner_chunks")

re_model = medical.ZeroShotRelationExtractionModel.pretrained("re_zeroshot_biobert", "en", "clinical/models")\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")\
    .setRelationalCategories({
                            "ADE": ["{DRUG} causes {PROBLEM}."],
                            "IMPROVE": ["{DRUG} improves {PROBLEM}.", "{DRUG} cures {PROBLEM}."],
                            "REVEAL": ["{TEST} reveals {PROBLEM}."]})\
    .setMultiLabel(True)

pipeline = nlp.Pipeline() \
    .setStages([documenter,
                sentencer,
                tokenizer,
                words_embedder,
                ner_clinical,
                ner_clinical_converter,
                ner_posology,
                ner_posology_converter,
                chunk_merger,
                pos_tagger,
                dependency_parser,
                re_ner_chunk_filter,
                re_model])

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
[ | ][ / ][OK!]
ner_posology download started this may take some time.
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
re_zeroshot_biobert download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
# create Spark DF

sample_text = "Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer."

data = spark.createDataFrame([[sample_text]]).toDF("text")
data.show(truncate=False)

+---------------------------------------------------------------------------------------+
text |
+---------------------------------------------------------------------------------------+
Paracetamol can alleviate headache or sickness. An MRI test can be used to find cancer.|
+---------------------------------------------------------------------------------------+

In [0]:
zero_model = pipeline.fit(data)

results = zero_model.transform(data)

results\
    .selectExpr("explode(relations) as relation")\
    .show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
relation |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
{category, 1086, 1297, ADE, {entity1_begin -> 134, relation -> ADE, hypothesis -> oxaprozin causes tense bullae., confidence -> 0.979243, nli_prediction -> entail, entity1 -> DRUG, syntactic_distance -> 4, chunk2 -> tense bullae, entity2_end -> 201, entity1_end -> 142, entity2_begin -> 190, entity2 -> PROBLEM, chunk1 -> oxaprozin, sentence -> 1}, []} |
{category, 441, 660, ADE, {entity1_begin -> 134, relation -> ADE, hypothesis -> oxaprozin causes rheumatoid arthritis., confidence -> 0.98758495, nli_prediction -> entail, entity1 -> DRUG, syntactic_distance -> 3, chunk2 -> rheumatoid arthritis, entity2_end -> 173, entity1_end -> 142, entity2_begin -> 154, entity2 -> PROBLEM, chunk1 -> oxaprozin, sentence -> 1}, []}|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [0]:
# results in Spark DF

from pyspark.sql import functions as F

results.select(F.explode(F.arrays_zip(results.relations.metadata, results.relations.result)).alias("cols"))\
       .select(F.expr("cols['0']['sentence']").alias("sentence"),
               F.expr("cols['0']['entity1_begin']").alias("entity1_begin"),
               F.expr("cols['0']['entity1_end']").alias("entity1_end"),
               F.expr("cols['0']['chunk1']").alias("chunk1"),
               F.expr("cols['0']['entity1']").alias("entity1"),
               F.expr("cols['0']['entity2_begin']").alias("entity2_begin"),
               F.expr("cols['0']['entity2_end']").alias("entity2_end"),
               F.expr("cols['0']['chunk2']").alias("chunk2"),
               F.expr("cols['0']['entity2']").alias("entity2"),
               F.expr("cols['0']['hypothesis']").alias("hypothesis"),
               F.expr("cols['0']['nli_prediction']").alias("nli_prediction"),
               F.expr("cols['1']").alias("relation"),
               F.expr("cols['0']['confidence']").alias("confidence"),
       ).show(truncate=70)

+--------+-------------+-----------+---------+-------+-------------+-----------+--------------------+-------+--------------------------------------+--------------+--------+----------+
sentence|entity1_begin|entity1_end| chunk1|entity1|entity2_begin|entity2_end| chunk2|entity2| hypothesis|nli_prediction|relation|confidence|
+--------+-------------+-----------+---------+-------+-------------+-----------+--------------------+-------+--------------------------------------+--------------+--------+----------+
 1| 134| 142|oxaprozin| DRUG| 190| 201| tense bullae|PROBLEM| oxaprozin causes tense bullae.| entail| ADE| 0.979243|
 1| 134| 142|oxaprozin| DRUG| 154| 173|rheumatoid arthritis|PROBLEM|oxaprozin causes rheumatoid arthritis.| entail| ADE|0.98758495|
+--------+-------------+-----------+---------+-------+-------------+-----------+--------------------+-------+--------------------------------------+--------------+--------+----------+

## Train a Custom Relation Extraction Model

Now, we will train our custom RE model with sample data, let's see how it works

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_clinical_rel_dataset.csv

dbutils.fs.cp("file:/databricks/driver/i2b2_clinical_rel_dataset.csv", "dbfs:/")

Out[75]: True

In [0]:

data = spark.read.option("header","true").format("csv").load("/i2b2_clinical_rel_dataset.csv")
data = data.select( 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel','dataset')

data.show(10)

# you only need these columns>> 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel'
# ('dataset' column is optional)

+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
 sentence|firstCharEnt1|firstCharEnt2|lastCharEnt1|lastCharEnt2| chunk1| chunk2| label1| label2| rel|dataset|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
VITAL SIGNS - Tem...| 49| 75| 64| 84| respiratory rate| saturation| test| test| O| test|
No lotions , crea...| 3| 34| 9| 42| lotions| incisions|treatment| problem|TrNAP| test|
Because of expect...| 11| 58| 54| 68|expected long ter...| a picc line|treatment|treatment| O| train|
She states this l...| 16| 82| 31| 92| light-headedness| diaphoresis| problem| problem| PIP| train|
Initial electroca...| 196| 145| 239| 176|an inferior and r...|1-mm st depressio...| problem| problem| PIP| test|
Abdominal ultraso...| 1| 54| 20| 71|abdominal ultrasound| gallbladder sludge| test| problem| TeRP| test|
At that time he w...| 99| 139| 133| 173|ir placement of a...|his abdominopelvi...|treatment| problem| TrAP| test|
He was transferre...| 143| 195| 154| 213| reintubation| worsening pneumonia|treatment| problem| TrAP| test|
5) Anemia - previ...| 12| 35| 23| 47| previous w/u| elev ferritin| test| problem| TeRP| test|
The patient &apos...| 161| 149| 179| 156| po pain medications| his pain|treatment| problem| TrAP| train|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
only showing top 10 rows

In [0]:
data.groupby('dataset').count().show()


+-------+-----+
dataset|count|
+-------+-----+
 train| 350|
 test| 650|
+-------+-----+

In [0]:
'''
# custom graph

from sparknlp_jsl.training import tf_graph

tf_graph.print_model_params("relation_extraction")

tf_graph.build("relation_extraction",
             build_params={"input_dim": 10000,
                          "output_dim": 10,
                          'batch_norm':1,
                          "hidden_layers": [300, 200],
                          "hidden_act": "relu",
                          'hidden_act_l2':1},
                          'model_location'=".",
                          'model_filename'="re_in6000D_out10.pb")
'''

'''
# ready to use graph
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/RE_in1200D_out20.pb
'''

Out[82]: '\n# ready to use graph\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/RE_in1200D_out20.pb\n'

In [0]:
from sparknlp_jsl.training import REDatasetHelper

# map entity columns to dataset columns
column_map = {
    "begin1": "firstCharEnt1",
    "end1": "lastCharEnt1",
    "begin2": "firstCharEnt2",
    "end2": "lastCharEnt2",
    "chunk1": "chunk1",
    "chunk2": "chunk2",
    "label1": "label1",
    "label2": "label2"
}

# apply preprocess function to dataframe
data = REDatasetHelper(data).create_annotation_column(
    column_map,
    ner_column_name="train_ner_chunks" # optional, default train_ner_chunks
)

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

In [0]:

data.show(10)


+------------+--------------------+--------------------+-------------+-------+---------+------------+---------+--------------------+--------------------+-----+-------------+
lastCharEnt1| train_ner_chunks| chunk2|firstCharEnt2|dataset| label2|lastCharEnt2| label1| chunk1| sentence| rel|firstCharEnt1|
+------------+--------------------+--------------------+-------------+-------+---------+------------+---------+--------------------+--------------------+-----+-------------+
 64|[{chunk, 49, 64, ...| saturation| 75| test| test| 84| test| respiratory rate|VITAL SIGNS - Tem...| O| 49|
 9|[{chunk, 3, 9, lo...| incisions| 34| test| problem| 42|treatment| lotions|No lotions , crea...|TrNAP| 3|
 54|[{chunk, 11, 54, ...| a picc line| 58| train|treatment| 68|treatment|expected long ter...|Because of expect...| O| 11|
 31|[{chunk, 16, 31, ...| diaphoresis| 82| train| problem| 92| problem| light-headedness|She states this l...| PIP| 16|
 239|[{chunk, 196, 239...|1-mm st depressio...| 145| test| problem| 176| problem|an inferior and r...|Initial electroca...| PIP| 196|
 20|[{chunk, 1, 20, a...| gallbladder sludge| 54| test| problem| 71| test|abdominal ultrasound|Abdominal ultraso...| TeRP| 1|
 133|[{chunk, 99, 133,...|his abdominopelvi...| 139| test| problem| 173|treatment|ir placement of a...|At that time he w...| TrAP| 99|
 154|[{chunk, 143, 154...| worsening pneumonia| 195| test| problem| 213|treatment| reintubation|He was transferre...| TrAP| 143|
 23|[{chunk, 12, 23, ...| elev ferritin| 35| test| problem| 47| test| previous w/u|5) Anemia - previ...| TeRP| 12|
 179|[{chunk, 161, 179...| his pain| 149| train| problem| 156|treatment| po pain medications|The patient &apos...| TrAP| 161|
+------------+--------------------+--------------------+-------------+-------+---------+------------+---------+--------------------+--------------------+-----+-------------+
only showing top 10 rows

We have prepared our train and test data in desired format for traininig.


#### RE Graph

We will use `TFGraphBuilder` annotator which can be used to create graphs automatically in the model training pipeline.

`TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the approach.

You can also create a custom graph by using `tf_graph` module in Spark NLP for Healthcare.

In [0]:
data.groupby('rel').count().show()


+-----+-----+
 rel|count|
+-----+-----+
 TrIP| 14|
 TrAP| 164|
 TeCP| 26|
 O| 414|
TrNAP| 14|
 TrCP| 28|
 PIP| 153|
 TrWP| 11|
 TeRP| 176|
+-----+-----+

In [0]:
!pip install -q tensorflow==2.11.0 tensorflow-addons

WARNING: You are using pip version 21.0.1; however, version 23.2.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
from sparknlp_jsl.annotator import TFGraphBuilder


In [0]:
graph_folder= "./tf_graphs"

re_graph_builder = TFGraphBuilder()\
    .setModelName("relation_extraction")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"]) \
    .setLabelColumn("rel")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("re_graph.pb")\
    .setHiddenLayers([300, 200])\
    .setHiddenAct("relu")\
    .setHiddenActL2(True)\
    .setHiddenWeightsL2(False)\
    .setBatchNorm(False)

In [0]:
train_documenter = nlp.DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

train_tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

train_embedder = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")
    
dependency_parser = nlp.DependencyParserModel.pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reApproach = medical.RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(80)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setModelFile("/dbfs/re_graphs/rel_e.in1200.out20.pb")\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setOutputLogsPath('dbfs:/re_logs')

finisher = nlp.Finisher()\
    .setInputCols(["relations"])\
    .setOutputCols(["relations_out"])\
    .setCleanAnnotations(False)\
    .setValueSplitSymbol(",")\
    .setAnnotationSplitSymbol(",")\
    .setOutputAsArray(False)

train_pipeline = nlp.Pipeline(stages=[
    train_documenter, 
    train_tokenizer, 
    train_embedder, 
    pos_tagger, 
    dependency_parser, 
    re_graph_builder,
    reApproach, 
    finisher
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]

In [0]:
%%time

rel_model = train_pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}
relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 3.11 s, sys: 2.49 s, total: 5.6 s
Wall time: 23.1 s

Let's get predictions from test data and evaluate the results

In [0]:
result = rel_model.transform(test_data)

**Save the model and evaluate the performance** <br/>
We will create a pipeline and use our trained RE model by calling with `.load` method. Then we will get predictions by transforming our test set on this pipeline.

In [0]:
# save model
rel_model.stages[-2].write().overwrite().save('dbfs:/databricks/driver/models/custom_RE_model')


In [0]:
customReModel = medical.RelationExtractionModel.load("dbfs:/databricks/driver/models/custom_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations_pred")\
    .setMaxSyntacticDistance(0)

test_pipeline = nlp.Pipeline(stages=[
    train_documenter, 
    train_tokenizer, 
    train_embedder, 
    pos_tagger, 
    dependency_parser, 
    customReModel])

test_results = test_pipeline.fit(test_data).transform(test_data)

In [0]:
test_results.show(5)


+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
lastCharEnt1| train_ner_chunks| chunk2|firstCharEnt2|dataset| label2|lastCharEnt2| label1| chunk1| sentence| rel|firstCharEnt1| sentences| tokens| embeddings| pos_tags| dependencies| relations_pred|
+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 64|[{chunk, 49, 64, ...| saturation| 75| test| test| 84| test| respiratory rate|VITAL SIGNS - Tem...| O| 49|[{document, 0, 10...|[{token, 0, 4, VI...|[{word_embeddings...|[{pos, 0, 4, NN, ...|[{dependency, 0, ...|[{category, 49, 8...|
 9|[{chunk, 3, 9, lo...| incisions| 34| test|problem| 42|treatment| lotions|No lotions , crea...|TrNAP| 3|[{document, 0, 44...|[{token, 0, 1, No...|[{word_embeddings...|[{pos, 0, 1, DD, ...|[{dependency, 0, ...|[{category, 3, 42...|
 239|[{chunk, 196, 239...|1-mm st depressio...| 145| test|problem| 176| problem|an inferior and r...|Initial electroca...| PIP| 196|[{document, 0, 24...|[{token, 0, 6, In...|[{word_embeddings...|[{pos, 0, 6, JJ, ...|[{dependency, 0, ...|[{category, 145, ...|
 20|[{chunk, 1, 20, a...| gallbladder sludge| 54| test|problem| 71| test|abdominal ultrasound|Abdominal ultraso...| TeRP| 1|[{document, 0, 95...|[{token, 0, 8, Ab...|[{word_embeddings...|[{pos, 0, 8, JJ, ...|[{dependency, 0, ...|[{category, 1, 71...|
 133|[{chunk, 99, 133,...|his abdominopelvi...| 139| test|problem| 173|treatment|ir placement of a...|At that time he w...| TrAP| 99|[{document, 0, 17...|[{token, 0, 1, At...|[{word_embeddings...|[{pos, 0, 1, II, ...|[{dependency, 0, ...|[{category, 99, 1...|
+------------+--------------------+--------------------+-------------+-------+-------+------------+---------+--------------------+--------------------+-----+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

When you check the `relations_pred` column, you can see that some of the relations between the chunk pairs were not detected this time.

Now we will get metrics using the ground truth (`rel`) and prediction (`relations_pred`) columns in the result dataframe.

In [0]:
pd_test_results = test_results.select('rel', 'relations_pred.result').toPandas()
pd_test_results.head()

Out[91]:

,rel,result
0,O,[O]
1,TrNAP,[O]
2,PIP,[PIP]
3,TeRP,[TeRP]
4,TrAP,[TrAP]


We will explode the `result` column and fill null values as `O` label.

In [0]:
pd_test_results = pd_test_results.explode("result").fillna("O")
pd_test_results.result.value_counts()

Out[92]: O 328
PIP 116
TeRP 99
TrAP 99
TrNAP 4
TeCP 2
TrCP 1
TrIP 1
Name: result, dtype: int64

In [0]:
pd_test_results.rel.value_counts()

Out[93]: O 274
TeRP 116
TrAP 108
PIP 88
TrCP 20
TeCP 15
TrIP 11
TrNAP 9
TrWP 9
Name: rel, dtype: int64

In [0]:
from sklearn.metrics import classification_report
print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

precision recall f1-score support

 O 0.69 0.82 0.75 274
 PIP 0.54 0.72 0.62 88
 TeCP 0.00 0.00 0.00 15
 TeRP 0.78 0.66 0.72 116
 TrAP 0.60 0.55 0.57 108
 TrCP 0.00 0.00 0.00 20
 TrIP 1.00 0.09 0.17 11
 TrNAP 0.00 0.00 0.00 9
 TrWP 0.00 0.00 0.00 9

 accuracy 0.65 650
 macro avg 0.40 0.32 0.31 650
weighted avg 0.62 0.65 0.62 650

# 6.Clinical Entity Resolvers

# 6.Clinical Entity Resolvers

## Sentence Entity Resolver Models

**Entity Resolvers in SparkNLP**
- RxNorm
- ICD10-CM
- ICD-9-CM
- CPT
- SNOMED
- LOINC
- UMLS
- HPO
- HCPCS
- ICD-O
- MeSH
- NDC
- RX-CUI
- ATC


Sparknlp has more than **50 pretrained sentence entity resolver models**

<center><b>MODEL LIST</b>

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [sbertresolve_icd10cm_slim_billable_hcc_med](https://nlp.johnsnowlabs.com/2021/05/25/sbertresolve_icd10cm_slim_billable_hcc_med_en.html)  | 2| [sbertresolve_jsl_rxnorm_augmented_med](https://nlp.johnsnowlabs.com/2021/12/28/sbertresolve_jsl_rxnorm_augmented_med_en.html)  | 3| [sbertresolve_ner_model_finder](https://nlp.johnsnowlabs.com/2022/09/05/sbertresolve_ner_model_finder_en.html)  | 4| [sbertresolve_rxnorm_disposition](https://nlp.johnsnowlabs.com/2021/08/28/sbertresolve_rxnorm_disposition_en.html)  |
| 5| [sbertresolve_snomed_bodyStructure_med](https://nlp.johnsnowlabs.com/2021/06/15/sbertresolve_snomed_bodyStructure_med_en.html)  | 6| [sbertresolve_snomed_conditions](https://nlp.johnsnowlabs.com/2021/08/28/sbertresolve_snomed_conditions_en.html)  | 7| [sbiobert_jsl_rxnorm_cased](https://nlp.johnsnowlabs.com/2021/12/23/sbiobert_jsl_rxnorm_cased_en.html)  | 8| [sbiobertresolve_HPO](https://nlp.johnsnowlabs.com/2021/05/16/sbiobertresolve_HPO_en.html)  |
| 9| [sbiobertresolve_atc](https://nlp.johnsnowlabs.com/2022/03/01/sbiobertresolve_atc_en_3_0.html)  | 10| [sbiobertresolve_clinical_abbreviation_acronym](https://nlp.johnsnowlabs.com/2022/01/03/sbiobertresolve_clinical_abbreviation_acronym_en.html)  | 11| [sbiobertresolve_clinical_snomed_procedures_measurements](https://nlp.johnsnowlabs.com/2021/11/15/sbiobertresolve_clinical_snomed_procedures_measurements_en.html)  | 12| [sbiobertresolve_cpt](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_cpt_en.html)  |
| 13| [sbiobertresolve_cpt_augmented](https://nlp.johnsnowlabs.com/2021/05/30/sbiobertresolve_cpt_augmented_en.html)  | 14| [sbiobertresolve_cpt_procedures_augmented](https://nlp.johnsnowlabs.com/2021/05/30/sbiobertresolve_cpt_procedures_augmented_en.html)  | 15| [sbiobertresolve_cpt_procedures_measurements_augmented](https://nlp.johnsnowlabs.com/2022/05/10/sbiobertresolve_cpt_procedures_measurements_augmented_en_3_0.html)  | 16| [sbiobertresolve_cvx](https://nlp.johnsnowlabs.com/2022/10/12/sbiobertresolve_cvx_en.html)  |
| 17| [sbiobertresolve_hcc_augmented](https://nlp.johnsnowlabs.com/2021/05/30/sbiobertresolve_hcc_augmented_en.html)  | 18| [sbiobertresolve_hcpcs](https://nlp.johnsnowlabs.com/2021/09/29/sbiobertresolve_hcpcs_en.html)  | 19| [sbiobertresolve_icd10cm](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_icd10cm_en.html)  | 20| [sbiobertresolve_icd10cm_augmented](https://nlp.johnsnowlabs.com/2021/10/31/sbiobertresolve_icd10cm_augmented_en.html)  |
| 21| [sbiobertresolve_icd10cm_augmented_billable_hcc](https://nlp.johnsnowlabs.com/2021/11/01/sbiobertresolve_icd10cm_augmented_billable_hcc_en.html)  | 22| [sbiobertresolve_icd10cm_generalised](https://nlp.johnsnowlabs.com/2021/09/29/sbiobertresolve_icd10cm_generalised_en.html)  | 23| [sbiobertresolve_icd10cm_slim_billable_hcc](https://nlp.johnsnowlabs.com/2021/05/25/sbiobertresolve_icd10cm_slim_billable_hcc_en.html)  | 24| [sbiobertresolve_icd10cm_slim_normalized](https://nlp.johnsnowlabs.com/2021/05/17/sbiobertresolve_icd10cm_slim_normalized_en.html)  |
| 25| [sbiobertresolve_icd10pcs](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_icd10pcs_en.html)  | 26| [sbiobertresolve_icd10pcs_augmented](https://nlp.johnsnowlabs.com/2022/10/28/sbiobertresolve_icd10pcs_augmented_en.html)  | 27| [sbiobertresolve_icd9](https://nlp.johnsnowlabs.com/2022/09/30/sbiobertresolve_icd9_en.html)  | 28| [sbiobertresolve_icdo](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_icdo_en.html)  |
| 29| [sbiobertresolve_icdo_augmented](https://nlp.johnsnowlabs.com/2021/06/22/sbiobertresolve_icdo_augmented_en.html)  | 30| [sbiobertresolve_icdo_base](https://nlp.johnsnowlabs.com/2021/07/02/sbiobertresolve_icdo_base_en.html)  | 31| [sbiobertresolve_jsl_rxnorm_augmented](https://nlp.johnsnowlabs.com/2021/12/27/sbiobertresolve_jsl_rxnorm_augmented_en.html)  | 32| [sbiobertresolve_loinc](https://nlp.johnsnowlabs.com/2021/05/16/sbiobertresolve_loinc_en.html)  |
| 33| [sbiobertresolve_loinc_augmented](https://nlp.johnsnowlabs.com/2021/11/23/sbiobertresolve_loinc_augmented_en.html)  | 34| [sbiobertresolve_loinc_cased](https://nlp.johnsnowlabs.com/2021/12/24/sbiobertresolve_loinc_cased_en.html)  | 35| [sbiobertresolve_mesh](https://nlp.johnsnowlabs.com/2021/11/14/sbiobertresolve_mesh_en.html)  | 36| [sbiobertresolve_ndc](https://nlp.johnsnowlabs.com/2021/11/27/sbiobertresolve_ndc_en.html)  |
| 37| [sbiobertresolve_rxcui](https://nlp.johnsnowlabs.com/2021/05/16/sbiobertresolve_rxcui_en.html)  | 38| [sbiobertresolve_rxnorm](https://nlp.johnsnowlabs.com/2021/10/10/sbiobertresolve_rxnorm_en.html)  | 39| [sbiobertresolve_rxnorm_action_treatment](https://nlp.johnsnowlabs.com/2022/04/25/sbiobertresolve_rxnorm_action_treatment_en_2_4.html)  | 40| [sbiobertresolve_rxnorm_augmented](https://nlp.johnsnowlabs.com/2022/01/03/sbiobertresolve_rxnorm_augmented_en.html)  |
| 41| [sbiobertresolve_rxnorm_augmented_cased](https://nlp.johnsnowlabs.com/2021/12/28/sbiobertresolve_rxnorm_augmented_cased_en.html)  | 42| [sbiobertresolve_rxnorm_augmented_re](https://nlp.johnsnowlabs.com/2022/02/09/sbiobertresolve_rxnorm_augmented_re_en.html)  | 43| [sbiobertresolve_rxnorm_disposition](https://nlp.johnsnowlabs.com/2021/08/12/sbiobertresolve_rxnorm_disposition_en.html)  | 44| [sbiobertresolve_rxnorm_ndc](https://nlp.johnsnowlabs.com/2021/10/05/sbiobertresolve_rxnorm_ndc_en.html)  |
| 45| [sbiobertresolve_snomed_auxConcepts](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_snomed_auxConcepts_en.html)  | 46| [sbiobertresolve_snomed_auxConcepts_int](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_snomed_auxConcepts_int_en.html)  | 47| [sbiobertresolve_snomed_bodyStructure](https://nlp.johnsnowlabs.com/2021/06/15/sbiobertresolve_snomed_bodyStructure_en.html)  | 48| [sbiobertresolve_snomed_drug](https://nlp.johnsnowlabs.com/2022/01/18/sbiobertresolve_snomed_drug_en.html)  |
| 49| [sbiobertresolve_snomed_findings](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_snomed_findings_en.html)  | 50| [sbiobertresolve_snomed_findings_aux_concepts](https://nlp.johnsnowlabs.com/2021/07/14/sbiobertresolve_snomed_findings_aux_concepts_en.html)  | 51| [sbiobertresolve_snomed_findings_int](https://nlp.johnsnowlabs.com/2020/11/27/sbiobertresolve_snomed_findings_int_en.html)  | 52| [sbiobertresolve_snomed_procedures_measurements](https://nlp.johnsnowlabs.com/2021/11/11/sbiobertresolve_snomed_procedures_measurements_en.html)  |
| 53| [sbiobertresolve_umls_clinical_drugs](https://nlp.johnsnowlabs.com/2021/10/11/sbiobertresolve_umls_clinical_drugs_en.html)  | 54| [sbiobertresolve_umls_disease_syndrome](https://nlp.johnsnowlabs.com/2021/10/11/sbiobertresolve_umls_disease_syndrome_en.html)  | 55| [sbiobertresolve_umls_drug_substance](https://nlp.johnsnowlabs.com/2021/12/06/sbiobertresolve_umls_drug_substance_en.html)  | 56| [sbiobertresolve_umls_findings](https://nlp.johnsnowlabs.com/2021/10/03/sbiobertresolve_umls_findings_en.html)  |
| 57| [sbiobertresolve_umls_major_concepts](https://nlp.johnsnowlabs.com/2021/10/03/sbiobertresolve_umls_major_concepts_en.html)  | 58| [sbluebertresolve_loinc](https://nlp.johnsnowlabs.com/2021/04/29/sbluebertresolve_loinc_en.html)  | 59| [sbluebertresolve_loinc_uncased](https://nlp.johnsnowlabs.com/2022/01/18/sbluebertresolve_loinc_uncased_en.html)  | 60| [sbluebertresolve_rxnorm_augmented_uncased](https://nlp.johnsnowlabs.com/2021/12/28/sbluebertresolve_rxnorm_augmented_uncased_en.html)  |


<br>
<br>

<b>PRETRAINED PIPELINE LIST</b>

|index|model|
|-----:|:-----|
| 1| [cvx_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/10/12/cvx_resolver_pipeline_en.html)  |
| 2| [icd10cm_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/11/02/icd10cm_resolver_pipeline_en.html)  |
| 3| [icd9_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/09/30/icd9_resolver_pipeline_en.html)  |
| 4| [medication_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/09/01/medication_resolver_pipeline_en.html)  |
| 5| [medication_resolver_transform_pipeline](https://nlp.johnsnowlabs.com/2022/09/01/medication_resolver_transform_pipeline_en.html)  |
| 6| [umls_clinical_findings_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_clinical_findings_resolver_pipeline_en_3_0.html)  |
| 7| [umls_disease_syndrome_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_disease_syndrome_resolver_pipeline_en_3_0.html)  |
| 8| [umls_drug_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/26/umls_drug_resolver_pipeline_en_3_0.html)  |
| 9| [umls_drug_substance_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/25/umls_drug_substance_resolver_pipeline_en_3_0.html)  |
| 10| [umls_major_concepts_resolver_pipeline](https://nlp.johnsnowlabs.com/2022/07/25/umls_major_concepts_resolver_pipeline_en_3_0.html)  |





</center>

You can find all resolver models and more on [Models Hub](https://nlp.johnsnowlabs.com/models?task=Entity+Resolution&edition=Spark+NLP+for+Healthcare) page.

Our pipelines provide more metadata about the matching process other than providing the code in the "result" field;

- target_text -> Text to resolve
- resolved_text -> Best match text
- confidence -> Relative confidence for the top match (distance to probability)
- confidence_ratio -> Relative confidence for the top match. TopMatchConfidence / SecondMatchConfidence
- alternative_codes -> List of other plausible codes (in the KNN neighborhood)
- all_k_resolutions -> All codes descriptions
- all_k_results -> All resolved codes for metrics calculation purposes
- sentence -> SentenceId

We create a pipeline that from each of these problems, will try to assign a resolution on the content, the sentence embeddings and some pretrained models for resolver annotation.

The architecture of this new pipeline will be as follows:

- DocumentAssembler (text -> document)

- SentenceDetector (document -> sentence)

- Tokenizer (sentence -> token)

- WordEmbeddingsModel ([sentence, token] -> embeddings)

- MedicalNerModel ([sentence, token, embeddings] -> ner)

- NerConverter (["sentence, token, ner] -> ner_chunk

- Chunk2Doc (ner_chunk) -> ner_chunk_doc

- BertSentenceEmbeddings (ner_chunk_doc) -> sbert_embeddings

- SentenceEntityResolverModel ([ner_chunk, sbert_embeddings] -> resolution)

So from a text we end having a list of Named Entities (ner_chunk) and their resolutions.

`setPreservePosition(True)` takes exactly the original indices (under some tokenization conditions it might include some undesires chars like `")","]"...)`

`setPreservePosition(False)` takes adjusted indices based on substring indexing of the first (for begin) and last (for end) tokens


Let's see how we build a resolver pipeline

Firstly, we will write some **generic functions** for getting the codes and relation pairs

In [0]:
# returns LP resolution results

import pandas as pd
pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='icd10cm_code', col_name="ner_chunk", hcc=False):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []
    all_k_aux_labels=[]

    for chunk, code in zip(full_light_result[0][col_name], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))
        all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
        if hcc:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
        else:
            all_k_aux_labels.append([])

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes, 'all_codes':all_codes, 
                       'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})
    
    if hcc:

        df['billable'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[0] for i in x])
        df['hcc_status'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[1] for i in x])
        df['hcc_code'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[2] for i in x])

    df = df.drop(['all_k_aux_labels'], axis=1)
    
    return df

In [0]:
# returns spark df resolution results

def get_codes_from_df(result_df, chunk, output_col, hcc= False):


    if hcc:

        df = result_df.select(F.explode(F.arrays_zip(chunk+'.result',
                                                           chunk+'.metadata',
                                                           output_col+'.result',
                                                           output_col+'.metadata')).alias("cols")) \
                                     .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                             F.expr("cols['0']").alias("ner_chunk"),
                                             F.expr("cols['1']['entity']").alias("entity"),
                                             F.expr("cols['2']").alias("icd10_code"),
                                             F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                             F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                             F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()



        codes = []
        resolutions = []
        hcc_all = []

        for code, resolution, hcc in zip(df['all_codes'], df['resolutions'], df['hcc_list']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))
            hcc_all.append(hcc.split(":::"))

        df['all_codes'] = codes
        df['resolutions'] = resolutions
        df['hcc_list'] = hcc_all

    else:

        df = result_df.select(F.explode(F.arrays_zip(chunk+'.result',
                                                           chunk+'.metadata',
                                                           output_col+'.result',
                                                           output_col+'.metadata')).alias("cols")) \
                                     .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                             F.expr("cols['0']").alias("ner_chunk"),
                                             F.expr("cols['1']['entity']").alias("entity"),
                                             F.expr("cols['2']").alias(f"{output_col}"),
                                             F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                             F.expr("cols['3']['all_k_resolutions']").alias("resolutions")).toPandas()



        codes = []
        resolutions = []

        for code, resolution in zip(df['all_codes'], df['resolutions']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))

        df['all_codes'] = codes
        df['resolutions'] = resolutions


    return df

In [0]:
# returns billable informations in distinct columns

def extract_billable(bil):
  
    billable = []
    status = []
    code = []
 
    for b in bil:
        billable.append(b.split("||")[0])
        status.append(b.split("||")[1])
        code.append(b.split("||")[2])

    return (billable, status, code)

In [0]:
# returns relation pair dataframe

def get_relations_df (results, col='relations'):
    rel_pairs=[]
    for rel in results[0][col]:
        rel_pairs.append(( 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
        ))

    rel_df = pd.DataFrame(rel_pairs, columns=['entity1','entity1_begin','entity1_end','chunk1',
                                              'entity2','entity2_end','entity2_end','chunk2', 
                                              'confidence'])
    
    # limit df columns to get entity and chunks with results only
    rel_df = rel_df.iloc[:,[0,3,4,7,8]]
    
    return rel_df

### RxNorm Resolver

- RxNorm is a second vocabulary for prescription drugs. RxNorm provides a set of codes for clinical drugs, which are the combination of active ingredients, dose form, and strength of a drug. For example, the RxNorm code for ciprofloxacin 500 mg 24-hour extended-release tablet (the generic name for Cipro XR 500 mg) is RX10359383, regardless of brand or packaging.

- The goal of RxNorm is to allow computer systems to communicate drug-related information efficiently and unambiguously. Produced by the National Library of Medicine (NLM), RxNorm is available for distribution in both Metathesaurus Relation (MR) and Rich Release Format (RRF) tables. Currently there are no RxNorm names available for drugs with more than four active ingredients, those that are sold over the counter (OTC) or those that are international, due to the lack of appropriate information available about such drugs.


<img src="https://www.nlm.nih.gov/research/umls/rxnorm/RxNorm_Drug_Relationships.png" width="750" alt="RxNorm Overview">

**Pretrained Models**

- `sbiobertresolve_rxnorm`
- `demo_sbiobertresolve_rxnorm`
- `sbiobertresolve_rxnorm_dispo`
- `sbiobertresolve_rxnorm_disposition`
- `sbertresolve_rxnorm_disposition`
- `sbiobertresolve_rxnorm_ndc`
- `sbiobertresolve_rxnorm_augmented`
- `sbiobertresolve_rxnorm_augmented_re`

Now, we will create a pipeline with `sbiobertresolve_rxnorm_augmented` pretrained model and see the results.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sentence_embeddings")\
      .setCaseSensitive(False)

rxnorm_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = nlp.PipelineModel(
      stages = [
          documentAssembler,
          sbert_embedder,
          rxnorm_resolver])

rxnorm_lp = nlp.LightPipeline(rxnorm_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
setInputCols in ENTITY_6ce17d64b14a expecting 1 columns. Provided column amount: 2. Which should be columns from the following annotators: ['sentence_embeddings']

In [0]:
text = 'metformin 100 mg'

%time get_codes (rxnorm_lp, text, col_name = 'ner_chunk', vocab='rxnorm_code')

CPU times: user 72.9 ms, sys: 1.06 s, total: 1.13 s
Wall time: 34.7 s
Out[100]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,metformin 100 mg,0,15,2200518,"[2200518, 861024, 141916, 334738, 332848, 104494, 199955, 333262, 439563, 429178, 103910, 429408, 450523, 1744000, 401938, 402346, 1726496, 485246, 198758, 316350, 213092]","[metformin hydrochloride 100 MG/ML Extended Release Suspension, metformin hydrochloride 100 MG/ML, fenofibrate 100 MG Oral Capsule, fenofibrate 100 MG, ciprofibrate 100 MG, ciprofibrate 100 MG Oral Tablet, rutin 100 MG Oral Tablet, rutin 100 MG, fendiline 100 MG, fendiline 100 MG Oral Tablet, carbamazepine 100 MG Oral Tablet [Epimaz], perazine 100 MG Oral Tablet, perazine 100 MG, emtricitabine 100 MG, miglustat 100 MG Oral Capsule, miglustat 100 MG, azacitidine 100 MG, azacitidine 100 MG Injection, niacin 100 MG Oral Capsule, niacin 100 MG, dolasetron 100 MG Oral Tablet [Anzemet]]","[0.0658, 0.0658, 0.0684, 0.0684, 0.0697, 0.0697, 0.0765, 0.0765, 0.0775, 0.0775, 0.0780, 0.0804, 0.0804, 0.0818, 0.0811, 0.0811, 0.0812, 0.0812, 0.0811, 0.0811, 0.0836]"


### RxNorm with DrugNormalizer

We have `DrugNormalizer` annotator that transforms text to the format used in the RxNorm and SNOMED standards.

It has **`policy`** parameter to remove patterns from text. Valid policy values are:
- "all",
- "abbreviations",
- "dosages" <br/>

Default is "all". "abbreviation" policy used to expend common drugs abbreviations, "dosages" policy used to convert drugs dosages and values to the standard form (see examples bellow).

For more details about the usage of `DrugNormalizer` annotator, please visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/23.Drug_Normalizer.ipynb)

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("ner_chunk_v0")

drug_normalizer = medical.DrugNormalizer() \
    .setInputCols("ner_chunk_v0") \
    .setOutputCol("ner_chunk") \
    .setPolicy('all')

rxnorm_pipelineModel2 = nlp.PipelineModel(
    stages = [
        documentAssembler,
        drug_normalizer,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp2 = nlp.LightPipeline(rxnorm_pipelineModel2)

In [0]:
text = 'aspirin 10 meq/ 5 ml oral sol'

%time get_codes (rxnorm_lp2, text, col_name = 'ner_chunk', vocab='rxnorm_code')

CPU times: user 21.6 ms, sys: 12 ms, total: 33.6 ms
Wall time: 1.23 s
Out[102]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,aspirin 2 meq/ml oral solution,0,29,688214,"[688214, 342917, 688213, 755937, 756028, 979446, 755332, 545588, 1091135, 246229, 104692, 755920, 261342, 756064, 756078, 2381144, 1115998, 1092456, 248847, 1094188, 250944, 108870, 995243, 421803]","[aspirin 2.5 MG/ML Oral Solution, aspirin 2.2 MG/ML, aspirin 2.5 MG/ML, periciazine 2 MG/ML Oral Solution, fenspiride 2 MG/ML Oral Solution, metaproterenol sulfate 2 MG/ML Oral Solution [Alupent], dicyclomine hydrochloride 2 MG/ML Oral Solution [Merbentyl], racepinephrine 2 MG/ML Oral Solution [Orostat], methylphenidate hydrochloride 2 MG/ML Oral Solution [Methylin], thioridazine 2 MG/ML Oral Solution, temazepam 2 MG/ML Oral Solution [Euhypnos], butamirate 2 MG/ML Oral Solution, citalopram 2 MG/ML Oral Solution [Celexa], oxeladin 2 MG/ML Oral Solution, pipazethate 2 MG/ML Oral Solution, fenfluramine 2.2 MG/ML Oral Solution [Fintepla], ephedrine sulfate 2.2 MG/ML Oral Solution, diphenhydramine hydrochloride 2.5 MG/ML Oral Solution [Diphenhist], homatropine 2 MG/ML Oral Solution, diphenhydramine hydrochloride 2.5 MG/ML Oral Solution [Diphedryl], piracetam 2 MG/ML Oral Solution, theophylline 2 MG/ML Oral Solution, hydroxyzine hydrochloride 2 MG/ML Oral Solution [Atarax], codeine 2 MG/ML Oral Suspension]","[0.0347, 0.0566, 0.0748, 0.0947, 0.1040, 0.1009, 0.1075, 0.1081, 0.1097, 0.1155, 0.1137, 0.1149, 0.1123, 0.1154, 0.1178, 0.1175, 0.1196, 0.1212, 0.1211, 0.1242, 0.1196, 0.1236, 0.1214, 0.1175]"


### Drug Spell Checker

We have pretrained spell checker model for drugs. Let's see how it works.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

spell = nlp.NorvigSweetingModel.pretrained("spellcheck_drug_norvig", "en", "clinical/models")\
    .setInputCols("token")\
    .setOutputCol("corrected_token")\

spell_pipeline = nlp.Pipeline(stages = [documentAssembler,
                              tokenizer,
                              spell])

spell_model = spell_pipeline.fit(spark.createDataFrame([['']]).toDF('text'))

spell_lp = nlp.LightPipeline(spell_model)
corrected = spell_lp.annotate("You have to take Neutrcare and colfosinum and a bit of Fluorometholne & Ribotril")

spellcheck_drug_norvig download started this may take some time.
Approximate size to download 4.3 MB
[ | ][ / ][OK!]

In [0]:
print(" ".join(corrected['token']))
print(" ".join(corrected['corrected_token']))

You have to take Neutrcare and colfosinum and a bit of Fluorometholne & Ribotril
You have to take Neutracare and colfosinum and a bit of Fluorometholone & Rivotril

### ICD-10-CM Resolver

**ICD10 BACKGROUND INFO**

**ICD-10-CM vs. ICD-10-PCS**

- With the transition to ICD-10, in the United States, ICD-9 codes are segmented into ICD-10-CM and ICD-10-PCS codes. **The "CM" in ICD-10-CM codes stands for clinical modification**; ICD-10-CM codes were developed by the Centers for Disease Control and Prevention in conjunction with the National Center for Health Statistics (NCHS), for outpatient medical coding and reporting in the United States, as published by the World Health Organization (WHO).

- **The "PCS" in ICD-10-PCS codes stands for the procedural classification system**. ICD-10-PCS is a completely separate medical coding system from ICD-10-CM, containing an additional 87,000 codes for use ONLY in United States inpatient, hospital settings. The procedure classification system (ICD-10-PCS) was developed by the Centers for Medicare and Medicaid Services (CMS) in conjunction with 3M Health Information Management (HIM).

- ICD-10-CM codes add increased specificity to their ICD-9 predecessors, growing to five times the number of codes as the present system; a total of 68,000 clinical modification diagnosis codes. ICD-10-CM codes provide the ability to track and reveal more information about the quality of healthcare, allowing healthcare providers to better understand medical complications, better design treatment and care, and better comprehend and determine the outcome of care.

- ICD-10-PCS is used only for inpatient, hospital settings in the United States, and is meant to replace volume 3 of ICD-9 for facility reporting of inpatient procedures. Due to the rapid and constant state of flux in medical procedures and technology, ICD-10-PCS was developed to accommodate the changing landscape. Common procedures, lab tests, and educational sessions that are not unique to the inpatient, hospital setting have been omitted from ICD-10-PCS.

- ICD-10 is confusing enough when you’re trying to digest the differences between ICD-9 and ICD-10, but there are also different types of ICD-10 codes that providers should be aware of.


**Primary difference between ICD-10-CM and ICD-10-PCS**

- When most people talk about ICD-10, they are referring to ICD-10CM. This is the code set for diagnosis coding and is used for all healthcare settings in the United States. ICD-10PCS, on the other hand, is used in hospital inpatient settings for inpatient procedure coding.

**ICD-10-CM breakdown**

- Approximately 68,000 codes
- 3–7 alphanumeric characters
- Facilitates timely processing of claims


**ICD-10-PCS breakdown**

- Will replace ICD-9-CM for hospital inpatient use only.
- ICD-10-PCS will not replace CPT codes used by physicians. According to HealthCare Information Management, Inc. (HCIM), “Its only intention is to identify inpatient facility services in a way not directly related to physician work, but directed towards allocation of hospital services.”
- 7 alphanumeric characters

- ICD-10-PCS is very different from ICD-9-CM procedure coding due to its ability to be more specific and accurate. “This becomes increasingly important when assessing and tracking the quality of medical processes and outcomes, and compiling statistics that are valuable tools for research,” according to HCIM.

**Hierarchical Condition Category (HCC)**

- Hierarchical condition category (HCC) coding is a risk-adjustment model originally designed to estimate future health care costs for patients. The Centers for Medicare & Medicaid Services (CMS) HCC model was initiated in 2004 but is becoming increasingly prevalent as the environment shifts to value-based payment models.

- Hierarchical condition category relies on ICD-10 coding to assign risk scores to patients. Each HCC is mapped to an ICD-10 code. Along with demographic factors (such as age and gender), insurance companies use HCC coding to assign patients a risk adjustment factor (RAF) score. Using algorithms, insurances can use a patient’s RAF score to predict costs. For example, a patient with few serious health conditions could be expected to have average medical costs for a given time. However, a patient with multiple chronic conditions would be expected to have higher health care utilization and costs.



**Pretrained ICD-10 Models**

- `sbiobertresolve_icd10cm_augmented`
- `sbiobertresolve_icd10pcs`
- `sbiobertresolve_icd10cm_augmented_billable_hcc`
- `sbiobertresolve_icd10cm`
- `sbiobertresolve_icd10cm_slim_normalized`
- `sbiobertresolve_icd10cm_slim_billable_hcc`
- `sbertresolve_icd10cm_slim_billable_hcc_med`
- `sbiobertresolve_icd10cm_generalised`

Let's create an ICD10 pipeline with `sbiobertresolve_icd10cm_augmented_billable_hcc` model and see how it works

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector DL annotator, processes various sentences per line
sentenceDetectorDL = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare", "en", 'clinical/models') \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# WordEmbeddingsModel pretrained "embeddings_clinical" includes a model of 1.7Gb that needs to be downloaded
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("word_embeddings")

# Named Entity Recognition for clinical concepts.
clinical_ner = medical.NerModel().pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "word_embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_chunk = medical.NerConverterInternal()\
    .setInputCols("sentence","token","clinical_ner")\
    .setOutputCol("clinical_ner_chunk")\
    .setWhiteList(["PROBLEM"])

c2doc = nlp.Chunk2Doc()\
    .setInputCols("clinical_ner_chunk")\
    .setOutputCol("ner_chunk_doc")

sbiobert_embedder = nlp.BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
    .setInputCols(["ner_chunk_doc"])\
    .setOutputCol("sentence_embeddings")\
    .setCaseSensitive(False)

icd_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
    .setInputCols(["clinical_ner_chunk", "sentence_embeddings"]) \
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")


# Build up the pipeline
resolver_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        sentenceDetectorDL,
        tokenizer,
        word_embeddings,
        clinical_ner,
        clinical_ner_chunk,
        c2doc,
        sbiobert_embedder,
        icd_resolver
  ])

icd_model = resolver_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
[ | ][OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
setInputCols in ENTITY_05f90ef679b8 expecting 1 columns. Provided column amount: 2. Which should be columns from the following annotators: ['sentence_embeddings']

In [0]:
#sample clinical note
clinical_note = (
    'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years '
    'prior to presentation and subsequent type two diabetes mellitus, associated '
    'with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, '
    'presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. '
    'Two weeks prior to presentation, she was treated with a five-day course of amoxicillin '
    'for a respiratory tract infection. She had been on dapagliflozin for six months '
    'at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; '
    'significantly, her abdominal examination was benign with no tenderness or guarding. Pertinent '
    'laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, '
    'creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) '
    '10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed '
    'as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for '
    'starvation ketosis, as she reported poor oral intake for three days prior to admission.')

data_ner = spark.createDataFrame([[clinical_note]]).toDF("text")

In [0]:
icd10_result = icd_model.transform(data_ner)

In [0]:
%%time
icd10_result.select("icd10cm_code.metadata").show(truncate=100)

+----------------------------------------------------------------------------------------------------+
 metadata|
+----------------------------------------------------------------------------------------------------+
[{chunk -> 0, all_k_results -> O24.4:::O24.41:::O24.43:::Z86.32:::Z87.5:::O24.31:::O24.11:::O24.1...|
+----------------------------------------------------------------------------------------------------+

CPU times: user 125 ms, sys: 53.5 ms, total: 178 ms
Wall time: 51 s

In [0]:

import pandas as pd

pd.set_option('display.max_colwidth', 0)

icd10_df = icd10_result.select(F.explode(F.arrays_zip(icd10_result.clinical_ner_chunk.result, 
                                                   icd10_result.clinical_ner_chunk.metadata, 
                                                   icd10_result.icd10cm_code.result, 
                                                   icd10_result.icd10cm_code.metadata)).alias("cols")) \
                            .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                     F.expr("cols['0']").alias("ner_chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()
 
 
codes = []
resolutions = []
hcc_all = []

for code, resolution, hcc in zip(icd10_df['all_codes'], icd10_df['resolutions'], icd10_df['hcc_list']):
    
    codes.append(code.split(':::'))
    resolutions.append(resolution.split(':::'))
    hcc_all.append(hcc.split(":::"))

icd10_df['all_codes'] = codes  
icd10_df['resolutions'] = resolutions
icd10_df['hcc_list'] = hcc_all

The values in billable, hcc_code and hcc_status columns are seperated by || and we will change them to a list.

In [0]:
icd10_df["billable"] = icd10_df["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,0]    
icd10_df["hcc_status"] = icd10_df["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,1]
icd10_df["hcc_score"] = icd10_df["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,2]

icd10_df.drop("hcc_list", axis=1, inplace= True)

In [0]:

icd10_df.head(15)

Out[111]:

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,billable,hcc_status,hcc_score
0,0,gestational diabetes mellitus,PROBLEM,O24.4,"[O24.4, O24.41, O24.43, Z86.32, Z87.5, O24.31, O24.11, O24.1, O24.81]","[gestational diabetes mellitus [gestational diabetes mellitus], gestational diabetes mellitus (disorder) [gestational diabetes mellitus in pregnancy], postpartum gestational diabetes mellitus [gestational diabetes mellitus in the puerperium], history of gestational diabetes mellitus [personal history of gestational diabetes], history of gestational diabetes mellitus (situation) [personal history of complications of pregnancy, childbirth and the puerperium], pre-existing diabetes mellitus in pregnancy [unspecified pre-existing diabetes mellitus in pregnancy], pre-existing diabetes mellitus in pregnancy (disorder) [pre-existing type 2 diabetes mellitus, in pregnancy], pre-existing type 2 diabetes mellitus in pregnancy [pre-existing type 2 diabetes mellitus, in pregnancy, childbirth and the puerperium], other pre-existing diabetes mellitus in pregnancy [other pre-existing diabetes mellitus in pregnancy]]","[0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O24.11,"[O24.11, E11.8, E11, E13.9, E11.9, E11.3, E11.44, Z86.3, Z86.39, E11.32, E11.43, E11.5]","[pre-existing type 2 diabetes mellitus [pre-existing type 2 diabetes mellitus, in pregnancy], disorder associated with type 2 diabetes mellitus [type 2 diabetes mellitus with unspecified complications], type 2 diabetes mellitus [type 2 diabetes mellitus], secondary diabetes mellitus [other specified diabetes mellitus without complications], diabetes mellitus type 2 [type 2 diabetes mellitus without complications], disorder of eye due to type 2 diabetes mellitus [type 2 diabetes mellitus with ophthalmic complications], disorder of nervous system due to type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic amyotrophy], history of diabetes mellitus type 2 (situation) [personal history of endocrine, nutritional and metabolic diseases], history of diabetes mellitus type 2 [personal history of other endocrine, nutritional and metabolic disease], secondary endocrine diabetes mellitus [type 2 diabetes mellitus with mild nonproliferative diabetic retinopathy], neurological disorder with type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic autonomic (poly)neuropathy], peripheral circulatory disorder due to type 2 diabetes mellitus [type 2 diabetes mellitus with circulatory complications]]","[0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]","[0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0]","[0, 18, 0, 19, 19, 0, 18, 0, 0, 0, 18, 0]"
2,0,an acute hepatitis,PROBLEM,B15,"[B15, K72.0, B17.9, B17.2, Z03.89, B15.9, K75.2, K71.2, B19.9, B16, K70.1, B16.9]","[acute hepatitis a [acute hepatitis a], acute hepatitis [acute and subacute hepatic failure], acute infectious hepatitis [acute viral hepatitis, unspecified], acute hepatitis e [acute hepatitis e], acute infectious hepatitis suspected [encounter for observation for other suspected diseases and conditions ruled out], acute type a viral hepatitis [hepatitis a without hepatic coma], acute focal hepatitis [nonspecific reactive hepatitis], toxic liver disease with acute hepatitis [toxic liver disease with acute hepatitis], fulminant hepatitis [unspecified viral hepatitis without hepatic coma], acute hepatitis b [acute hepatitis b], acute alcoholic hepatitis [alcoholic hepatitis], acute fulminating viral hepatitis [acute hepatitis b without delta-agent and without hepatic coma]]","[0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,0,obesity,PROBLEM,E66.9,"[E66.9, E66.8, Z68.41, Q13.0, E66, E66.01, Z86.39, E34.9, H35.50, Z83.49, Q55.62, E66.3]","[obesity [obesity, unspecified], abdominal obesity [other obesity], obese [body mass index [bmi] 40.0-44.9, adult], central obesity [coloboma of iris], 

### Entity Resolver Visualizer

Let's apply `EntityResolverVisualizer`  by using `sparknlp_display` library to see the results of the pipeline in a nicer layout:

Let's apply EntityResolverVisualizer by using sparknlp_display library to see the results of the pipeline in a nicer layout:

In [0]:
from sparknlp_display import EntityResolverVisualizer

icd10_lp = nlp.LightPipeline(icd_model)

light_result = icd10_lp.fullAnnotate(clinical_note)

visualiser = EntityResolverVisualizer()

# Change color of an entity label
visualiser.set_label_colors({'PROBLEM':'#008080'})

vis= visualiser.display(light_result[0], 'clinical_ner_chunk', 'icd10cm_code', return_html=True)

displayHTML(vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM O24.4 gestational diabetes mellitus [gestational diabetes mellitus] diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM O24.11 pre-existing type 2 diabetes mellitus [pre-existing type 2 diabetes mellitus, in pregnancy] , associated with an acute hepatitis PROBLEM B15 acute hepatitis a [acute hepatitis a] , and obesity PROBLEM E66.9 obesity [obesity, unspecified] with a body mass index PROBLEM Z68.41 finding of body mass index [body mass index [bmi] 40.0-44.9, adult] (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria PROBLEM R35 polyuria [polyuria] , polydipsia PROBLEM R63.1 polydipsia [polydipsia] , poor appetite PROBLEM R63.0 poor appetite [anorexia] , and vomiting PROBLEM R11.1 vomiting [vomiting] . Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection PROBLEM J98.8 respiratory tract infection [other specified respiratory disorders] . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa PROBLEM K13.0 dry lips [diseases of lips] ; significantly, her abdominal examination was benign with no tenderness PROBLEM R52 tenderness (finding) [pain, unspecified] or guarding PROBLEM M62.83 guarding [muscle spasm] . Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing PROBLEM O04.6 hemolysis [delayed or excessive hemorrhage following (induced) termination of pregnancy] due to significant lipemia PROBLEM E78.5 lipidaemia [hyperlipidemia, unspecified] . The patient was initially admitted for starvation ketosis PROBLEM E87.2 starvation ketoacidosis [acidosis] , as she reported poor oral intake PROBLEM E63.9 inadequate oral intake [nutritional deficiency, unspecified] for three days prior to admission.

### CPT Resolver

The Current Procedural Terminology (CPT) code set is a medical code set maintained by the American Medical Association. The CPT code set describes medical, surgical, and diagnostic services and is designed to communicate uniform information about medical services and procedures among physicians, coders, patients, accreditation organizations, and payers for administrative, financial, and analytical purposes.

**Pretrained Models**

- `sbiobertresolve_cpt`
- `sbiobertresolve_cpt_procedures_augmented`
- `sbiobertresolve_cpt_augmented`
- `sbiobertresolve_cpt_procedures_measurements_augmented`

**We will create a pipeline to detect bodyparts and imaging tests entities and their relations. Then we will use these related chunks in CPT resolver model to get their CPT code.**

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetectorDL = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare", "en", 'clinical/models') \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("word_embeddings")

pos_tagger = nlp.PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

dependency_parser = nlp.DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

# Named Entity Recognition for radiology reports.
radiology_ner = medical.NerModel.pretrained("jsl_rd_ner_wip_greedy_clinical", "en", "clinical/models") \
   .setInputCols(["sentence", "token", "word_embeddings"]) \
   .setOutputCol("ner")

radiology_ner_chunker = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

re_model = medical.RelationExtractionModel()\
    .pretrained("re_bodypart_directions", "en", 'clinical/models')\
    .setInputCols(["word_embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(["imagingtest-bodypart", "bodypart-imagingtest"])\
    .setMaxSyntacticDistance(4)\
    .setPredictionThreshold(0.5)


# Build up the pipeline
rd_relation_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        sentenceDetectorDL,
        tokenizer,
        word_embeddings,
        pos_tagger,
        dependency_parser,
        radiology_ner,
        radiology_ner_chunker,
        re_model
        ])


rd_rel_model = rd_relation_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
jsl_rd_ner_wip_greedy_clinical download started this may take some time.
[ | ][ / ][ — ][OK!]
re_bodypart_directions download started this may take some time.
[ | ][ / ][OK!]

In [0]:
light_rel_model = nlp.LightPipeline(rd_rel_model)

In [0]:
text="Left shin pain. I have ordered x-ray of the left fibula and knee today. The patient will return to the clinic in 3 weeks. He is to call me in the interim for any problems."
light_result = light_rel_model.fullAnnotate(text)

get_relations_df(light_result)

Out[116]:

,entity1,chunk1,entity2,chunk2,confidence
0,ImagingTest,x-ray,BodyPart,fibula,1.0
1,ImagingTest,x-ray,BodyPart,knee,0.9458425


Now we can get CPT codes of these related chunks.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("clinical_ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
    .setInputCols(["clinical_ner_chunk"])\
    .setOutputCol("sentence_embeddings")\
    .setCaseSensitive(False)

cpt_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_augmented","en", "clinical/models") \
    .setInputCols(["clinical_ner_chunk", "sentence_embeddings"]) \
    .setOutputCol("cpt_code")\
    .setDistanceFunction("EUCLIDEAN")


cpt_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        cpt_resolver])


cpt_lp = nlp.LightPipeline(cpt_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_cpt_augmented download started this may take some time.
[ | ][ / ][ — ][ \ ][OK!]
setInputCols in ENTITY_f87c226549b5 expecting 1 columns. Provided column amount: 2. Which should be columns from the following annotators: ['sentence_embeddings']

In [0]:
text = 'fibula x-ray'

%time get_codes (cpt_lp, text, col_name='clinical_ner_chunk', vocab='cpt_code')

CPU times: user 74.9 ms, sys: 479 ms, total: 554 ms
Wall time: 19 s
Out[118]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,fibula x-ray,0,11,73590,"[73590, 27676, 27455, 27712, 27457, 27707, 73550, 73090, 27635]","[Tibia and fibula X-ray, Incision of fibula, Incision of fibula, Incision of fibula, Incision of fibula, Incision of fibula, Femur X-ray (procedure), Forearm X-ray, Excision of lesion of fibula (procedure)]","[0.0895, 0.0970, 0.0970, 0.0970, 0.0970, 0.0970, 0.1360, 0.1364, 0.1474]"


In [0]:
text = 'knee x-ray'

%time get_codes (cpt_lp, text, col_name='clinical_ner_chunk', vocab='cpt_code')

CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 245 ms
Out[119]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,knee x-ray,0,9,73564,"[73564, 73562, 73580, 27486, 1010487, 27540, 27332, 27310, 27334, 27335, 27330, 73600, 73610, 1005006, 27447, 27446]","[Knee X-ray, Knee X-ray, Knee arthrogram, Knee joint operations, Radiologic examination, knee, Operative procedure on knee, Arthrotomy of knee (procedure), Arthrotomy of knee (procedure), Arthrotomy of knee (procedure), Arthrotomy of knee (procedure), Arthrotomy of knee (procedure), Ankle X-ray, Ankle X-ray, Arthrotomy, knee, Knee replacement, Knee replacement]","[0.0000, 0.0000, 0.0907, 0.1096, 0.1171, 0.1309, 0.1472, 0.1472, 0.1472, 0.1472, 0.1472, 0.1483, 0.1483, 0.1538, 0.1527, 0.1527]"


In [0]:
text="TECHNIQUE IN DETAIL: After informed consent was obtained from the patient and his mother, the chest was scanned with portable ultrasound."
light_result = light_rel_model.fullAnnotate(text)

get_relations_df(light_result)

Out[120]:

,entity1,chunk1,entity2,chunk2,confidence
0,BodyPart,chest,ImagingTest,portable ultrasound,0.9999802


In [0]:
text = 'chest portable ultrasound'

%time get_codes (cpt_lp, text, col_name='clinical_ner_chunk', vocab='cpt_code')

CPU times: user 1.12 ms, sys: 9.11 ms, total: 10.2 ms
Wall time: 226 ms
Out[121]:

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,chest portable ultrasound,0,24,1010771,"[1010771, 76512, 76510, 3320F, 76977, 76999, 76377, 45391, 76881, 31620, 3319F, 76882]","[Diagnostic Ultrasound Procedures of the Chest, A scan ultrasound, A scan ultrasound, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan, USS - Ultrasound scan]","[0.1269, 0.1563, 0.1563, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582, 0.1582]"


### BertSentenceChunkEmbeddings


**BertSentenceChunkEmbeddings**

- This annotator let users to aggregate sentence embeddings and ner chunk embeddings to get more specific and accurate resolution codes. It works by averaging context and chunk embeddings to get contextual information. Input to this annotator is the context (sentence) and ner chunks, while the output is embedding for each chunk that can be fed to the resolver model. The `setChunkWeight` parameter can be used to control the influence of surrounding context.

- For more information and examples of `BertSentenceChunkEmbeddings` annotator, you can check here:
[24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb)

Lets do the same process like we did with `BertSentenceEmbeddings` annotator and compare the results. We will create a new pipeline by using this annotator with SentenceEntityResolverModel.

In [0]:
#Get average sentence-chunk Bert embeddings
sentence_chunk_embeddings = medical.BertSentenceChunkEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence", "clinical_ner_chunk"])\
    .setOutputCol("sentence_embeddings")\
    .setCaseSensitive(False)\
    .setChunkWeight(0.5) #default : 0.5

icd_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")

resolver_pipeline_SCE = nlp.Pipeline(
    stages = [
        document_assembler,
        sentenceDetectorDL,
        tokenizer,
        word_embeddings,
        clinical_ner,
        clinical_ner_chunk,
        sentence_chunk_embeddings,
        icd_resolver
  ])


model_SCE = resolver_pipeline_SCE.fit(spark.createDataFrame([['']]).toDF("text"))

sbiobert_base_cased_mli download started this may take some time.
[ | ][ / ][OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[ | ][OK!]

In [0]:
model_SCE.stages

Out[123]: [DocumentAssembler_c6fd97fcbd8f,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_85879c8bcc42,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_cd5ce67b529f,
 NER_CONVERTER_b136d8d9ff76,
 BERTSENTENCECHUNKEMBEDDINGS_5dd3578f2a54,
 ENTITY_05f90ef679b8]

In [0]:
icd10_result_SCE = model_SCE.transform(data_ner)

In [0]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

res_SCE_pd = icd10_result_SCE.select(F.explode(F.arrays_zip(icd10_result_SCE.clinical_ner_chunk.result, 
                                                   icd10_result_SCE.clinical_ner_chunk.metadata, 
                                                   icd10_result_SCE.icd10cm_code.result, 
                                                   icd10_result_SCE.icd10cm_code.metadata)).alias("cols")) \
                            .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                     F.expr("cols['0']").alias("ner_chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()
 
 
codes = []
resolutions = []
hcc_all = []

for code, resolution, hcc in zip(res_SCE_pd['all_codes'], res_SCE_pd['resolutions'], res_SCE_pd['hcc_list']):
    
    codes.append(code.split(':::'))
    resolutions.append(resolution.split(':::'))
    hcc_all.append(hcc.split(":::"))

res_SCE_pd['all_codes'] = codes  
res_SCE_pd['resolutions'] = resolutions
res_SCE_pd['hcc_list'] = hcc_all

In [0]:
res_SCE_pd["billable"] = res_SCE_pd["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,0]    
res_SCE_pd["hcc_status"] = res_SCE_pd["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,1]
res_SCE_pd["hcc_score"] = res_SCE_pd["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,2]

res_SCE_pd.drop("hcc_list", axis=1, inplace= True)

In [0]:
res_SCE_pd.head(15)

Out[127]:

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,billable,hcc_status,hcc_score
0,0,gestational diabetes mellitus,PROBLEM,O24.4,"[O24.4, O24.43, O24.41, O24.434, O24.42, O24.414, O24.13]","[gestational diabetes mellitus, class d [gestational diabetes mellitus], gestational diabetes mellitus in the puerperium [gestational diabetes mellitus in the puerperium], maternal gestational diabetes mellitus [gestational diabetes mellitus in pregnancy], gestational diabetes mellitus in the puerperium, insulin controlled [gestational diabetes mellitus in the puerperium, insulin controlled], gestational diabetes mellitus in childbirth [gestational diabetes mellitus in childbirth], gestational diabetes mellitus in pregnancy, insulin controlled [gestational diabetes mellitus in pregnancy, insulin controlled], pre-existing type 2 diabetes mellitus, in the puerperium [pre-existing type 2 diabetes mellitus, in the puerperium]]","[0, 0, 0, 1, 0, 1, 1]","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O24.13,"[O24.13, E11.6, E11.43, O24.11, E11.10, E11.0, O24.12, E11.4, E11.69, E11.1, O24.1, K31.8, E11.33, O24.4, E11.2]","[pre-existing type 2 diabetes mellitus, in the puerperium [pre-existing type 2 diabetes mellitus, in the puerperium], gastroparesis co-occurrent and due to type 2 diabetes mellitus [type 2 diabetes mellitus with other specified complications], gastroparesis with type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic autonomic (poly)neuropathy], pregnancy and type 2 diabetes mellitus [pre-existing type 2 diabetes mellitus, in pregnancy], ketoacidosis due to type 2 diabetes mellitus [type 2 diabetes mellitus with ketoacidosis without coma], hyperosmolar coma due to type 2 diabetes mellitus [type 2 diabetes mellitus with hyperosmolarity], pre-existing type 2 diabetes mellitus, in childbirth [pre-existing type 2 diabetes mellitus, in childbirth], cranial nerve palsy co-occurrent and due to type 2 diabetes mellitus [type 2 diabetes mellitus with neurological complications], angina due to type 2 diabetes mellitus [type 2 diabetes mellitus with other specified complication], ketoacidosis due to type 2 diabetes mellitus (disorder) [type 2 diabetes mellitus with ketoacidosis], pre-existing type 2 diabetes mellitus in pregnancy [pre-existing type 2 diabetes mellitus, in pregnancy, childbirth and the puerperium], diabetic gastroparesis associated with type 2 diabetes mellitus [other specified diseases of stomach and duodenum], moderate nonproliferative retinopathy co-occurrent and due to type 2 diabetes mellitus [type 2 diabetes mellitus with moderate nonproliferative diabetic retinopathy], gestational diabetes mellitus, class a>2< [gestational diabetes mellitus], persistent microalbuminuria due to type 2 diabetes mellitus [type 2 diabetes mellitus with kidney complications]]","[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 18, 0, 17, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0]"
2,0,an acute hepatitis,PROBLEM,B16.9,"[B16.9, B15.9, B16.2, B17.9, B16.0, E80.21, K71.2, O26.6, B18.1, F10.98, B15.0]","[acute fulminating type b viral hepatitis (disorder) [acute hepatitis b without delta-agent and without hepatic coma], acute fulminating type a viral hepatitis (disorder) [hepatitis a without hepatic coma], hepatic coma due to acute hepatitis b [acute hepatitis b without delta-agent with hepatic coma], acute fulminating viral hepatitis (disorder) [acute viral hepatitis, unspecified], acute viral hepatitis b with hepatic coma [acute hepatitis b with delta-agent with hepatic coma], acute intermittent (hepatic) porphyria [acute intermittent (hepatic) porphyria], toxic liver disease with acute hepatitis (disorder) [toxic liver disease with acute hepatitis], acute fatty liver of pregnancy [liver and biliary tract disorders in pregnancy, childbirth and the puerperium], chronic viral hepatitis b with hepatic coma [chronic viral hepatitis b withou

In [0]:
sentence_df = icd10_result.select(F.explode(F.arrays_zip(icd10_result.sentence.metadata, icd10_result.sentence.result)).alias("cols")) \
                          .select(F.expr("cols['0']['sentence']").alias("sent_id"),
                                  F.expr("cols['1']").alias("sentence_all")).toPandas()

comparison_df = pd.merge(icd10_df.loc[:,'sent_id':'resolutions'],res_SCE_pd.loc[:,'sent_id':'resolutions'],
                         on=['sent_id',"ner_chunk", "entity"], how='inner')
comparison_df.columns=['sent_id','ner_chunk', 'entity', 'icd10_code', 'all_codes', 'resolutions', 'icd10_code_SCE', 'all_codes_SCE', 'resolutions_SCE']

comparison_df = pd.merge(sentence_df, comparison_df,on="sent_id").drop('sent_id', axis=1)
comparison_df.head(15)
     

Out[128]:

,sentence_all,ner_chunk,entity,icd10_code,all_codes,resolutions,icd10_code_SCE,all_codes_SCE,resolutions_SCE
0,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",gestational diabetes mellitus,PROBLEM,O24.4,"[O24.4, O24.41, O24.43, Z86.32, Z87.5, O24.31, O24.11, O24.1, O24.81]","[gestational diabetes mellitus [gestational diabetes mellitus], gestational diabetes mellitus (disorder) [gestational diabetes mellitus in pregnancy], postpartum gestational diabetes mellitus [gestational diabetes mellitus in the puerperium], history of gestational diabetes mellitus [personal history of gestational diabetes], history of gestational diabetes mellitus (situation) [personal history of complications of pregnancy, childbirth and the puerperium], pre-existing diabetes mellitus in pregnancy [unspecified pre-existing diabetes mellitus in pregnancy], pre-existing diabetes mellitus in pregnancy (disorder) [pre-existing type 2 diabetes mellitus, in pregnancy], pre-existing type 2 diabetes mellitus in pregnancy [pre-existing type 2 diabetes mellitus, in pregnancy, childbirth and the puerperium], other pre-existing diabetes mellitus in pregnancy [other pre-existing diabetes mellitus in pregnancy]]",O24.4,"[O24.4, O24.43, O24.41, O24.434, O24.42, O24.414, O24.13]","[gestational diabetes mellitus, class d [gestational diabetes mellitus], gestational diabetes mellitus in the puerperium [gestational diabetes mellitus in the puerperium], maternal gestational diabetes mellitus [gestational diabetes mellitus in pregnancy], gestational diabetes mellitus in the puerperium, insulin controlled [gestational diabetes mellitus in the puerperium, insulin controlled], gestational diabetes mellitus in childbirth [gestational diabetes mellitus in childbirth], gestational diabetes mellitus in pregnancy, insulin controlled [gestational diabetes mellitus in pregnancy, insulin controlled], pre-existing type 2 diabetes mellitus, in the puerperium [pre-existing type 2 diabetes mellitus, in the puerperium]]"
1,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",subsequent type two diabetes mellitus,PROBLEM,O24.11,"[O24.11, E11.8, E11, E13.9, E11.9, E11.3, E11.44, Z86.3, Z86.39, E11.32, E11.43, E11.5]","[pre-existing type 2 diabetes mellitus [pre-existing type 2 diabetes mellitus, in pregnancy], disorder associated with type 2 diabetes mellitus [type 2 diabetes mellitus with unspecified complications], type 2 diabetes mellitus [type 2 diabetes mellitus], secondary diabetes mellitus [other specified diabetes mellitus without complications], diabetes mellitus type 2 [type 2 diabetes mellitus without complications], disorder of eye due to type 2 diabetes mellitus [type 2 diabetes mellitus with ophthalmic complications], disorder of nervous system due to type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic amyotrophy], history of diabetes mellitus type 2 (situation) [personal history of endocrine, nutritional and metabolic diseases], history of diabetes mellitus type 2 [personal history of other endocrine, nutritional and metabolic disease], secondary endocrine diabetes mellitus [type 2 diabetes mellitus with mild nonproliferative diabetic retinopathy], neurological disorder with type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic autonomic (poly)neuropathy], peripheral circulatory disorder due to type 2 diabetes mellitus [type 2 diabetes mellitus with circulatory complications]]",O24.13,"[O2

### Router - Using Resolver Models Together

- Normally, when we need to use more than one sentence entity resolver models in the same pipeline, we used to hit `BertSentenceEmbeddings` annotator more than once given the number of different resolver models in the same pipeline. Now we are introducing a solution with the help of `Router` annotator that could allow us to feed all the NER chunks to `BertSentenceEmbeddings` at once and then route the output of Sentence Embeddings to different resolver models needed.

- In this example, lets use `sbiobertresolve_rxnorm_augmented` RxNorm model and `sbiobertresolve_icd10cm_augmented_billable_hcc` ICD10CM-HCC models together. First we will get the `PROBLEM` entities form `ner_clinical` model and then get the `DRUG` entities from `ner_posology` model. Then we will merge them and use the `Chunk2Doc` annotator to create sentence chunks to populate Sentence Embeddings column. Then, we route the embeddings of `PROBLEM` entities to ICD10CM model and embeddings of `DRUG` entities to RxNorm model at the same time.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
    .setInputCols("document")\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token")\
    .setOutputCol("word_embeddings")

# to get PROBLEM entitis
clinical_ner = medical.NerModel().pretrained("ner_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "word_embeddings"]) \
    .setOutputCol("clinical_ner")

clinical_ner_chunk = medical.NerConverterInternal()\
    .setInputCols("sentence","token","clinical_ner")\
    .setOutputCol("clinical_ner_chunk")\
    .setWhiteList(["PROBLEM"])

# to get DRUG entities
posology_ner = medical.NerModel().pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "word_embeddings"]) \
    .setOutputCol("posology_ner")

posology_ner_chunk = medical.NerConverterInternal()\
    .setInputCols("sentence","token","posology_ner")\
    .setOutputCol("posology_ner_chunk")\
    .setWhiteList(["DRUG"])

# merge the chunks into a single ner_chunk
chunk_merger = medical.ChunkMergeApproach()\
    .setInputCols("clinical_ner_chunk","posology_ner_chunk")\
    .setOutputCol("final_ner_chunk")\
    .setMergeOverlapping(False)


# convert chunks to doc to get sentence embeddings of them
chunk2doc = nlp.Chunk2Doc()\
    .setInputCols("final_ner_chunk")\
    .setOutputCol("doc_final_chunk")


sbiobert_embeddings = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli","en","clinical/models")\
    .setInputCols(["doc_final_chunk"])\
    .setOutputCol("sbert_embeddings")\
    .setCaseSensitive(False)

# filter PROBLEM entity embeddings
router_sentence_icd10 = medical.Router() \
    .setInputCols("sbert_embeddings") \
    .setFilterFieldsElements(["PROBLEM"]) \
    .setOutputCol("problem_embeddings")

# filter DRUG entity embeddings
router_sentence_rxnorm = medical.Router() \
    .setInputCols("sbert_embeddings") \
    .setFilterFieldsElements(["DRUG"]) \
    .setOutputCol("drug_embeddings")

# use problem_embeddings only
icd_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_slim_billable_hcc","en", "clinical/models") \
    .setInputCols(["problem_embeddings"]) \
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")


# use drug_embeddings only
rxnorm_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
    .setInputCols(["drug_embeddings"]) \
    .setOutputCol("rxnorm_code")\
    .setDistanceFunction("EUCLIDEAN")


routerPipeline  = nlp.Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        clinical_ner_chunk,
        posology_ner,
        posology_ner_chunk,
        chunk_merger,
        chunk2doc,
        sbiobert_embeddings,
        router_sentence_icd10,
        router_sentence_rxnorm,
        icd_resolver,
        rxnorm_resolver
])

router_model = routerPipeline.fit(spark.createDataFrame([['']]).toDF("text"))

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
[ | ][OK!]
ner_posology download started this may take some time.
[ | ][OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_icd10cm_slim_billable_hcc download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[ | ][OK!]

In [0]:
#sample clinical note
clinical_note = """The patient is a 41-year-old Vietnamese female with a cough that started last week.
She has had right-sided chest pain radiating to her back with fever starting yesterday.
She has a history of pericarditis in May 2006 and developed cough with right-sided chest pain.
MEDICATIONS
1. Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and her INR was 2.3.
2. Amiodarone 100 mg p.o. daily.
"""

In [0]:
df = spark.createDataFrame([[clinical_note]]).toDF("text")

result = router_model.transform(df)

In [0]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

icd10_result = result.select(F.explode(F.arrays_zip(result.clinical_ner_chunk.result, 
                                                   result.clinical_ner_chunk.metadata, 
                                                   result.icd10cm_code.result, 
                                                   result.icd10cm_code.metadata)).alias("cols")) \
                            .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                     F.expr("cols['0']").alias("ner_chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()
 
 
codes = []
resolutions = []
hcc_all = []

for code, resolution, hcc in zip(icd10_result['all_codes'], icd10_result['resolutions'], icd10_result['hcc_list']):
    
    codes.append(code.split(':::'))
    resolutions.append(resolution.split(':::'))
    hcc_all.append(hcc.split(":::"))

icd10_result['all_codes'] = codes  
icd10_result['resolutions'] = resolutions
icd10_result['hcc_list'] = hcc_all

In [0]:
icd10_result["billable"] = icd10_result["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,0]    
icd10_result["hcc_status"] = icd10_result["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,1]
icd10_result["hcc_score"] = icd10_result["hcc_list"].apply(extract_billable).apply(pd.Series).iloc[:,2]

icd10_result.drop("hcc_list", axis=1, inplace= True)
     

In [0]:
icd10_result.head(15)

Out[134]:

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,billable,hcc_status,hcc_score
0,0,a cough,PROBLEM,R05,"[R05, R05.3, R05.1, A37, R05.2, R06.01, R06.7, R06.0, R06.2, R06, J38.4, R05.4, G44.83, R00.2, R06.00, R07.1, R07.0, R09.81, R12, S22.5, R49.0, R07, L29, R06.83, G47.51]","[cough [cough], chronic cough [chronic cough], acute cough [acute cough], whooping cough [whooping cough], subacute cough [subacute cough], orthopnea [orthopnea], sneezing [sneezing], dyspnea [dyspnea], wheezing [wheezing], abnormalities of breathing [abnormalities of breathing], edema of larynx [edema of larynx], cough syncope [cough syncope], primary cough headache [primary cough headache], palpitations [palpitations], dyspnea, unspecified [dyspnea, unspecified], chest pain on breathing [chest pain on breathing], pain in throat [pain in throat], nasal congestion [nasal congestion], heartburn [heartburn], flail chest [flail chest], dysphonia [dysphonia], pain in throat and chest [pain in throat and chest], pruritus [pruritus], snoring [snoring], confusional arousals [confusional arousals]]","[0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1,right-sided chest pain,PROBLEM,R10.11,"[R10.11, M79.621, M79.604, M79.601, M25.511, M79.631, R07.1, M79.661, M79.651, M25.551, M79.641, M25.531, G90.511, M79.644, H57.11, M25.541, R10.31, R07.8, R07.89, M25.521, M25.561, G90.521, R07.2, I83.811, M79.671]","[right upper quadrant pain [right upper quadrant pain], pain in right upper arm [pain in right upper arm], pain in right leg [pain in right leg], pain in right arm [pain in right arm], pain in right shoulder [pain in right shoulder], pain in right forearm [pain in right forearm], chest pain on breathing [chest pain on breathing], pain in right lower leg [pain in right lower leg], pain in right thigh [pain in right thigh], pain in right hip [pain in right hip], pain in right hand [pain in right hand], pain in right wrist [pain in right wrist], complex regional pain syndrome i of right upper limb [complex regional pain syndrome i of right upper limb], pain in right finger(s) [pain in right finger(s)], ocular pain, right eye [ocular pain, right eye], pain in joints of right hand [pain in joints of right hand], right lower quadrant pain [right lower quadrant pain], other chest pain [other chest pain], other chest pain [other chest pain], pain in right elbow [pain in right elbow], pain in right knee [pain in right knee], complex regional pain syndrome i of right lower limb [complex regional pain syndrome i of right lower limb], precordial pain [precordial pain], varicose veins of right lower extremity with pain [varicose veins of right lower extremity with pain], pain in right foot [pain in right foot]]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,1,fever,PROBLEM,A68,"[A68, A78, R50.2, M04.1, A96.2, A68.9, R11.0, T67.7, A79.0, A92.1, R50.82, R50.9, R51, R50.8, T67.6, R50.83, R60.1, R10.81, A68.0, R09.81, A92.4, L29, R10.817, A95, R23.1]","[relapsing fevers [relapsing fevers], q fever [q fever], drug induced fever [drug induced fever], periodic fever syndromes [periodic fever syndromes], lassa fever [lassa fever], relapsing fever, unspecified [relapsing fever, unspecified], nausea [nausea], heat edema [heat edema], trench fever [trench fever], o'nyong-nyong fever [o'nyong-nyong fever], postprocedural fever [postprocedural fever], fever, unspecified [fever, unspecified], headache [headache], other specified fever [other specified fever], heat fatigue, transient [heat fatigue, transient], postvaccination fever [postvaccination fever], generalized edema [generalized edema], abdominal tenderness

In [0]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

rxnorm_result = result.select(F.explode(F.arrays_zip(result.posology_ner_chunk.result, 
                                                   result.posology_ner_chunk.metadata, 
                                                   result.rxnorm_code.result, 
                                                   result.rxnorm_code.metadata)).alias("cols")) \
                            .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                     F.expr("cols['0']").alias("ner_chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()
 
 
codes = []
resolutions = []
hcc_all = []

for code, resolution, hcc in zip(rxnorm_result['all_codes'], rxnorm_result['resolutions'], rxnorm_result['hcc_list']):
    
    codes.append(code.split(':::'))
    resolutions.append(resolution.split(':::'))
    hcc_all.append(hcc.split(":::"))

rxnorm_result['all_codes'] = codes  
rxnorm_result['resolutions'] = resolutions
rxnorm_result['hcc_list'] = hcc_all

In [0]:
rxnorm_result.head()


Out[136]:

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions,hcc_list
0,4,Coumadin,DRUG,202421,"[202421, 374998, 2898, 1855075, 128793, 1598, 152085, 218274, 21732, 375868, 1435932, 227590, 202392, 374560, 87866, 1603522, 5011, 54908]","[Coumadin, coumarin Oral Tablet, coumarin, coumaran, Vicodin, dicumarol, Mycifradin, Medcodin, cresatin, cresatin Oral Solution, Reumacetin, Geocillin, Garamycin, ardeparin Injectable Solution, ardeparin, glabridin, gramicidin, policresulen]","[Brand Name, Clinical Drug Form, Ingredient, Ingredient, Brand Name, Ingredient, Brand Name, Brand Name, Ingredient, Clinical Drug Form, Brand Name, Brand Name, Brand Name, Clinical Drug Form, Ingredient, Ingredient, Ingredient, Ingredient]"
1,6,Amiodarone,DRUG,703,"[703, 1663223, 1151983, 1151982, 203114, 377132, 1151981, 727379, 370568, 620, 1944369, 151347, 722, 370575, 1000082, 1000084, 5936, 17767, 2184119, 1153467, 1364410, 151908]","[amiodarone, amiodarone Injection, amiodarone Pill, amiodarone Oral Product, amiodarone hydrochloride, amiodarone Injectable Solution, amiodarone Injectable Product, amiodarone Prefilled Syringe, amiodarone Oral Tablet, amantadine, amantadine Extended Release Oral Capsule, Amsidine, amoxapine, amoxapine Oral Tablet, alcaftadine, alcaftadine Ophthalmic Solution, iodipamide, amlodipine, amlodipine Oral Suspension, adenosine Injectable Product, ioxapine, Iopidine]","[Ingredient, Clinical Drug Form, Clinical Dose Group, Clinical Dose Group, Precise Ingredient, Clinical Drug Form, Clinical Dose Group, Clinical Drug Form, Clinical Drug Form, Ingredient, Clinical Drug Form, Brand Name, Ingredient, Clinical Drug Form, Ingredient, Clinical Drug Form, Ingredient, Ingredient, Clinical Drug Form, Clinical Dose Group, Ingredient, Brand Name]"


### Sentence Entity Resolvers with EntityChunkEmbedding

**Entity Chunk Embeddings** uses BERT Sentence embeddings to compute a weighted average vector represention of related entity chunks. This model is a subclass of `BertSentenceEmbeddings` and shares all parameters with it. It can load any pretrained `BertSentenceEmbeddings` model.

The input of the model consists of chunks of recognized named entities. One or more entities are selected as target entities and for each of them a list of related entities is specified (if empty, all other entities are assumed to be related).

 The model looks for chunks of the target entities and then tries to pair each target entity (e.g. DRUG) with other related entities (e.g. DOSAGE, STRENGTH, FORM, etc). The criterion for pairing a target entity with another related entity is that they appear in the same sentence and the **maximal syntactic distance** is below a predefined threshold.

The relationship between target and related entities is one-to-many, meaning that if there are multiple instances of the same target entity (e.g. DRUG) within a sentence, the model will map a related entity (e.g. DOSAGE) to at most one of the instances of the target entity.

For example, if there is a sentence; "The patient was given 125 mg of paracetamol and metformin", the model will pair "125 mg" to "paracetamol", but not to "metformin". The output of the model is an average embeddings of the chunks of each of the target entities and their related entities. It is possible to specify a particular weight for each entity type. An entity can be defined both as target a entity and as a related entity for some other target entity.

Now, let's see how `EntityChunkEmbeddings` works.

In [0]:
documenter = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = nlp.SentenceDetector() \
    .setInputCols("document") \
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer() \
    .setInputCols("sentence") \
    .setOutputCol("token")

embeddings = nlp.WordEmbeddingsModel() \
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

posology_ner_model = medical.NerModel()\
    .pretrained("ner_posology_large", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal()\
    .setInputCols("sentence", "token", "ner")\
    .setOutputCol("ner_chunk")

pos_tager = nlp.PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token")\
    .setOutputCol("pos_tag")

dependency_parser = nlp.DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tag", "token"])\
    .setOutputCol("dependencies")

entity_chunk_embeddings = medical.EntityChunkEmbeddings()\
    .pretrained("sbiobert_base_cased_mli","en","clinical/models")\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("drug_chunk_embeddings")\
    .setMaxSyntacticDistance(3)

entity_chunk_embeddings.setTargetEntities({"DRUG": ["STRENGTH", "ROUTE", "FORM"]})
entity_chunk_embeddings.setEntityWeights({"DRUG": 0.8, "STRENGTH": 0.2, "ROUTE": 0.2, "FORM": 0.2})

rxnorm_re = medical.SentenceEntityResolverModel\
    .pretrained("sbiobertresolve_rxnorm_augmented_re", "en","clinical/models")\
    .setInputCols(["drug_chunk_embeddings"])\
    .setOutputCol("rxnorm_code")\
    .setDistanceFunction("EUCLIDEAN")

rxnorm_pipeline_re = nlp.Pipeline(
    stages = [
        documenter,
        sentence_detector,
        tokenizer,
        embeddings,
        posology_ner_model,
        ner_converter,
        pos_tager,
        dependency_parser,
        entity_chunk_embeddings,
        rxnorm_re
        ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_posology_large download started this may take some time.
[ | ][ / ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
sbiobert_base_cased_mli download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
sbiobertresolve_rxnorm_augmented_re download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
import pandas as pd
sampleText = ["The patient was given metformin 500 mg tablet, 2.5 mg of coumadin and then ibuprofen.",
              "The patient was given metformin 400 mg, coumadin 5 mg, coumadin, amlodipine 10 MG tablet"]

sample_df = pd.DataFrame({'text':sampleText}).reset_index()

In [0]:
data_df = spark.createDataFrame(sample_df)

results = rxnorm_pipeline_re.fit(data_df).transform(data_df)

In [0]:
results = rxnorm_pipeline_re.fit(data_df).transform(data_df)

In [0]:
results.select("index", F.explode(F.arrays_zip(results.ner_chunk.result,
                                               results.ner_chunk.metadata)).alias("cols")) \
.select("index", F.expr("cols['0']").alias("chunk"),
                 F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----+----------+---------+
index|chunk |ner_label|
+-----+----------+---------+
0 |metformin |DRUG |
0 |500 mg |STRENGTH |
0 |tablet |FORM |
0 |2.5 mg |STRENGTH |
0 |coumadin |DRUG |
0 |ibuprofen |DRUG |
1 |metformin |DRUG |
1 |400 mg |STRENGTH |
1 |coumadin |DRUG |
1 |5 mg |STRENGTH |
1 |coumadin |DRUG |
1 |amlodipine|DRUG |
1 |10 MG |STRENGTH |
1 |tablet |FORM |
+-----+----------+---------+

**Merged chunks by internal relation extraction model feature**

- We specified the relations as following by `.setTargetEntities` parameter in the EntityChunkEmbeddings annotator :    
`.setTargetEntities({"DRUG": ["STRENGTH", "ROUTE", "FORM"]})`

- EntityChunkEmbeddings calculates those new chunks embeddings according to the weights specified in`.setEntityWeights` parameter.

`.setEntityWeights({"DRUG": 0.8, "STRENGTH": 0.2, "ROUTE": 0.2, "FORM": 0.2})`

In [0]:
results.select('index','drug_chunk_embeddings.result').show(truncate = False)

+-----+--------------------------------------------------------------------+
index|result |
+-----+--------------------------------------------------------------------+
0 |[metformin 500 mg tablet, 2.5 mg coumadin, ibuprofen] |
1 |[metformin 400 mg, coumadin 5 mg, coumadin, amlodipine 10 MG tablet]|
+-----+--------------------------------------------------------------------+

**RxNorm results**

In [0]:
results.select('index', F.explode(F.arrays_zip(results.drug_chunk_embeddings.result,
                                               results.rxnorm_code.result,
                                               results.rxnorm_code.metadata).alias("col")))\
        .select('index', F.expr("col['0']").alias("chunk"),
                         F.expr("col['1']").alias("rxnorm_code_"),
                         F.expr("col['2']['resolved_text']").alias("Concept_Name")).show(truncate = 50)

+-----+-----------------------+------------+---------------------------------+
index| chunk|rxnorm_code_| Concept_Name|
+-----+-----------------------+------------+---------------------------------+
 0|metformin 500 mg tablet| 860974| metformin hydrochloride 500 MG|
 0| 2.5 mg coumadin| 855313|warfarin sodium 2.5 MG [Coumadin]|
 0| ibuprofen| 1747293| ibuprofen Injection|
 1| metformin 400 mg| 332809| metformin 400 MG|
 1| coumadin 5 mg| 855333| warfarin sodium 5 MG [Coumadin]|
 1| coumadin| 202421| Coumadin|
 1|amlodipine 10 MG tablet| 308135| amlodipine 10 MG Oral Tablet|
+-----+-----------------------+------------+---------------------------------+

### Healthcare Codes Mapping by Using Pretrained Pipelines

|index|model|
|-----:|:-----|
| 1| [icd10_icd9_mapping](https://nlp.johnsnowlabs.com/2022/09/30/icd10_icd9_mapping_en.html)   |
| 2| [icdo_snomed_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icdo_snomed_mapping_en_3_0.html)   |
| 3| [icd10cm_snomed_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icd10cm_snomed_mapping_en_3_0.html)   |
| 4| [rxnorm_ndc_mapping](https://nlp.johnsnowlabs.com/2022/06/27/rxnorm_ndc_mapping_en_3_0.html)   |
| 5| [rxnorm_umls_mapping](https://nlp.johnsnowlabs.com/2022/06/27/rxnorm_umls_mapping_en_3_0.html)   |
| 6| [snomed_icd10cm_mapping](https://nlp.johnsnowlabs.com/2022/06/27/snomed_icd10cm_mapping_en_3_0.html)   |
| 7| [snomed_icdo_mapping](https://nlp.johnsnowlabs.com/2022/06/27/snomed_icdo_mapping_en_3_0.html)   |
| 8| [snomed_umls_mapping](https://nlp.johnsnowlabs.com/2022/06/27/snomed_umls_mapping_en_3_0.html)   |
| 9| [icd10cm_umls_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icd10cm_umls_mapping_en_3_0.html)   |
| 10| [mesh_umls_mapping](https://nlp.johnsnowlabs.com/2021/07/01/mesh_umls_mapping_en.html)   |
| 11| [rxnorm_mesh_mapping](https://nlp.johnsnowlabs.com/2021/07/01/rxnorm_mesh_mapping_en.html)   |


**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=Chunk+Mapping&edition=Spark+NLP+for+Healthcare)**


Lets show an example of ICD codes mapping to Snomed Codes to show how these pipelines work.

In [0]:
from sparknlp.pretrained import PretrainedPipeline

In [0]:
icd10_snomed_pipeline = nlp.PretrainedPipeline("icd10cm_snomed_mapping", "en", "clinical/models")

icd10cm_snomed_mapping download started this may take some time.
Approx size to download 1 MB
[ | ][ / ][OK!]

In [0]:
icd10_snomed_pipeline.annotate('M8950 E119 H16269')

Out[146]: {'document': ['M8950 E119 H16269'],
 'icd10cm_code': ['M8950', 'E119', 'H16269'],
 'snomed_code': ['716868003', '170771004', '51264003']}

|**ICD10CM Code** | **ICD10CM Details** | **SNOMED Code** | **SNOMED Details** |
| ---------- | -----------:| ---------- | -----------:|
| M8950 |  Osteolysis, unspecified site | 716868003 | Multicentric osteolysis nodulosis arthropathy spectrum |
| E119 | Type 2 diabetes mellitus | 170771004 | Diabetic - follow-up default |
| H16269 | Vernal keratoconjunctivitis, with limbar and corneal involvement, unspecified eye | 51264003 | Limbal AND/OR corneal involvement in vernal conjunctivitis |

|**ICD10CM Code** | **ICD10CM Details** | **SNOMED Code** | **SNOMED Details** |
| ---------- | -----------:| ---------- | -----------:|
| M8950 |  Osteolysis, unspecified site | 716868003 | Multicentric osteolysis nodulosis arthropathy spectrum |
| E119 | Type 2 diabetes mellitus | 170771004 | Diabetic - follow-up default |
| H16269 | Vernal keratoconjunctivitis, with limbar and corneal involvement, unspecified eye | 51264003 | Limbal AND/OR corneal involvement in vernal conjunctivitis |


Also, if you want to see more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.1.Healthcare_Code_Mapping.ipynb).



# 7.Chunk Mapping

We can use ChunkMapper to map entities with their associated code/reference based on pre-defined dictionaries.

There is an AnnotatorApproach of the ChunkMapper,  `ChunkMapperApproach`,  which can be used to train ChunkMapper models by giving a custom mapping dictionary. To use pretriained models, there is a `ChunkMapperModel` annotator.

The annotator also allows using **fuzzy matching**, which can take into consideration parts of the tokens that can map even when word order is different, char ngrams that can map even when there are typos, and using fuzzy distance metric (Jaccard, Levenshtein, etc.).

### Pretrained ChunkMapper Models

**<center>MAPPER MODELS**

|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [abbreviation_category_mapper](https://nlp.johnsnowlabs.com/2022/11/16/abbreviation_category_mapper_en.html)  | 2| [abbreviation_mapper](https://nlp.johnsnowlabs.com/2022/05/11/abbreviation_mapper_en_3_0.html)  | 3| [cvx_code_mapper](https://nlp.johnsnowlabs.com/2022/10/12/cvx_code_mapper_en.html)  |
| 4| [cvx_name_mapper](https://nlp.johnsnowlabs.com/2022/10/12/cvx_name_mapper_en.html)  | 5| [drug_action_treatment_mapper](https://nlp.johnsnowlabs.com/2022/03/31/drug_action_treatment_mapper_en_3_0.html)  | 6| [drug_ade_mapper](https://nlp.johnsnowlabs.com/2022/08/23/drug_ade_mapper_en.html)  |
| 7| [drug_brandname_ndc_mapper](https://nlp.johnsnowlabs.com/2022/05/11/drug_brandname_ndc_mapper_en_3_0.html)  | 8| [drug_category_mapper](https://nlp.johnsnowlabs.com/2022/12/18/drug_category_mapper_en.html)  | 9| [icd10_icd9_mapper](https://nlp.johnsnowlabs.com/2022/09/30/icd10_icd9_mapper_en.html)  |
| 10| [icd10cm_mapper](https://nlp.johnsnowlabs.com/2022/10/29/icd10cm_mapper_en.html)  | 11| [icd10cm_snomed_mapper](https://nlp.johnsnowlabs.com/2022/06/26/icd10cm_snomed_mapper_en_3_0.html)  | 12| [icd10cm_umls_mapper](https://nlp.johnsnowlabs.com/2022/06/26/icd10cm_umls_mapper_en_3_0.html)  |
| 13| [icd9_icd10_mapper](https://nlp.johnsnowlabs.com/2022/09/30/icd9_icd10_mapper_en.html)  | 14| [icd9_mapper](https://nlp.johnsnowlabs.com/2022/09/30/icd9_mapper_en.html)  | 15| [icdo_snomed_mapper](https://nlp.johnsnowlabs.com/2022/06/26/icdo_snomed_mapper_en_3_0.html)  |
| 16| [kegg_disease_mapper](https://nlp.johnsnowlabs.com/2022/11/18/kegg_disease_mapper_en.html)  | 17| [kegg_drug_mapper](https://nlp.johnsnowlabs.com/2022/11/21/kegg_drug_mapper_en.html)  | 18| [mesh_umls_mapper](https://nlp.johnsnowlabs.com/2022/06/26/mesh_umls_mapper_en_3_0.html)  |
| 19| [ndc_drug_brandname_mapper](https://nlp.johnsnowlabs.com/2023/02/22/ndc_drug_brandname_mapper_en.html)  | 20| [normalized_section_header_mapper](https://nlp.johnsnowlabs.com/2022/06/26/normalized_section_header_mapper_en_3_0.html)  | 21| [rxnorm_action_treatment_mapper](https://nlp.johnsnowlabs.com/2022/05/08/rxnorm_action_treatment_mapper_en_3_0.html)  |
| 22| [rxnorm_drug_brandname_mapper](https://nlp.johnsnowlabs.com/2023/02/09/rxnorm_drug_brandname_mapper_en.html)  | 23| [rxnorm_mapper](https://nlp.johnsnowlabs.com/2022/06/27/rxnorm_mapper_en_3_0.html)  | 24| [rxnorm_ndc_mapper](https://nlp.johnsnowlabs.com/2022/05/20/rxnorm_ndc_mapper_en_3_0.html)  |
| 25| [rxnorm_nih_mapper](https://nlp.johnsnowlabs.com/2023/02/23/rxnorm_nih_mapper_en.html)  | 26| [rxnorm_normalized_mapper](https://nlp.johnsnowlabs.com/2022/09/29/rxnorm_normalized_mapper_en.html)  | 27| [rxnorm_umls_mapper](https://nlp.johnsnowlabs.com/2022/06/26/rxnorm_umls_mapper_en_3_0.html)  |
| 28| [snomed_icd10cm_mapper](https://nlp.johnsnowlabs.com/2022/06/26/snomed_icd10cm_mapper_en_3_0.html)  | 29| [snomed_icdo_mapper](https://nlp.johnsnowlabs.com/2022/06/26/snomed_icdo_mapper_en_3_0.html)  | 30| [snomed_umls_mapper](https://nlp.johnsnowlabs.com/2022/06/27/snomed_umls_mapper_en_3_0.html)  |
| 31| [umls_clinical_drugs_mapper](https://nlp.johnsnowlabs.com/2022/07/06/umls_clinical_drugs_mapper_en_3_0.html)  | 32| [umls_clinical_findings_mapper](https://nlp.johnsnowlabs.com/2022/07/08/umls_clinical_findings_mapper_en_3_0.html)  | 33| [umls_disease_syndrome_mapper](https://nlp.johnsnowlabs.com/2022/07/11/umls_disease_syndrome_mapper_en_3_0.html)  |
| 34| [umls_drug_substance_mapper](https://nlp.johnsnowlabs.com/2022/07/11/umls_drug_substance_mapper_en_3_0.html)  | 35| [umls_major_concepts_mapper](https://nlp.johnsnowlabs.com/2022/07/11/umls_major_concepts_mapper_en_3_0.html)  | 36| []()|

**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=Chunk+Mapping&edition=Spark+NLP+for+Healthcare)**

<br>

**<center>PRETRAINED MAPPER PIPELINES**

|index|model|
|-----:|:-----|
| 1| [icd10_icd9_mapping](https://nlp.johnsnowlabs.com/2022/09/30/icd10_icd9_mapping_en.html)  |
| 2| [icd10cm_snomed_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icd10cm_snomed_mapping_en_3_0.html)  |
| 3| [icd10cm_umls_mapping](https://nlp.johnsnowlabs.com/2021/05/04/icd10cm_umls_mapping_en.html)  |
| 4| [icdo_snomed_mapping](https://nlp.johnsnowlabs.com/2022/06/27/icdo_snomed_mapping_en_3_0.html)  |
| 5| [mesh_umls_mapping](https://nlp.johnsnowlabs.com/2021/05/04/mesh_umls_mapping_en.html)  |
| 6| [rxnorm_mesh_mapping](https://nlp.johnsnowlabs.com/2021/05/04/rxnorm_mesh_mapping_en.html)  |
| 7| [rxnorm_ndc_mapping](https://nlp.johnsnowlabs.com/2022/06/27/rxnorm_ndc_mapping_en_3_0.html)  |
| 8| [rxnorm_umls_mapping](https://nlp.johnsnowlabs.com/2021/05/04/rxnorm_umls_mapping_en.html)  |
| 9| [snomed_icd10cm_mapping](https://nlp.johnsnowlabs.com/2021/05/02/snomed_icd10cm_mapping_en.html)  |
| 10| [snomed_icdo_mapping](https://nlp.johnsnowlabs.com/2022/06/27/snomed_icdo_mapping_en_3_0.html)  |
| 11| [snomed_umls_mapping](https://nlp.johnsnowlabs.com/2021/05/04/snomed_umls_mapping_en.html)  |



You can check [Healthcare Code Mapping Notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/11.1.Healthcare_Code_Mapping.ipynb) for the examples of pretrained mapper pipelines.

Let's create a piepline with `drug_action_treatment_mapper` and see how it works.

Pretrained `drug_action_treatment_mapper` model maps drugs with their corresponding `action` and `treatment` through `ChunkMapperModel()` annotator. <br/>

We will create a pipeline consisting `bert_token_classifier_drug_development_trials` ner model to extract ner chunk as well as `ChunkMapperModel()`.

In [0]:
#ChunkMapper Pipeline
document_assembler = nlp.DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = nlp.SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

btfc_ner =  medical.BertForTokenClassification.pretrained("bert_token_classifier_drug_development_trials", "en", "clinical/models")\
      .setInputCols("token","sentence")\
      .setOutputCol("btfc_ner")

btfc_ner_converter = nlp.NerConverter()\
      .setInputCols("sentence", "token", "btfc_ner")\
      .setOutputCol("btfc_ner_chunk")

#drug_action_treatment_mapper
chunkerMapper= medical.ChunkMapperModel().pretrained("drug_action_treatment_mapper", "en", "clinical/models")\
    .setInputCols(["btfc_ner_chunk"])\
    .setOutputCol("mappings")\
    .setRels(["action", "treatment"])


mapperPipeline = nlp.Pipeline().setStages([
    document_assembler,
    sentence_detector,
    tokenizer,
    btfc_ner,
    btfc_ner_converter,
    chunkerMapper])


test_data = spark.createDataFrame([["The patient was female and patient of Dr. X. and she was given Dermovate, Aspagin"]]).toDF("text")

res = mapperPipeline.fit(test_data).transform(test_data)

bert_token_classifier_drug_development_trials download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
drug_action_treatment_mapper download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
res.select(F.explode('btfc_ner_chunk.result').alias("chunks")).show(truncate=False)

+---------+
chunks |
+---------+
Dermovate|
Aspagin |
+---------+

In [0]:
res.selectExpr("action_mappings.metadata").show(truncate=False)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2903428274375586> in <module> 
 ----> 1 res . selectExpr ( "action_mappings.metadata" ) . show ( truncate = False ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in selectExpr (self, *expr) 
 1841 if len ( expr ) == 1 and isinstance ( expr [ 0 ] , list ) : 
 1842 expr = expr [ 0 ] 
 -> 1843 jdf = self . _jdf . selectExpr ( self . _jseq ( expr ) ) 
 1844 return DataFrame ( jdf , self . sql_ctx ) 
 1845 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Column 'action_mappings.metadata' does not exist. Did you mean one of the following? [mappings, btfc_ner, document, btfc_ner_chunk, sentence, text, token]; line 1 pos 0;
'Project ['action_mappings.metadata]
+- Project [text#10717, document#10724, sentence#10731, token#10738, btfc_ner#10745, btfc_ner_chunk#10753, UDF(array(btfc_ner_chunk#10753)) AS mappings#10762]
 +- Project [text#10717, document#10724, sentence#10731, token#10738, btfc_ner#10745, UDF(array(sentence#10731, token#10738, btfc_ner#10745)) AS btfc_ner_chunk#10753]
 +- Project [text#10717, document#10724, sentence#10731, token#10738, btfc_ner#10715 AS btfc_ner#10745]
 +- Project [text#10717, document#10724, sentence#10731, token#10714 AS token#10738, btfc_ner#10715]
 +- Project [text#10717, document#10724, sentence#10713 AS sentence#10731, token#10714, btfc_ner#10715]
 +- Project [text#10717, document#10712 AS document#10724, sentence#10713, token#10714, btfc_ner#10715]
 +- Project [text#10711 AS text#10717, document#10712, sentence#10713, token#10714, btfc_ner#10715]
 +- SerializeFromObject [if (assertnotnull(input[0, org.apache.spark.sql.Row, true]).isNullAt) null else staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, validateexternaltype(getexternalrowfield(assertnotnull(input[0, org.apache.spark.sql.Row, true]), 0, text), StringType, false), true, false, true) AS text#10711, if (assertnotnull(input[0, org.apache.spark.sql.Row, true]).isNullAt) null else mapobjects(lambdavariable(MapObject, ObjectType(class java.lang.Object), true, 32086), if (isnull(validateexternaltype(lambdavariable(MapObject, ObjectType(class java.lang.Object), true, 32086), StructField(annotatorType,StringType,true), StructField(begin,IntegerType,false), StructField(end,IntegerType,false), StructField(result,StringType,true), StructField(metadata,MapType(StringType,StringType,true),true), StructField(embeddings,ArrayType(FloatType,false),true), false))) null else named_struct(annotatorType, if (validateexternaltype(lambdavariable(MapObject, ObjectType(class java.lang.Object), true, 32086), StructField(annotatorType,StringType,true), StructField(begin,IntegerType,false), StructField(end,IntegerType,false), StructField(result,StringType,true), StructField(metadata,MapType(StringType,StringType,true),true), StructField(embeddings,ArrayType(FloatType,false),true), false).isNullAt) null else staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, validateexternaltype(getexternalrowfield(validateexternaltype(lambdavariable(MapObject, ObjectType(class java.lang.Object), true, 32086), StructField(annotatorType,StringType,true), StructField(begin,IntegerType,false), StructField(end,IntegerType,false), StructField(result,StringType,true), StructField(metadata,MapType(StringType,StringType,true),true), StructField(embeddings,ArrayType(FloatType,false),true), false), 0, an

In [0]:
res.select(F.explode(F.arrays_zip(res.btfc_ner_chunk.result, res.action_mappings.result, res.action_mappings.metadata)).alias("col"))\
   .select(F.expr("col['0']").alias("ner_chunk"),
           F.expr("col['1']").alias("pharmaceutical_action"),
           F.expr("col['2']['all_relations']").alias("other_pharmaceutical_actions")).show(truncate=False)
     

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2903428274375587> in <module> 
 ----> 1 res . select ( F . explode ( F . arrays_zip ( res . btfc_ner_chunk . result , res . action_mappings . result , res . action_mappings . metadata ) ) . alias ( "col" ) ) \ 
 2 .select(F.expr("col['0']").alias("ner_chunk"),
 3 F . expr ( "col['1']" ) . alias ( "pharmaceutical_action" ) , 
 4 F.expr("col['2']['all_relations']").alias("other_pharmaceutical_actions")).show(truncate=False)
 5 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1798 """
 1799 if name not in self . columns : 
 -> 1800 raise AttributeError(
 1801 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1802 jc = self . _jdf . apply ( name ) 

 AttributeError : 'DataFrame' object has no attribute 'action_mappings'

### Chunk Mapping with Fuzzy Distance Calculation

We will create a sample text with a typo in the "dermovae", then will feed the `drug_action_treatment_mapper` model with fuzzy matching feature to map **action** of each drug.

Also, we can modify the `setAllowMultiTokenChunk()` parameter to control matching of multi-token chunks. We will check that feature by observing the "Warfarina Lusa".

In [0]:
sample_text= "The patient was female and patient of Dr. X. and she was given dermovae, Warfarina Lusa"
test_data = spark.createDataFrame([[sample_text]]).toDF("text")


In [0]:
chunkerMapper_fuzzy = medical.ChunkMapperModel.pretrained("drug_action_treatment_mapper", "en", "clinical/models")\
    .setInputCols(["btfc_ner_chunk"])\
    .setOutputCol("mappings")\
    .setRels(["action"]) \
    .setAllowMultiTokenChunk(True) \
    .setEnableFuzzyMatching(True) \
    .setFuzzyMatchingDistanceThresholds(0.3)


mapperPipeline = nlp.Pipeline().setStages([
    document_assembler,
    sentence_detector,
    tokenizer,
    btfc_ner,
    btfc_ner_converter,
    chunkerMapper_fuzzy])


fuzzy_res = mapperPipeline.fit(test_data).transform(test_data)

drug_action_treatment_mapper download started this may take some time.
[ | ][OK!]

In [0]:
fuzzy_res.select(F.explode(F.arrays_zip(fuzzy_res.mappings.result,
                                        fuzzy_res.mappings.metadata)).alias("col"))\
    .select(F.expr("col['1']['entity']").alias("ner_chunk"),
            F.expr("col['1']['__trained__']").alias("fixed_chunk"),
            F.expr("col['0']").alias("action_mapping_result"),
            F.expr("col['1']['relation']").alias("relation ")).show(truncate=False)

+--------------+--------------+---------------------+---------+
ner_chunk |fixed_chunk |action_mapping_result|relation |
+--------------+--------------+---------------------+---------+
dermovae |dermovat |anti-inflammatory |action |
dermovae |dermatovate |anti-inflammatory |action |
dermovae |dermovate |anti-inflammatory |action |
dermovae |dermovel |antiallergic |action |
Warfarina Lusa|Warfarina Lusa|anticoagulant |action |
+--------------+--------------+---------------------+---------+

As seen above, mapper model was able to return mappings based on fuzzy matching even though there is a typo in "dermovae". <br/>

Also, it returned mappings for "Warfarina Lusa" since we set `setAllowMultiTokenChunk(True)` parameter.

### Creating a Mapper Model

There is a `ChunkMapperApproach()` to create your own mapper model. <br/>

This receives an `ner_chunk` and a Json with a mapping of ner entities and relations, and returns the `ner_chunk` augmented with the relations from the Json ontology. <br/> We give the path of json file to the `setDictionary()` parameter.




Let's create an example Json, then create a drug mapper model. This model will match the given drug name (only "metformin" for our example) with correpsonding action and treatment.  

The format of json file should be like following:

In [0]:
data_set = {
  "mappings": [
    {
      "key": "metformin",
      "relations": [
        {
          "key": "action",
          "values" : ["hypoglycemic", "Drugs Used In Diabetes"]
        },
        {
          "key": "treatment",
          "values" : ["diabetes", "t2dm"]
        }
      ]
    }
  ]
}

import json
with open('/dbfs/sample_drug.json', 'w') as f:
    json.dump(data_set, f)

By using `setRels()` parameter, we tell the model which type of mappings we want. In our case, we set the parameter as `setRels(["action", "treatment"])`.

Also, we can manage the case status while creating a model by using `setLowerCase()` parameter.

Let's create a pipeline and see it in action.

In [0]:
document_assembler = nlp.DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = nlp.SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

posology_ner_model = medical.NerModel.pretrained("ner_posology_large", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("posology_ner")

posology_ner_converter = medical.NerConverterInternal()\
    .setInputCols("sentence", "token", "posology_ner")\
    .setOutputCol("posology_ner_chunk")\
    .setWhiteList(["DRUG"])

chunkerMapper = medical.ChunkMapperApproach()\
      .setInputCols(["posology_ner_chunk"])\
      .setOutputCol("mappings")\
      .setDictionary("/sample_drug.json")\
      .setLowerCase(True) \
      .setRel("action") #or treatment

mapper_pipeline = nlp.Pipeline().setStages([
        document_assembler,
        sentence_detector,
        tokenizer, 
        word_embeddings,
        posology_ner_model, 
        posology_ner_converter, 
        chunkerMapper])


empty_data = spark.createDataFrame([[""]]).toDF("text")
mapper_model = mapper_pipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_posology_large download started this may take some time.
[ | ][OK!]

In [0]:
mapper_model.stages[-1].write().overwrite().save("dbfs:/drug_mapper")

In [0]:
#loading the trained model from disk
chunkerMapper = medical.ChunkMapperModel.load("dbfs:/drug_mapper")\
      .setInputCols(["posology_ner_chunk"])\
      .setOutputCol("mappings")\
      .setRels(["action", "treatment"]) \
      .setLowerCase(True)

mapper_pipeline = nlp.Pipeline().setStages([
        document_assembler,
        sentence_detector,
        tokenizer,
        word_embeddings,
        posology_ner_model,
        posology_ner_converter,
        chunkerMapper])


test_data = spark.createDataFrame([["The patient was given 1 unit of Metformin daily."]]).toDF("text")

mapper_model = mapper_pipeline.fit(test_data)
res= mapper_model.transform(test_data)

In [0]:
#ner result
res.select(F.explode('posology_ner_chunk.result').alias("chunks")).show(truncate=False)

+---------+
chunks |
+---------+
Metformin|
+---------+

In [0]:
#mapping result
res.select(F.explode(F.arrays_zip(res.mappings.result,
                                  res.mappings.metadata)).alias("col"))\
    .select(F.expr("col['1']['entity']").alias("ner_chunk"),
            F.expr("col['0']").alias("mapping_result"),
            F.expr("col['1']['relation']").alias("relation"),
            F.expr("col['1']['all_relations']").alias("all_mappings")).show(truncate=False)

+---------+--------------+---------+----------------------+
ner_chunk|mapping_result|relation |all_mappings |
+---------+--------------+---------+----------------------+
Metformin|hypoglycemic |action |Drugs Used In Diabetes|
Metformin|diabetes |treatment|t2dm |
+---------+--------------+---------+----------------------+

As you see, we successfully created our custom mapper model.

### ResolverMerger - Using Sentence Entity Resolver and `ChunkMapperModel` Together

We can merge the results of `ChunkMapperModel` and `SentenceEntityResolverModel` by using `ResolverMerger` annotator.

We can detect our results that fail by `ChunkMapperModel` with `ChunkMapperFilterer` and then merge the resolver and mapper results with `ResolverMerger`

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

ner_model = medical.NerModel.pretrained("ner_posology_greedy", "en", "clinical/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal()\
    .setInputCols("sentence", "token", "ner")\
    .setOutputCol("chunk")

chunkerMapper = medical.ChunkMapperModel.pretrained("rxnorm_mapper", "en", "clinical/models")\
    .setInputCols(["chunk"])\
    .setOutputCol("RxNorm_Mapper")\
    .setRel("rxnorm_code")

cfModel = medical.ChunkMapperFilterer() \
    .setInputCols(["chunk", "RxNorm_Mapper"]) \
    .setOutputCol("chunks_fail") \
    .setReturnCriteria("fail")

chunk2doc = nlp.Chunk2Doc() \
    .setInputCols("chunks_fail") \
    .setOutputCol("doc_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
    .setInputCols(["doc_chunk"])\
    .setOutputCol("sentence_embeddings")\
    .setCaseSensitive(False)

resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented", "en", "clinical/models") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("resolver_code") \
    .setDistanceFunction("EUCLIDEAN")

resolverMerger = medical.ResolverMerger()\
    .setInputCols(["resolver_code","RxNorm_Mapper"])\
    .setOutputCol("RxNorm")

mapper_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        sentence_detector,
        tokenizer,
        word_embeddings,
        ner_model,
        ner_converter,
        chunkerMapper,
        chunkerMapper,
        cfModel,
        chunk2doc,
        sbert_embedder,
        resolver,
        resolverMerger
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = mapper_pipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_posology_greedy download started this may take some time.
[ | ][ / ][ — ][ \ ][OK!]
rxnorm_mapper download started this may take some time.
[ | ][ / ][OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[ | ][OK!]

In [0]:
samples = [['The patient was given Adapin 10 MG, coumadn 5 mg'],
           ['The patient was given Avandia 4 mg, Tegretol, zitiga'] ]

result = model.transform(spark.createDataFrame(samples).toDF("text"))

In [0]:
result.selectExpr('chunk.result as chunk',
                  'RxNorm_Mapper.result as RxNorm_Mapper',
                  'chunks_fail.result as chunks_fail',
                  'resolver_code.result as resolver_code',
                  'RxNorm.result as RxNorm')\
      .show(truncate = False)

+--------------------------------+----------------------+--------------+-------------+------------------------+
chunk |RxNorm_Mapper |chunks_fail |resolver_code|RxNorm |
+--------------------------------+----------------------+--------------+-------------+------------------------+
[Adapin 10 MG, coumadn 5 mg] |[1000049, NONE] |[coumadn 5 mg]|[200883] |[1000049, 200883] |
[Avandia 4 mg, Tegretol, zitiga]|[261242, 203029, NONE]|[zitiga] |[220989] |[261242, 203029, 220989]|
+--------------------------------+----------------------+--------------+-------------+------------------------+

# 8.Pretrained Clinical Pipelines

In order to save you from creating a pipeline from scratch, Spark NLP also has a pre-trained pipelines that are already fitted using certain annotators and transformers according to various use cases.

Here is the list of clinical pre-trained pipelines:

**1.   explain_clinical_doc_carp** :

> A pipeline with `ner_clinical`, `assertion_dl`, `re_clinical` and `ner_posology`. It will extract clinical and medication entities, assign assertion status and find relationships between clinical entities.

**2.   explain_clinical_doc_era** :

> A pipeline with `ner_clinical_events`, `assertion_dl` and `re_temporal_events_clinical`. It will extract clinical entities, assign assertion status and find temporal relationships between clinical entities.

**3.   explain_clinical_doc_ade** :

> A pipeline for `Adverse Drug Events (ADE)` with `ner_ade_biobert`, `assertiondl_biobert`, `classifierdl_ade_conversational_biobert` and `re_ade_biobert`. It will classify the document, extract `ADE` and `DRUG` entities, assign assertion status to `ADE` entities, and relate them with `DRUG` entities, then assign ADE status to a text (`True` means ADE, `False` means not related to ADE).

**letter codes in the naming conventions:**

> c : ner_clinical

> e : ner_clinical_events

> r : relation extraction

> p : ner_posology

> a : assertion

> ade : adverse drug events

**Relation Extraction types:**

`re_clinical` >> TrIP (improved), TrWP (worsened), TrCP (caused problem), TrAP (administered), TrNAP (avoided), TeRP (revealed problem), TeCP (investigate problem), PIP (problems related)

`re_temporal_events_clinical` >> `AFTER`, `BEFORE`, `OVERLAP`

**4. explain_clinical_doc_medication:**

> A pipeline for detecting posology entities with the `ner_posology_large` NER model, assigning their assertion status with `assertion_jsl` model, and extracting relations between posology-related terminology with `posology_re` relation extraction model.


**5. explain_clinical_doc_radiology**

> A pipeline for detecting radiology entities with the `ner_radiology` NER model, assigning their assertion status with `assertion_dl_radiology` model, and extracting relations between the diagnosis, test, and findings with `re_test_problem_finding` relation extraction model.

**6. Clinical Deidentification** :

>This pipeline can be used to deidentify PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `AGE`, `CONTACT`, `DATE`, `ID`, `LOCATION`, `NAME`, `PROFESSION`, `CITY`, `COUNTRY`, `DOCTOR`, `HOSPITAL`, `IDNUM`, `MEDICALRECORD`, `ORGANIZATION`, `PATIENT`, `PHONE`, `PROFESSION`, `STREET`, `USERNAME`, `ZIP`, `ACCOUNT`, `LICENSE`, `VIN`, `SSN`, `DLN`, `PLATE`, `IPADDR` entities.

**7. NER Pipelines:**

> Pipelines for all the available pretrained NER models.

**8. BERT Based NER Pipelines**

> Pipelines for all the available Bert token classification models.

**9. ner_profiling_clinical and ner_profiling_biobert:**

> Pipelines for exploring all the available pretrained NER models at once.

**10. ner_model_finder**

> A pipeline trained with bert embeddings that can be used to find the most appropriate NER model given the entity name.

**11. Resolver Pipelines**

> Pipelines for converting clinical entities to their UMLS CUI codes and medication entities to their ADE, Action, Treatment, UMLS, RxNorm, ICD9, SNOMED and NDC codes.

**12. Oncology Pipelines**

> Pipelines includes Named-Entity Recognition, Assertion Status, Relation Extraction and Entity Resolution models to extract information from oncology texts.


**Also, you can find clinical CODE MAPPING pretrained pipelines in this notebook: [Healthcare Code Mapping Notebook](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/11.1.Healthcare_Code_Mapping.ipynb)**




You can visit [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.Pretrained_Clinical_Pipelines.ipynb) in order to see the usage of all these pipelines.

Now, firstly let's pick the `explain_clinical_doc_carp` and see how it works.

In [0]:
from sparknlp.pretrained import PretrainedPipeline

In [0]:
pipeline = nlp.PretrainedPipeline('explain_clinical_doc_carp', 'en', 'clinical/models')

explain_clinical_doc_carp download started this may take some time.
Approx size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
pipeline.model.stages

Out[170]: [DocumentAssembler_9619f8fd837c,
 SentenceDetector_c0b14c755033,
 REGEX_TOKENIZER_7c616f0cb7a5,
 POS_6f55785005bf,
 dependency_d5a8da6c9093,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_cd5ce67b529f,
 NER_CONVERTER_2f1dcb61b142,
 MedicalNerModel_4a303d875127,
 NER_CONVERTER_a8cff4d56af8,
 ASSERTION_DL_25881ab6309e,
 RelationExtractionModel_9c255241fec3]

In [0]:
# Load pretrained pipeline from local disk:

# >> pipeline_local = PretrainedPipeline.from_disk('/root/cache_pretrained/explain_clinical_doc_carp_en_2.5.5_2.4_1597841630062')

We will annotate our pretrained pipeline with sample data and see the result

In [0]:
text ="""28-year-old female with a history of gestational diabetes mellitus, used to take metformin 1000 mg two times a day, presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .
She was seen by the endocrinology service and discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals.
"""

annotations = pipeline.annotate(text)

annotations.keys()


Out[172]: dict_keys(['sentences', 'clinical_ner_tags', 'document', 'clinical_ner_chunks', 'assertion', 'clinical_relations', 'posology_ner_tags', 'tokens', 'posology_ner_chunks', 'embeddings', 'pos_tags', 'dependencies'])

In [0]:
import pandas as pd

rows = list(zip(annotations['tokens'], annotations['clinical_ner_tags'], annotations['posology_ner_tags'], annotations['pos_tags'], annotations['dependencies']))

df = pd.DataFrame(rows, columns = ['tokens','clinical_ner_tags','posology_ner_tags','POS_tags','dependencies'])

df.head(20)

Out[173]:

,tokens,clinical_ner_tags,posology_ner_tags,POS_tags,dependencies
0,28-year-old,O,O,JJ,female
1,female,O,O,NN,ROOT
2,with,O,O,II,history
3,a,O,O,DD,history
4,history,O,O,NN,female
5,of,O,O,II,history
6,gestational,B-PROBLEM,O,JJ,of
7,diabetes,I-PROBLEM,O,NN,mellitus
8,mellitus,I-PROBLEM,O,NN,gestational
9,",",O,O,NN,used


# 9.Clinical Text Classification

# Classifiers

The below classifiers will be used in this notebook.ClassifierDL, MultiClassifierDL, and GenericClassifier will be trained using healthcare_100d, embeddingd_clinical, and bert sentence embeddings(sbiobert_base_cased_mli). DocumentLogRegClassifier accepts tokens, so sentence embeddings are not utilized during DocumentLogRegClassifier training.

## ClassifierDL

ClassifierDL is a generic Multi-class Text Classification annotator. ClassifierDL uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. The ClassifierDL annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes. For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl).


##  MultiClassifierDL

MultiClassifierDL is a Multi-label Text Classification annotator.MultiClassifierDL uses a Bidirectional GRU with a convolutional model built inside TensorFlow and supports up to 100 classes.  For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#multiclassifierdl).


## GenericClassifier

GenericClassifier is a TensorFlow model for the generic classification of feature vectors in Healthcare  Lİbrary. It takes FEATURE_VECTOR annotations from FeaturesAssembler as input, classifies them, and outputs CATEGORY annotations. Please see [the link](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier) for more information.

## GenericLogRegClassifierApproach

`GenericLogRegClassifierApproach` is a derivative of GenericClassifier which implements a multinomial *Logistic Regression*. This is a single layer neural network with the logistic function at the output. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores varying between 0 and 1. Training data requires "text" and their "label" columns only and the trained model will be a `GenericLogRegClassifierModel()`.


## GenericSVMClassifierApproach

`GenericSVMClassifierApproach` is a derivative of GenericClassifier which implements *SVM (Support Vector Machine)* classification. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1. Taining data requires "text" and their "label" columns only and the trained model will be a `GenericSVMClassifierModel()`

## DocumentLogRegClassifier

DocumentLogRegClassifier is a model to classify documents with a Logarithmic Regression algorithm in Healthcare  Library. Training data requires columns for text and labels. The result is a trained DocumentLogRegClassifierModel. you can get more info [here](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentlogregclassifier).


## DocumentMLClassifierApproach

`DocumentMLClassifierApproach` trains a model to classify documents with a Logistic Regression or Linear SVM algorithms. Training data requires columns for text and their label. The result is a trained GenericClassifierModel.

## MedicalBertForSequenceClassification

`MedicalBertForTokenClassification` annotator can load BERT Models with sequence classification/regression head on top (a linear layer on top of the pooled output) e.g. for multi-class document classification tasks.

Transformer models are compatible with Spark NLP library. So if you want to train and import a transformer based `BertForSequenceClassification` model to Spark NLP, you can do it by using [Import Transformers Into Spark NLP](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/15.Import_Transformers_Into_Spark_NLP.ipynb) notebook. **BUT**, don't forget to use `BertForSequenceClassification` open source annotator while importing.

### Load & Prepare ADE Classification Dataset

In [0]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE-Negative Dataset**

In [0]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

Out[176]:

,text,category
0,Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.,neg
1,"""Retinoic acid syndrome"" was prevented with short-time treatment of high dose (4 x 1.5 g/m2) cytarabine.",neg
2,BACKGROUND: External beam radiation therapy often is avoided in the treatment of rhabdomyosarcoma (RMS) in young children because of the long-term sequelae.,neg
3,"Although the enuresis ceased, she developed throbbing headaches, nausea, vomiting, paresthesia, lethargy, fatigue, and altered mental status over the next 7 days.",neg
4,A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.,neg


**ADE-Positive Dataset**

In [0]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

Out[177]:

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was present, and perhaps enhanced activation of dihydrotachysterol by rifampicin, could have led to increased calcium-release into the circulation.",pos
2,Unaccountable severe hypercalcemia in a patient treated for hypoparathyroidism with dihydrotachysterol.,pos
3,METHODS: We report two cases of pseudoporphyria caused by naproxen and oxaprozin.,pos
4,METHODS: We report two cases of pseudoporphyria caused by naproxen and oxaprozin.,pos


**Merging positive and negative datasets**

In [0]:
ade_df= pd.concat([df_neg, df_pos]).sample(frac=1) #merge and shuffle the data
ade_df.head()

Out[178]:

,text,category
1684,With this initial therapy we have had no treatment failures.,neg
1616,She did not use contact lenses postoperatively.,neg
6247,Use of rituximab and irradiated donor-derived lymphocytes to control Epstein-Barr virus-associated lymphoproliferation in patients undergoing related haplo-identical stem cell transplantation.,neg
591,Succinylcholine-induced hyperkalemia in a patient with mucositis secondary to chemotherapy.,pos
6458,"A patient presented with palmar inflammation and throbbing joint pain in his hands, as well as erythematous, pustular, and micaceous scaling skin on his right foot, legs, elbows and hands.",neg


In [0]:
ade_df["category"].value_counts()

Out[179]: neg 16695
pos 6821 
Name: category, dtype: int64

In [0]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 1684 to 2008
Data columns (total 2 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 text 23516 non-null object
 1 category 23516 non-null object
dtypes: object(2)
memory usage: 551.2+ KB

In [0]:
spark_df = spark.createDataFrame(ade_df)
spark_df.show(3)

+--------------------+--------+
 text|category|
+--------------------+--------+
 With this initia...| neg|
 She did not use ...| neg|
 Use of rituximab...| neg|
+--------------------+--------+
only showing top 3 rows

In [0]:
spark_df.groupBy("category").count().show()
spark_df.printSchema()

+--------+-----+
category|count|
+--------+-----+
 pos| 6821|
 neg|16695|
+--------+-----+

root
-- text: string (nullable = true)
-- category: string (nullable = true)

**Get train & test set**

In [0]:
train_data, test_data = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Train Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Train Dataset Count: 18768
Test Dataset Count: 4748

### DocumentMLClassifier with Logistic Regression

The below pipeline uses the Logistic Regression of `DocumentMLClassifier` by setting `setClassificationModelClass("logreg")`.

In [0]:
document = nlp.DocumentAssembler()\
            .setInputCol("text")\
            .setOutputCol("document")

token = nlp.Tokenizer()\
          .setInputCols("document")\
          .setOutputCol("token")

classifier_logreg = medical.DocumentMLClassifierApproach() \
          .setInputCols("token") \
          .setLabelCol("category") \
          .setOutputCol("prediction") \
          .setClassificationModelClass("logreg")\


pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

logreg_model = pipeline.fit(train_data)
result_logreg = logreg_model.transform(test_data).cache()

In [0]:
result_logreg.show()

+--------------------+--------+--------------------+--------------------+--------------------+
 text|category| document| token| prediction|
+--------------------+--------+--------------------+--------------------+--------------------+
 Acute enlargemen...| neg|[{document, 0, 13...|[{token, 1, 5, Ac...|[{category, 1, 13...|
 After 1 month of...| neg|[{document, 0, 92...|[{token, 1, 5, Af...|[{category, 1, 92...|
Localized pannicu...| pos|[{document, 0, 14...|[{token, 0, 8, Lo...|[{category, 0, 14...|
 BACKGROUND: We r...| neg|[{document, 0, 12...|[{token, 1, 10, B...|[{category, 1, 12...|
 To date, reports...| neg|[{document, 0, 12...|[{token, 1, 2, To...|[{category, 1, 12...|
We report 5 patie...| pos|[{document, 0, 35...|[{token, 0, 1, We...|[{category, 0, 35...|
 Tardive dyskines...| neg|[{document, 0, 36...|[{token, 1, 7, Ta...|[{category, 1, 36...|
We report a case ...| pos|[{document, 0, 18...|[{token, 0, 1, We...|[{category, 0, 18...|
 After clinically...| neg|[{document, 0, 89...|[{token, 1, 5, Af...|[{category, 1, 89...|
The patient exper...| pos|[{document, 0, 98...|[{token, 0, 2, Th...|[{category, 0, 98...|
 The mechanism by...| neg|[{document, 0, 90...|[{token, 1, 3, Th...|[{category, 1, 90...|
 We used telmisar...| neg|[{document, 0, 11...|[{token, 1, 2, We...|[{category, 1, 11...|
The purpose of th...| pos|[{document, 0, 16...|[{token, 0, 2, Th...|[{category, 0, 16...|
Toxic hepatitis d...| pos|[{document, 0, 88...|[{token, 0, 4, To...|[{category, 0, 88...|
 CASE SUMMARY: A ...| neg|[{document, 0, 17...|[{token, 1, 4, CA...|[{category, 1, 17...|
 Type I, blue-bla...| neg|[{document, 0, 27...|[{token, 1, 4, Ty...|[{category, 1, 27...|
 We report the sp...| neg|[{document, 0, 17...|[{token, 1, 2, We...|[{category, 1, 17...|
 Because steroid-...| neg|[{document, 0, 24...|[{token, 1, 7, Be...|[{category, 1, 24...|
 Nocardia asteroi...| neg|[{document, 0, 81...|[{token, 1, 8, No...|[{category, 1, 81...|
Alternating sinus...| pos|[{document, 0, 15...|[{token, 0, 10, A...|[{category, 0, 15...|
+--------------------+--------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result_logreg.printSchema()

root
-- text: string (nullable = true)
-- category: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- prediction: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
result_logreg.select("text","prediction").show(4, truncate=100)

+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
 text| prediction|
+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
 Acute enlargement and subsequent rupture of an abdominal aortic aneurysm in a patient receiving ...|[{category, 1, 134, neg, {sentence -> 0, chunk -> 0, confidence -> 0.9999}, []}]|
 After 1 month of treatment, he was diagnosed as having acute renal failure secondary to HUS.| [{category, 1, 92, neg, {sentence -> 0, chunk -> 0, confidence -> 0.9889}, []}]|
Localized panniculitis and subsequent lipoatrophy with subcutaneous glatiramer acetate (Copaxone)...|[{category, 0, 147, pos, {sentence -> 0, chunk -> 0, confidence -> 1.0000}, []}]|
 BACKGROUND: We report six cases of psychosis in patients with akinetic-rigid syndromes who were ...|[{category, 1, 121, neg, {sentence -> 0, chunk -> 0, confidence -> 0.9999}, []}]|
+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
only showing top 4 rows

In [0]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

precision recall f1-score support

 neg 0.90 0.91 0.91 3382
 pos 0.78 0.74 0.76 1366

 accuracy 0.86 4748
 macro avg 0.84 0.83 0.83 4748
weighted avg 0.86 0.86 0.86 4748

### GenericLogRegClassifier

Firstly, we will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [0]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
trainingData_with_embeddings = embeddings_pipeline.fit(train_data).transform(train_data)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
 text|category| sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
 "St. Anthony's fire" may be due to stimulation of alpha-adrenergic receptors...| neg|[{sentence_embeddings, 0, 107, "St. Anthony's fire" may be due to stimulatio...|
 16 patients with cutaneous or subcutaneous melanoma recurrence on an extremi...| neg|[{sentence_embeddings, 0, 131, 16 patients with cutaneous or subcutaneous me...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
only showing top 2 rows

In [0]:
testData_with_embeddings = embeddings_pipeline.fit(test_data).transform(test_data)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=80)

+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
 text|category| sentence_embeddings|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
 48 h after interrupting treatment, she recovered normal neurological status ...| neg|[{sentence_embeddings, 0, 149, 48 h after interrupting treatment, she recove...|
 A 17-yr-old boy with Crohn's disease and growth retardation developed an acu...| neg|[{sentence_embeddings, 0, 149, A 17-yr-old boy with Crohn's disease and grow...|
+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
only showing top 2 rows

In [0]:
testData_with_embeddings.printSchema()

root
-- text: string (nullable = true)
-- category: string (nullable = true)
-- sentence_embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
log_folder="ADE_logs_healthcare_100d"
!mkdir -p $log_folder

**Creating Graph**

In [0]:
!pip install -q tensorflow==2.11.0
!pip install -q tensorflow-addons

WARNING: You are using pip version 21.0.1; however, version 23.2.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
 WARNING: You are using pip version 21.0.1; however, version 23.2.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "file:/dbfs/gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [0]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [0]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: file:/dbfs/gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to file:/dbfs/gc_graph/log_reg_graph.pb

In [0]:
!cat $log_folder/GenericLogRegClassifierApproach*

cat: 'ADE_logs_healthcare_100d/GenericLogRegClassifierApproach*': No such file or directory

In [0]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [0]:
pred_df.printSchema()
pred_df.select(pred_df.prediction).show(5, truncate=False)
pred_df.select(pred_df.category, pred_df.prediction.result).show(5, truncate=False)

root
-- text: string (nullable = true)
-- category: string (nullable = true)
-- sentence_embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- feature_vector: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- prediction: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

+-------------------------------------------------------+
prediction |
+-------------------------------------------------------+
[{category, 0, 0, neg, {confidence -> 0.74633425}, []}]|
[{category, 0, 0, neg, {confidence -> 0.7310879}, []}] |
[{category, 0, 0, neg, {confidence -> 0.8143567}, []}] |
[{category, 0, 0, neg, {confidence -> 0.79353786}, []}]|
[{category, 0, 0, neg, {confidence -> 0.66491276}, []}]|
+-------------------------------------------------------+
only showing top 5 rows

+--------+-----------------+
category|prediction.result|
+--------+-----------------+
neg |[neg] |
neg |[neg] |
neg |[neg] |
neg |[neg] |
neg |[neg] |
+--------+-----------------+
only showing top 5 rows

In [0]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

precision recall f1-score support

 neg 0.78 0.93 0.85 3382
 pos 0.66 0.36 0.46 1366

 accuracy 0.76 4748
 macro avg 0.72 0.64 0.66 4748
weighted avg 0.75 0.76 0.74 4748

### Pretrained Clinical Text Classification Models

Spark NLP for Healthcare serves a variety of pretrained clinical text classification models. <br/>

In this notebook, we will cover some example pipelines with pretrained text classification models.

#### **`genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli`**

This Generic Classifier model is intended for detecting alcohol use in clinical notes and trained by using GenericClassifierApproach annotator. <br/>

`Present`: if the patient was a current consumer of alcohol. <br/>
`Past`: the patient was a consumer in the past and had quit. <br/>
`Never`: if the patient had never consumed alcohol. <br/>
`None`: if there was no related text.

In [0]:
from pyspark.sql.types import StringType

document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_embeddings = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

generic_classifier = medical.GenericClassifierModel.pretrained("genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli", 'en', 'clinical/models')\
    .setInputCols(["features"])\
    .setOutputCol("class")

pipeline = nlp.Pipeline(stages=[
    document_assembler,
    sentence_embeddings,
    features_asm,
    generic_classifier
])

text_list = ["Retired schoolteacher, now substitutes. Lives with wife in location 1439. Has a 27 yo son and a 25 yo daughter. He uses alcohol and cigarettes",
             "The patient quit smoking approximately two years ago with an approximately a 40 pack year history, mostly cigar use. He also reports 'heavy alcohol use', quit 15 months ago.",
             "Employee in neuro departmentin at the Center Hospital 18. Widower since 2001. Current smoker since 20 years. No EtOH or illicits.",
             "Patient smoked 4 ppd x 37 years, quitting 22 years ago. He is widowed, lives alone, has three children."]

df = spark.createDataFrame(text_list, StringType()).toDF("text")

result = pipeline.fit(df).transform(df)

result.select("text", "class.result").show(truncate=100)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli download started this may take some time.
[ | ][ / ][ — ][ \ ][OK!]
+----------------------------------------------------------------------------------------------------+---------+
 text| result|
+----------------------------------------------------------------------------------------------------+---------+
Retired schoolteacher, now substitutes. Lives with wife in location 1439. Has a 27 yo son and a 2...|[Present]|
The patient quit smoking approximately two years ago with an approximately a 40 pack year history...| [Past]|
Employee in neuro departmentin at the Center Hospital 18. Widower since 2001. Current smoker sinc...| [Never]|
Patient smoked 4 ppd x 37 years, quitting 22 years ago. He is widowed, lives alone, has three chi...| [None]|
+----------------------------------------------------------------------------------------------------+---------+

#### **`bert_sequence_classifier_sdoh_community_present_status`**

`bert_sequence_classifier_sdoh_community_present_status` classifies the clinical texts related to social support such as a family member or friend in the clinical documents. A discharge summary was classified `True` for Community-Present if the discharge summary had passages related to active social support and `False` if such passages were not found in the discharge summary.

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

sequenceClassifier = medical.BertForSequenceClassification.pretrained("bert_sequence_classifier_sdoh_community_present_status", "en", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("classes")

pipeline = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier
])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

data = spark.createDataFrame([["Right inguinal hernia repair in childhood Cervical discectomy 3 years ago Umbilical hernia repair 2137. Retired schoolteacher, now substitutes. Lives with wife in location 1439. Has a 27 yo son and a 25 yo daughter. Name (NI) past or present smoking hx, no EtOH."],
                              ["Atrial Septal Defect with Right Atrial Thrombus Pulmonary Hypertension Obesity, Obstructive Sleep Apnea. Denies tobacco and ETOH. Works as cafeteria worker."]]).toDF("text")

result = model.transform(data)

result.select("text", "classes.result").show(truncate=False)

bert_sequence_classifier_sdoh_community_present_status download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
text |result |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
Right inguinal hernia repair in childhood Cervical discectomy 3 years ago Umbilical hernia repair 2137. Retired schoolteacher, now substitutes. Lives with wife in location 1439. Has a 27 yo son and a 25 yo daughter. Name (NI) past or present smoking hx, no EtOH.|[True] |
Atrial Septal Defect with Right Atrial Thrombus Pulmonary Hypertension Obesity, Obstructive Sleep Apnea. Denies tobacco and ETOH. Works as cafeteria worker. |[False]|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+

#### **`classifierdl_ade_biobert`**

`classifierdl_ade_biobert` classifies text/sentence in two categories: <br/>

`True` : The sentence is talking about a possible ADE <br/>
`False` : The sentences doesn’t have any information about an ADE.

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained('biobert_pubmed_base_cased')\
    .setInputCols(["document", 'token'])\
    .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classifier = nlp.ClassifierDLModel.pretrained('classifierdl_ade_biobert', 'en', 'clinical/models')\
    .setInputCols(['sentence_embeddings'])\
    .setOutputCol('class')

nlp_pipeline = nlp.Pipeline(stages=[document_assembler,
                                tokenizer,
                                embeddings,
                                sentence_embeddings,
                                classifier])


data = spark.createDataFrame([["I feel a bit drowsy & have a little blurred vision after taking an insulin"], ["I feel great after taking tylenol"]]).toDF("text")

model = nlp_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
result = model.transform(data)

result.select("text", "class.result").show(truncate=False)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
classifierdl_ade_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
+--------------------------------------------------------------------------+-------+
text |result |
+--------------------------------------------------------------------------+-------+
I feel a bit drowsy & have a little blurred vision after taking an insulin|[True] |
I feel great after taking tylenol |[False]|
+--------------------------------------------------------------------------+-------+

#### **`classifierdl_gender_biobert`**

`classifierdl_gender_biobert` classifies the gender of the patient in the clinical document using context.

In [0]:
document_assembler = nlp.DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
  .setInputCols(['document'])\
  .setOutputCol('token')

biobert_embeddings = nlp.BertEmbeddings().pretrained("biobert_pubmed_base_cased") \
  .setInputCols(["document", "token"])\
  .setOutputCol("bert_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
  .setInputCols(["document", "bert_embeddings"]) \
  .setOutputCol("sentence_bert_embeddings") \
  .setPoolingStrategy("AVERAGE")

genderClassifier = nlp.ClassifierDLModel.pretrained("classifierdl_gender_biobert", "en", "clinical/models") \
  .setInputCols(["sentence_bert_embeddings"]) \
  .setOutputCol("gender")

nlp_pipeline = nlp.Pipeline(stages=[document_assembler,
                                tokenizer, biobert_embeddings,
                                sentence_embeddings,
                                genderClassifier])



data = spark.createDataFrame([["social history: shows that  does not smoke cigarettes or drink alcohol, lives in a nursing home. family history: shows a family history of breast cancer"],
                              ["The patient took Advil and he experienced an adverse reaction."]]).toDF("text")

model = nlp_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
result = model.transform(data)

result.select("text", "gender.result").show(truncate=False)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
classifierdl_gender_biobert download started this may take some time.
Approximate size to download 21 MB
[ | ][ / ][ — ][ \ ][ | ][OK!]
+--------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
text |result |
+--------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
social history: shows that does not smoke cigarettes or drink alcohol, lives in a nursing home. family history: shows a family history of breast cancer|[Female]|
The patient took Advil and he experienced an adverse reaction. |[Male] |
+--------------------------------------------------------------------------------------------------------------------------------------------------------+--------+

# 9.Serving Spark NLP with API: Fast API with LightPipelines

<b><h1><font color='darkred'>!!! ATTENTION !!! </font><h1><b>

<b>Before continue, <font color='darkred'> RESET the COLAB RUNTIME, INSTALL the LIBRARIES  </font> and go ahead because of the memory issue.<b>

In [0]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

## Using Fast API and LightPipeline

You can serve SparkNLP + FastAPI on Docker. To do that, we will create a project with the following files:

`Dockerfile`: Image for creating a SparkNLP + FastAPI Docker image <br/>
`requirements.txt`: PIP Requirements <br/>
`entrypoint.sh`: Dockerfile entrypoint <br/>
`content/`: folder containing FastAPI webapp and SparkNLP keys <br/>
`content/main.py`: FastAPI webapp, entrypoint <br/>
`content/sparknlp_keys.json`: SparkNLP keys (for Healthcare or OCR) <br/>

### Dockerfile

`Dockerfile`: Image for creating a SparkNLP + FastAPI Docker image

In [0]:
"""
FROM ubuntu:18.04
RUN apt-get update && apt-get -y update

RUN apt-get -y update \
    && apt-get install -y wget \
    && apt-get install -y jq \
    && apt-get install -y lsb-release \
    && apt-get install -y openjdk-8-jdk-headless \
    && apt-get install -y build-essential python3-pip \
    && pip3 -q install pip --upgrade \
    && apt-get clean && rm -rf /var/lib/apt/lists/* /tmp/* /var/tmp/* \
         /usr/share/man /usr/share/doc /usr/share/doc-base

ENV PYSPARK_DRIVER_PYTHON=python3
ENV PYSPARK_PYTHON=python3

ENV LC_ALL=C.UTF-8
ENV LANG=C.UTF-8

EXPOSE 8515

COPY requirements.txt /
RUN pip install -r /requirements.txt

COPY entrypoint.sh /
RUN chmod +x /entrypoint.sh

COPY ./content/ /content/
WORKDIR content/

ENTRYPOINT ["/entrypoint.sh"]
"""

### Other files of the project

- `requirements.txt`: PIP Requirements

In [0]:
"""
pyspark==3.1.2
fastapi==0.70.1
uvicorn==0.16
wget==3.2
pandas
"""

- `entrypoint.sh`: Dockerfile entrypoint

In [0]:
"""
#!/bin/bash

export_json () {
    for s in $(echo $values | jq -r 'to_entries|map("\(.key)=\(.value|tostring)")|.[]' $1 ); do
        export $s
    done
}

export_json "/content/sparknlp_keys.json"

pip install --upgrade spark-nlp-jsl==$JSL_VERSION --user --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

if [ $? != 0 ];
then
    exit 1
fi

python3 /content/main.py
"""

### Example to serve 2 pipelines
We are going to download and store in memory two pipelines: `ner_profiling_clinical` and `clinical_deidentification`. This will reduce the latency of loading the models every time.

- `content/main.py`: FastAPI webapp, entrypoint

In [0]:
from fastapi import FastAPI
import uvicorn
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from johnsnowlabs import *


app = FastAPI()
event_list = dict()
pipelines = {}

@app.get("/benchmark/pipeline")
async def get_one_sequential_pipeline_result(modelname, text=''):
    # print(list(pipelines.keys()))
    if modelname is None or modelname not in pipelines.keys():
        return json.dumps({'error': f"{modelname} not in loaded list: {list(pipelines.keys())}"})

    return pipelines[modelname].annotate(text)

@app.on_event("startup")
async def startup_event():
    if 'pipeline_loaded' in event_list:
        return
    event_list['start_up']=datetime.now()
    print(f'startup has been started at {datetime.now()}...', )
    print(list(pipelines.keys()))

    print(f'****** spark nlp healthcare version fired up {datetime.now()} ******')
    event_list['sparknlp_fired']=datetime.now()

    print("- App started.")
    from google.colab import files
    print('Please Upload your John Snow Labs License using the button below')
    license_keys = files.upload()

    nlp.install()
    spark = nlp.start()

    print(f'****** Loading pretrained pipelines fired up {datetime.now()} ****** ')
    pipelines['ner_profiling_clinical'] = nlp.PretrainedPipeline('ner_profiling_clinical', 'en', 'clinical/models')
    pipelines['clinical_deidentification'] = nlp.PretrainedPipeline("clinical_deidentification", "en", "clinical/models")
    event_list['pipeline_loaded'] = datetime.now()

    print(event_list)

if __name__ == "__main__":
    uvicorn.run('main:app', host='0.0.0.0', port=8515)

### Keys file

...and last, but not least, add your sparknlp_keys.json to `content/sparknlp_keys.json`!
Don't forget to fulfill with your license values.

In [0]:
'''
{
  "AWS_ACCESS_KEY_ID": "",
  "AWS_SECRET_ACCESS_KEY": "",
  "SECRET": "",
  "SPARK_NLP_LICENSE": "",
  "JSL_VERSION": "",
  "PUBLIC_VERSION": ""
}
'''

### Building and running Docker
Spin up a Docker container using the SparkNLP+FastAPI Docker image we created before

In [0]:
"""
docker build -t johnsnowlabs/sparknlp:sparknlp_api .
docker run -v jsl_keys.json:/content/sparknlp_keys.json -p 8515:8515 -it johnsnowlabs/sparknlp:sparknlp_api
"""

### Consuming the API from a Python Script

Use this code to query the API either sequentially (1 call at a time) or sending N concurrent calls using ThreadPoolExecutor

In [0]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor

ner_text = """
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting. The patient was prescribed 1 capsule of Advil 10 mg for 5 days and magnesium hydroxide 100mg/1ml suspension PO.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

modelname = 'clinical_deidentification'
# modelname = 'ner_profiling_clinical'

def get_url(args):
    res = requests.get(args[0])
    return res

In [0]:
# 1 call
# ==================
query = f"?modelname={modelname}&text={ner_text}"
url = f"http://localhost:8515/benchmark/pipeline{query}"

print(get_url([url]))

In [0]:
# N calls in parallel
# ==================
list_of_urls = []

N_CALLS = 10
for i in range(0, N_CALLS):
  list_of_urls.append((url, i))

with ThreadPoolExecutor() as pool:
  response_list = list(pool.map(get_url, list_of_urls))
  print(response_list)

# 10.Serving Spark NLP with API: Synapse ML

<b><h1><font color='darkred'>!!! ATTENTION !!! </font><h1><b>

<b>Before continue, <font color='darkred'>FACTORY RESET the COLAB RUNTIME</font> and go ahead.<b>

In [0]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [0]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [0]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [0]:
import pandas as pd
import json
import string
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark.sql as SQL
from pyspark import keyword_only
from pyspark.ml import Pipeline, PipelineModel

import time
import requests
import uuid
import json
import requests
from concurrent.futures import ThreadPoolExecutor

# Automatically load license data and start a session with all jars user has access to

spark = nlp.start()

In [0]:
import synapse.ml
from synapse.ml.io import *

### Preparing a pipeline with Entity Resolution

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
document_assembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

# Sentence Detector DL annotator, processes various sentences per line
sentenceDetectorDL = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare", "en", 'clinical/models') \
      .setInputCols(["document"]) \
      .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

# WordEmbeddingsModel pretrained "embeddings_clinical" includes a model of 1.7Gb that needs to be downloaded
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("word_embeddings")

# Named Entity Recognition for clinical concepts.
clinical_ner = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "word_embeddings"]) \
      .setOutputCol("ner")

ner_converter_icd = medical.NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['PROBLEM'])\
      .setPreservePosition(False)

c2doc = nlp.Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc")

sbert_embedder = nlp.BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sentence_embeddings")\
      .setCaseSensitive(False)

icd_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
     .setInputCols(["ner_chunk"]) \
     .setOutputCol("icd10cm_code")\
     .setDistanceFunction("EUCLIDEAN")


# Build up the pipeline
resolver_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        sentenceDetectorDL,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter_icd,
        c2doc,
        sbert_embedder,
        icd_resolver
  ])

resolver_p_model = resolver_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

In [0]:
clinical_note = """A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years
    prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior
    episode of HTG-induced pancreatitis three years prior to presentation, associated
    with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2,
    presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.
    Two weeks prior to presentation, she was treated with a five-day course of amoxicillin
    for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin
    for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months
    at the time of presentation. Physical examination on presentation was significant for dry oral mucosa;
    significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent
    laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20,
    creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c)
    10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed
    as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for
    starvation ketosis, as she reported poor oral intake for three days prior to admission. However,
    serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap
    was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and
    lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L -
    the original sample was centrifuged and the chylomicron layer removed prior to analysis due to
    interference from turbidity caused by lipemia again. The patient was treated with an insulin drip
    for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within
    24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting
    of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on
    40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg
    two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She
    had close follow-up with endocrinology post discharge."""


data = spark.createDataFrame([[clinical_note]]).toDF("text")

### Creating a JSON file with the clinical note
Since SynapseML runs a webservice that accepts HTTP calls with json format

In [0]:
data_json = {"text": clinical_note }

### Running a Synapse server

In [0]:
serving_input = spark.readStream.server() \
    .address("localhost", 9999, "benchmark_api") \
    .option("name", "benchmark_api") \
    .load() \
    .parseRequest("benchmark_api", data.schema)

serving_output = resolver_p_model.transform(serving_input) \
    .makeReply("icd10cm_code")

server = serving_output.writeStream \
      .server() \
      .replyTo("benchmark_api") \
      .queryName("benchmark_query") \
      .option("checkpointLocation", "file:///tmp/checkpoints-{}".format(uuid.uuid1())) \
      .start()

In [0]:
def post_url(args):
    print(f"- Request {str(args[2])} launched!")
    res = requests.post(args[0], data=args[1])
    print(f"**Response {str(args[2])} received**")
    return res

# If you want to send parallel calls, just add more tuples to list_of_urls array
# tuple: (URL from above, json, number_of_call)
list_of_urls = [("http://localhost:9999/benchmark_api",json.dumps(data_json), 0)]

with ThreadPoolExecutor() as pool:
    response_list = list(pool.map(post_url,list_of_urls))

### Checking Results

In [0]:
for i in range (0, len(response_list[0].json())):
  print(response_list[0].json()[i]['result'])